In [1]:
import pandas as pd

file_path = 'mixalldata_clean.csv'
df = pd.read_csv(file_path, nrows=1)

print("Column names in the dataset:")
print(df.columns)

Column names in the dataset:
Index(['type', 'sendTime', 'sender', 'senderPseudo', 'messageID', 'class',
       'posx', 'posy', 'posz', 'posx_n', 'posy_n', 'posz_n', 'spdx', 'spdy',
       'spdz', 'spdx_n', 'spdy_n', 'spdz_n', 'aclx', 'acly', 'aclz', 'aclx_n',
       'acly_n', 'aclz_n', 'hedx', 'hedy', 'hedz', 'hedx_n', 'hedy_n',
       'hedz_n'],
      dtype='object')


In [12]:
import pandas as pd

# Load the dataset
file_path = 'mixalldata_clean.csv'
df = pd.read_csv(file_path)

# Specify numeric columns
numeric_columns = [
    'sendTime', 'sender', 'senderPseudo', 'posx', 'posy', 'posz', 'posx_n', 'posy_n', 'posz_n',
    'spdx', 'spdy', 'spdz', 'spdx_n', 'spdy_n', 'spdz_n',
    'aclx', 'acly', 'aclz', 'aclx_n', 'acly_n', 'aclz_n',
    'hedx', 'hedy', 'hedz', 'hedx_n', 'hedy_n', 'hedz_n', 'class'
]
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Sort by 'sender' and 'sendTime' for time-sequential ordering within each sender
df = df.sort_values(by=['senderPseudo', 'sendTime']).reset_index(drop=True)

# Separate filtered records into misbehavior and normal classes
misbehavior_records = df[df['class'] != 0]
normal_records = df[df['class'] == 0]

# Sample 2000 records per misbehavior class, ensuring each sender has at least 5 occurrences
misbehavior_sample = (
    misbehavior_records.groupby('class')
    .apply(lambda x: x.head(2500))
    .reset_index(drop=True)
)

# Calculate the remaining sample size needed from normal records to reach 100,000 total
normal_sample_size = 125000 - len(misbehavior_sample)

# Sample normal records while ensuring each sender occurs at least 5 times
normal_sample = (
    normal_records.groupby('sender')
    .apply(lambda x: x.head(5))  # Take at least 5 records per sender
    .reset_index(drop=True)
)

# If additional normal records are needed, sample from remaining normal records
additional_normal_records_needed = normal_sample_size - len(normal_sample)
if additional_normal_records_needed > 0:
    remaining_normal_sample = (
        normal_records[~normal_records.index.isin(normal_sample.index)]
        .sample(n=additional_normal_records_needed, random_state=42)
    )
    normal_sample = pd.concat([normal_sample, remaining_normal_sample])

# Combine misbehavior and normal samples, and sort by 'sendTime' for a sequential dataset
final_df = pd.concat([normal_sample, misbehavior_sample]).sort_values(by='sendTime').reset_index(drop=True)

# Display final class distribution to verify balance and sender occurrence condition
print("Class distribution in final dataset:")
print(final_df['class'].value_counts())

# Verify each sender appears at least 5 times in the final dataset
sender_occurrences = final_df['sender'].value_counts()
print("All senders have at least 5 occurrences:", (sender_occurrences >= 5).all())

Class distribution in final dataset:
0     85989
18     2500
1      2500
2      2500
3      2500
4      2500
5      2500
6      2500
7      2500
8      2500
19     2500
10     2500
11     2500
12     2500
13     2500
14     2500
15     2500
16     2500
17     2500
9      2500
Name: class, dtype: int64
All senders have at least 5 occurrences: False


In [10]:
# Find all distinct (unique) values in the 'OBU_device_ID' column
distinct_senders = final_df['sender'].unique()

print("Distinct senders:")
print(distinct_senders)
print("Total senders: " + str(len(distinct_senders)))

Distinct senders:
[     9     15     21 ... 147969 147963 147981]
Total senders: 18628


In [11]:
# Display the first 10 records with sender and sendTime
pd.set_option('display.float_format', '{:.0f}'.format)
print(df[['sender', 'sendTime']].head(10))

   sender  sendTime
0    1491     16281
1    1491     16281
2    1491     16282
3    1491     16282
4    1491     16282
5    1491     16282
6    1491     16282
7    1491     16283
8    1491     16283
9    1491     16283


In [6]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.callbacks import EarlyStopping

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']

X = df[features].values
Y = df['class'].astype(int).values

scaler = StandardScaler()
X = scaler.fit_transform(X)

sequence_length = len(features)
X = X.reshape(-1, sequence_length, 1)

num_classes = len(np.unique(Y))
Y = to_categorical(Y, num_classes=num_classes)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = Sequential([
    Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(sequence_length, 1)),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=2, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Y_pred = model.predict(X_test)

Y_pred_binary = Y_pred.argmax(axis=1)
Y_test_labels = Y_test.argmax(axis=1)

accuracy = accuracy_score(Y_test_labels, Y_pred_binary)
precision = precision_score(Y_test_labels, Y_pred_binary, average='weighted')
recall = recall_score(Y_test_labels, Y_pred_binary, average='weighted')
f1 = f1_score(Y_test_labels, Y_pred_binary, average='weighted')

end_time = time.time()
elapsed_time = end_time - start_time

print(f"CNN Accuracy: {accuracy:.4f}")
print(f"CNN Precision: {precision:.4f}")
print(f"CNN Recall: {recall:.4f}")
print(f"CNN F1-score: {f1:.4f}")
print(f"\nTraining time: {elapsed_time:.6f} seconds")

C:\Users\david\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 202s 3ms/step - accuracy: 0.6951 - loss: 1.2286 - val_accuracy: 0.7220 - val_loss: 1.0981
Epoch 2/10
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 207s 3ms/step - accuracy: 0.7263 - loss: 1.0749 - val_accuracy: 0.7286 - val_loss: 1.0611
Epoch 3/10
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 216s 3ms/step - accuracy: 0.7309 - loss: 1.0504 - val_accuracy: 0.7309 - val_loss: 1.0493
Epoch 4/10
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 212s 3ms/step - accuracy: 0.7329 - loss: 1.0371 - val_accuracy: 0.7325 - val_loss: 1.0400
Epoch 5/10
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 222s 3ms/step - accuracy: 0.7345 - loss: 1.0295 - val_accuracy: 0.7334 - val_loss: 1.0359
Epoch 6/10
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 224s 4ms/step - accuracy: 0.7354 - loss: 1.0252 - val_accuracy: 0.7327 - val_loss: 1.0328
Epoch 7/10
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 223s 3ms/step - accuracy: 0.7360 - loss: 1.0205 - val_accuracy: 0.7342 - val_loss: 1.0263
Epoch 8/10
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 225s 4ms/step - ac

C:\Users\david\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CNN Accuracy: 0.7356
CNN Precision: 0.6688
CNN Recall: 0.7356
CNN F1-score: 0.6651

Training time: 2226.826862 seconds


In [7]:
import numpy as np
import time

start_time = time.time()
random_index = np.random.randint(0, len(X_test))

sample = X_test[random_index].reshape(1, -1, 1)
true_label = Y_test[random_index]

predicted_label = model.predict(sample)

predicted_class = predicted_label.argmax(axis=1)[0]

print("Randomly selected sample:")
print(f"True label: {true_label}")
print(f"Predicted label (class): {predicted_class}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nPrediction time: {elapsed_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Randomly selected sample:
True label: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Predicted label (class): 0

Prediction time: 0.079967 seconds


In [24]:
import numpy as np
import time
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# 500-sample testing program 2
TIME_THRESHOLD = 0.1
total_samples_processed = 500
predictions_above_threshold = 0

all_true_labels = []
all_predicted_labels = []
prediction_times = []

if Y_test.ndim > 1:
    Y_test = np.argmax(Y_test, axis=1)

for i in range(total_samples_processed):
    start_time = time.time()
    
    random_index = np.random.randint(0, len(X_test))

    sample = X_test[random_index].reshape(1, -1, 1)
    true_label = Y_test[random_index]

    predicted_label = model.predict(sample)

    predicted_class = predicted_label.argmax(axis=1)[0]

    all_true_labels.append(true_label)
    all_predicted_labels.append(predicted_class)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    prediction_times.append(elapsed_time)

    if elapsed_time >= TIME_THRESHOLD:
        predictions_above_threshold += 1

conf_matrix = confusion_matrix(all_true_labels, all_predicted_labels)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    return precision, recall, f1

precision_per_class, recall_per_class, f1_per_class = calculate_metrics(conf_matrix)
avg_precision = np.mean(precision_per_class)
avg_recall = np.mean(recall_per_class)
avg_f1 = np.mean(f1_per_class)

average_prediction_time = np.mean(prediction_times)

print(f"Classification Accuracy: {np.mean(np.array(all_true_labels) == np.array(all_predicted_labels)):.2f}")
print(f"Classification Precision (avg): {avg_precision:.2f}")
print(f"Classification Recall (avg): {avg_recall:.2f}")
print(f"Classification F1-Score (avg): {avg_f1:.2f}")
print(f"Average Prediction Time: {average_prediction_time:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━

C:\Users\david\AppData\Local\Temp\ipykernel_12828\3578208992.py:46: RuntimeWarning: invalid value encountered in divide
  precision = np.nan_to_num(tp / (tp + fp))
C:\Users\david\AppData\Local\Temp\ipykernel_12828\3578208992.py:48: RuntimeWarning: invalid value encountered in divide
  f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))


In [3]:
import numpy as np
import pandas as pd
import os

# Constants
FAC_IMAGE_WIDTH = 2
FAC_IMAGE_HEIGHT = 2
FAC_NUM_CHANNELS = 12
FAC_SPECIAL_VALUE = 255
NUM_CLASSES = 20

def normalize_and_scale(values, min_values, max_values):
    range_values = max_values - min_values
    range_values[range_values == 0] = 1
    scaled = 255 * (values - min_values) / range_values
    scaled[np.isnan(values) | ~np.isfinite(values)] = FAC_SPECIAL_VALUE
    scaled[values == max_values] = 0
    return np.clip(scaled, 0, 255).astype(np.uint8)

def generate_image(group, misbehavior_index=None):
    min_values = np.array([group[col].min() for col in final_numeric_columns])
    max_values = np.array([group[col].max() for col in final_numeric_columns])

    data = group[final_numeric_columns].to_numpy()
    normalized_data = normalize_and_scale(data, min_values, max_values)

    reshaped_data = normalized_data.reshape(FAC_IMAGE_HEIGHT, FAC_IMAGE_WIDTH, FAC_NUM_CHANNELS)

    if misbehavior_index:
        row = (misbehavior_index - 1) // FAC_IMAGE_WIDTH
        col = (misbehavior_index - 1) % FAC_IMAGE_WIDTH
        reshaped_data[row, col, :3] = [255, 0, 0]
        
    return reshaped_data

final_numeric_columns = [
    'sendTime', 'senderPseudo', 'posx', 'posy', 'posx_n', 'posy_n',
    'spdx_n', 'spdy_n', 'aclx_n', 'acly_n', 'hedx_n', 'hedy_n'
]

output_dir = 'veremi_multilevel_images_2x2_ea'
os.makedirs(output_dir, exist_ok=True)

normal_messages = final_df[final_df['class'] == 0]

classification_results = []
image_counter = 1

for _ in range(8000):
    sampled_group = normal_messages.sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
    image_data = generate_image(sampled_group)
    npy_name = f'{output_dir}/image_{image_counter}.npy'
    np.save(npy_name, image_data)
    classification_results.append({'file_name': npy_name, 'classification': 0, 'misbehavior_index': 0})
    image_counter += 1

for misbehavior_class in range(1, 20):
    for _ in range(600):
        attack_group = final_df[final_df['class'] == misbehavior_class].sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
        misbehavior_index = np.random.randint(1, FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT + 1)
        image_data = generate_image(attack_group, misbehavior_index)
        npy_name = f'{output_dir}/image_{image_counter}.npy'
        np.save(npy_name, image_data)
        classification_results.append({'file_name': npy_name, 'classification': misbehavior_class, 'misbehavior_index': misbehavior_index})
        image_counter += 1

classification_df = pd.DataFrame(classification_results)
classification_df.to_csv('multiple_class_2x2_ea.csv', index=False)

print("Images generated and saved to 'multiple_class_2x2_ea.csv'.")

Images generated and saved to 'multiple_class_2x2_ea.csv'.


In [8]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (2, 2, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_2x2_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_2x2_ea.csv')

fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X = []
    Y_classification = []
    Y_index = []
    
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path)
            image_array = image_array / 255.0

            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)

    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

X_train, X_test, Y_train_classification, Y_test_classification, Y_train_index, Y_test_index = train_test_split(
    X, Y_classification, Y_index, test_size=0.2, random_state=42)
input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))

x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

classification_output = Dense(20, activation='softmax', name='classification_output')(x)

index_output = Dense(26, activation='softmax', name='index_output')(x)

model = Model(inputs=input_layer, outputs=[classification_output, index_output])

model.compile(optimizer='adam', 
              loss={'classification_output': 'sparse_categorical_crossentropy', 
                    'index_output': 'sparse_categorical_crossentropy'},
              metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, 
          {'classification_output': Y_train_classification, 'index_output': Y_train_index}, 
          epochs=FAC_EPOCHS, 
          batch_size=FAC_BATCH_SIZE, 
          validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
          callbacks=[early_stopping])

predictions = model.predict(X_test)
Y_pred_classification = np.argmax(predictions[0], axis=1)
Y_pred_index = np.argmax(predictions[1], axis=1)

accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)

accuracy_index = accuracy_score(Y_test_index, Y_pred_index)
f1_classification_manual = 2 * (precision_classification * recall_classification) / (precision_classification + recall_classification)

print("Classification Accuracy:", accuracy_classification)
print("Classification Precision:", precision_classification)
print("Classification Recall:", recall_classification)
print("Classification F1-score (sklearn):", f1_classification)
print("Classification F1-score (manual calculation):", f1_classification_manual)

print("Misbehavior Index Accuracy:", accuracy_index)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

save_dir = "models"
os.makedirs(save_dir, exist_ok=True)

model_save_path = os.path.join(save_dir, "veremi_images_m_2x2_ea_i.keras")
model.save(model_save_path)

print(f"\nModel saved at: {model_save_path}")

Epoch 1/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - classification_output_accuracy: 0.3914 - classification_output_loss: 2.2585 - index_output_accuracy: 0.8077 - index_output_loss: 0.7680 - loss: 3.0265 - val_classification_output_accuracy: 0.4229 - val_classification_output_loss: 2.2636 - val_index_output_accuracy: 0.6912 - val_index_output_loss: 0.7123 - val_loss: 2.9756
Epoch 2/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - classification_output_accuracy: 0.4714 - classification_output_loss: 1.7661 - index_output_accuracy: 0.9320 - index_output_loss: 0.1973 - loss: 1.9634 - val_classification_output_accuracy: 0.4642 - val_classification_output_loss: 1.8280 - val_index_output_accuracy: 0.9412 - val_index_output_loss: 0.2213 - val_loss: 2.0452
Epoch 3/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - classification_output_accuracy: 0.4920 - classification_output_loss: 1.6666 - index_output_accuracy: 0.9476 - index_output_loss: 0.1495 - loss: 1.8161 - val_classification_output_accurac

In [3]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (2, 2, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_2x2_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_2x2_ea.csv')
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X, Y_classification, Y_index = [], [], []
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path) / 255.0
            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)
    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accs, precs, recs, f1s, idx_accs = [], [], [], [], []
start_time = time.time()
fold = 1

for train_index, test_index in skf.split(X, Y_classification):
    X_train, X_test = X[train_index], X[test_index]
    Y_train_classification, Y_test_classification = Y_classification[train_index], Y_classification[test_index]
    Y_train_index, Y_test_index = Y_index[train_index], Y_index[test_index]

    input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))
    x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)

    classification_output = Dense(20, activation='softmax', name='classification_output')(x)
    index_output = Dense(26, activation='softmax', name='index_output')(x)

    model = Model(inputs=input_layer, outputs=[classification_output, index_output])
    model.compile(optimizer='adam',
                  loss={'classification_output': 'sparse_categorical_crossentropy',
                        'index_output': 'sparse_categorical_crossentropy'},
                  metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(X_train,
              {'classification_output': Y_train_classification, 'index_output': Y_train_index},
              epochs=FAC_EPOCHS,
              batch_size=FAC_BATCH_SIZE,
              validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
              callbacks=[early_stopping],
              verbose=0)

    predictions = model.predict(X_test, verbose=0)
    Y_pred_classification = np.argmax(predictions[0], axis=1)
    Y_pred_index = np.argmax(predictions[1], axis=1)

    accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
    precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    accuracy_index = accuracy_score(Y_test_index, Y_pred_index)

    print(f"Fold {fold} - Acc: {accuracy_classification:.4f}, Prec: {precision_classification:.4f}, Rec: {recall_classification:.4f}, F1: {f1_classification:.4f}, Index Acc: {accuracy_index:.4f}")

    accs.append(accuracy_classification)
    precs.append(precision_classification)
    recs.append(recall_classification)
    f1s.append(f1_classification)
    idx_accs.append(accuracy_index)
    fold += 1

print("\nCross Validation Results:")
print(f"Classification Accuracy: Mean={np.mean(accs):.4f}, Std={np.std(accs):.4f}")
print(f"Precision: Mean={np.mean(precs):.4f}, Std={np.std(precs):.4f}")
print(f"Recall: Mean={np.mean(recs):.4f}, Std={np.std(recs):.4f}")
print(f"F1-score: Mean={np.mean(f1s):.4f}, Std={np.std(f1s):.4f}")
print(f"Misbehavior Index Accuracy: Mean={np.mean(idx_accs):.4f}, Std={np.std(idx_accs):.4f}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Fold 1 - Acc: 0.5072, Prec: 0.2095, Rec: 0.2157, F1: 0.2045, Index Acc: 0.9626
Fold 2 - Acc: 0.4936, Prec: 0.2109, Rec: 0.1924, F1: 0.1904, Index Acc: 0.9611
Fold 3 - Acc: 0.5247, Prec: 0.2396, Rec: 0.2432, F1: 0.2358, Index Acc: 0.9624
Fold 4 - Acc: 0.5119, Prec: 0.2210, Rec: 0.2251, F1: 0.2183, Index Acc: 0.9606
Fold 5 - Acc: 0.5131, Prec: 0.2146, Rec: 0.2183, F1: 0.2063, Index Acc: 0.9626

Cross Validation Results:
Classification Accuracy: Mean=0.5101, Std=0.0101
Precision: Mean=0.2191, Std=0.0110
Recall: Mean=0.2189, Std=0.0164
F1-score: Mean=0.2111, Std=0.0152
Misbehavior Index Accuracy: Mean=0.9619, Std=0.0009

Training time: 329.979070 seconds


In [4]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_2x2_ea'
FAC_LABELS_CSV = 'multiple_class_2x2_ea.csv'
FAC_IMAGE_SIZE = (2, 2, 12)

def preprocess_image(file_path):
    """
    Load and preprocess an image for prediction.
    Expects .npy files containing image arrays.
    """
    image_array = np.load(file_path) / 255.0
    return np.expand_dims(image_array, axis=0)

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

random_index = np.random.choice(valid_labels.index)
selected_label = valid_labels.loc[random_index]
selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

start_time = time.time()
image_array = preprocess_image(selected_image_path)

predictions = model.predict(image_array)
predicted_classification = np.argmax(predictions[0])
predicted_index = np.argmax(predictions[1])
end_time = time.time()

print(f"Selected Image Path: {selected_image_path}")
print(f"True Classification: {selected_label['classification']}, True Misbehavior Index: {selected_label['misbehavior_index']}")
print(f"Predicted Classification: {predicted_classification}, Predicted Misbehavior Index: {predicted_index}")
print(f"\nElapsed time: {end_time - start_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Selected Image Path: veremi_multilevel_images_2x2_ea\image_4603.npy
True Classification: 0, True Misbehavior Index: 0
Predicted Classification: 0, Predicted Misbehavior Index: 0

Elapsed time: 0.069380 seconds


In [5]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_2x2_ea'
FAC_LABELS_CSV = 'multiple_class_2x2_ea.csv'
FAC_IMAGE_SIZE = (2, 2)
NUM_SAMPLES = 500
TIME_THRESHOLD = 0.1

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

def preprocess_image(file_path, target_size=(2, 2)):
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

total_classification_correct = 0
total_index_correct = 0
total_prediction_time = 0.0
predictions_above_threshold = 0
total_samples_processed = 0

true_classifications = []
predicted_classifications = []
true_indices = []
predicted_indices = []

for _ in range(NUM_SAMPLES):
    random_index = np.random.choice(valid_labels.index)
    selected_label = valid_labels.loc[random_index]
    selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

    if not os.path.exists(selected_image_path):
        continue

    try:
        start_time = time.time()
        image_array = preprocess_image(selected_image_path)
        image_array = np.expand_dims(image_array, axis=0)

        predictions = model.predict(image_array)
        predicted_classification = np.argmax(predictions[0])
        predicted_index = np.argmax(predictions[1])

        true_classifications.append(int(selected_label['classification']))
        predicted_classifications.append(predicted_classification)
        true_indices.append(int(selected_label['misbehavior_index']))
        predicted_indices.append(predicted_index)

        if predicted_classification == int(selected_label['classification']):
            total_classification_correct += 1
        if predicted_index == int(selected_label['misbehavior_index']):
            total_index_correct += 1

        prediction_time = time.time() - start_time
        total_prediction_time += prediction_time

        if prediction_time >= TIME_THRESHOLD:
            predictions_above_threshold += 1

        total_samples_processed += 1

    except Exception as e:
        print(f"Error during prediction: {e}")

classification_cm = confusion_matrix(true_classifications, predicted_classifications)
index_cm = confusion_matrix(true_indices, predicted_indices)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    total = np.sum(cm)

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    accuracy = np.sum(tp) / total if total > 0 else 0.0

    return accuracy, precision, recall, f1

classification_accuracy, classification_precision, classification_recall, classification_f1 = calculate_metrics(classification_cm)
index_accuracy, index_precision, index_recall, index_f1 = calculate_metrics(index_cm)

print("\nClassification Metrics:")
print(f"Accuracy: {classification_accuracy:.2f}")
print(f"Average Precision: {np.mean(classification_precision):.2f}")
print(f"Average Recall: {np.mean(classification_recall):.2f}")
print(f"Average F1-Score: {np.mean(classification_f1):.2f}")

print("\nIndex Prediction Metrics:")
print(f"Accuracy: {index_accuracy:.2f}")
print(f"Average Precision: {np.mean(index_precision):.2f}")
print(f"Average Recall: {np.mean(index_recall):.2f}")
print(f"Average F1-Score: {np.mean(index_f1):.2f}")

print(f"\nAverage Prediction Time: {total_prediction_time / total_samples_processed:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━

In [23]:
import numpy as np
import pandas as pd
import os

# Constants
FAC_IMAGE_WIDTH = 3
FAC_IMAGE_HEIGHT = 3
FAC_NUM_CHANNELS = 12
FAC_SPECIAL_VALUE = 255
NUM_CLASSES = 20

def normalize_and_scale(values, min_values, max_values):
    range_values = max_values - min_values
    range_values[range_values == 0] = 1
    scaled = 255 * (values - min_values) / range_values
    scaled[np.isnan(values) | ~np.isfinite(values)] = FAC_SPECIAL_VALUE
    scaled[values == max_values] = 0
    return np.clip(scaled, 0, 255).astype(np.uint8)

def generate_image(group, misbehavior_index=None):
    min_values = np.array([group[col].min() for col in final_numeric_columns])
    max_values = np.array([group[col].max() for col in final_numeric_columns])

    data = group[final_numeric_columns].to_numpy()
    normalized_data = normalize_and_scale(data, min_values, max_values)

    reshaped_data = normalized_data.reshape(FAC_IMAGE_HEIGHT, FAC_IMAGE_WIDTH, FAC_NUM_CHANNELS)

    if misbehavior_index:
        row = (misbehavior_index - 1) // FAC_IMAGE_WIDTH
        col = (misbehavior_index - 1) % FAC_IMAGE_WIDTH
        reshaped_data[row, col, :3] = [255, 0, 0]
        
    return reshaped_data

final_numeric_columns = [
    'sendTime', 'senderPseudo', 'posx', 'posy', 'posx_n', 'posy_n',
    'spdx_n', 'spdy_n', 'aclx_n', 'acly_n', 'hedx_n', 'hedy_n'
]

output_dir = 'veremi_multilevel_images_3x3_ea'
os.makedirs(output_dir, exist_ok=True)

normal_messages = final_df[final_df['class'] == 0]

classification_results = []
image_counter = 1

for _ in range(8000):
    sampled_group = normal_messages.sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
    image_data = generate_image(sampled_group)
    npy_name = f'{output_dir}/image_{image_counter}.npy'
    np.save(npy_name, image_data)
    classification_results.append({'file_name': npy_name, 'classification': 0, 'misbehavior_index': 0})
    image_counter += 1

for misbehavior_class in range(1, 20):
    for _ in range(600):
        attack_group = final_df[final_df['class'] == misbehavior_class].sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
        misbehavior_index = np.random.randint(1, FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT + 1)
        image_data = generate_image(attack_group, misbehavior_index)
        npy_name = f'{output_dir}/image_{image_counter}.npy'
        np.save(npy_name, image_data)
        classification_results.append({'file_name': npy_name, 'classification': misbehavior_class, 'misbehavior_index': misbehavior_index})
        image_counter += 1

classification_df = pd.DataFrame(classification_results)
classification_df.to_csv('multiple_class_3x3_ea.csv', index=False)

print("Images generated and saved to 'multiple_class_3x3_ea.csv'.")

Images generated and saved to 'multiple_class_3x3_ea.csv'.


In [9]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (3, 3, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_3x3_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_3x3_ea.csv')

fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X = []
    Y_classification = []
    Y_index = []
    
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path)
            image_array = image_array / 255.0

            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)

    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

X_train, X_test, Y_train_classification, Y_test_classification, Y_train_index, Y_test_index = train_test_split(
    X, Y_classification, Y_index, test_size=0.2, random_state=42)
input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))

x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

classification_output = Dense(20, activation='softmax', name='classification_output')(x)

index_output = Dense(26, activation='softmax', name='index_output')(x)

model = Model(inputs=input_layer, outputs=[classification_output, index_output])

model.compile(optimizer='adam', 
              loss={'classification_output': 'sparse_categorical_crossentropy', 
                    'index_output': 'sparse_categorical_crossentropy'},
              metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, 
          {'classification_output': Y_train_classification, 'index_output': Y_train_index}, 
          epochs=FAC_EPOCHS, 
          batch_size=FAC_BATCH_SIZE, 
          validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
          callbacks=[early_stopping])

predictions = model.predict(X_test)
Y_pred_classification = np.argmax(predictions[0], axis=1)
Y_pred_index = np.argmax(predictions[1], axis=1)

accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)

accuracy_index = accuracy_score(Y_test_index, Y_pred_index)
f1_classification_manual = 2 * (precision_classification * recall_classification) / (precision_classification + recall_classification)

print("Classification Accuracy:", accuracy_classification)
print("Classification Precision:", precision_classification)
print("Classification Recall:", recall_classification)
print("Classification F1-score (sklearn):", f1_classification)
print("Classification F1-score (manual calculation):", f1_classification_manual)

print("Misbehavior Index Accuracy:", accuracy_index)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

save_dir = "models"
os.makedirs(save_dir, exist_ok=True)

model_save_path = os.path.join(save_dir, "veremi_images_m_3x3_ea_i.keras")
model.save(model_save_path)

print(f"\nModel saved at: {model_save_path}")

Epoch 1/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - classification_output_accuracy: 0.3793 - classification_output_loss: 2.3114 - index_output_accuracy: 0.6300 - index_output_loss: 1.4130 - loss: 3.7243 - val_classification_output_accuracy: 0.4938 - val_classification_output_loss: 1.8484 - val_index_output_accuracy: 0.6554 - val_index_output_loss: 0.9014 - val_loss: 2.7518
Epoch 2/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - classification_output_accuracy: 0.6217 - classification_output_loss: 1.2204 - index_output_accuracy: 0.9216 - index_output_loss: 0.2199 - loss: 1.4403 - val_classification_output_accuracy: 0.6317 - val_classification_output_loss: 1.2166 - val_index_output_accuracy: 0.9186 - val_index_output_loss: 0.2410 - val_loss: 1.4561
Epoch 3/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - classification_output_accuracy: 0.6866 - classification_output_loss: 0.9879 - index_output_accuracy: 0.9433 - index_output_loss: 0.1553 - loss: 1.1432 - val_classification_output_accuracy

In [6]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (3, 3, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_3x3_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_3x3_ea.csv')
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X, Y_classification, Y_index = [], [], []
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path) / 255.0
            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)
    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accs, precs, recs, f1s, idx_accs = [], [], [], [], []
start_time = time.time()
fold = 1

for train_index, test_index in skf.split(X, Y_classification):
    X_train, X_test = X[train_index], X[test_index]
    Y_train_classification, Y_test_classification = Y_classification[train_index], Y_classification[test_index]
    Y_train_index, Y_test_index = Y_index[train_index], Y_index[test_index]

    input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))
    x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)

    classification_output = Dense(20, activation='softmax', name='classification_output')(x)
    index_output = Dense(26, activation='softmax', name='index_output')(x)

    model = Model(inputs=input_layer, outputs=[classification_output, index_output])
    model.compile(optimizer='adam',
                  loss={'classification_output': 'sparse_categorical_crossentropy',
                        'index_output': 'sparse_categorical_crossentropy'},
                  metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(X_train,
              {'classification_output': Y_train_classification, 'index_output': Y_train_index},
              epochs=FAC_EPOCHS,
              batch_size=FAC_BATCH_SIZE,
              validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
              callbacks=[early_stopping],
              verbose=0)

    predictions = model.predict(X_test, verbose=0)
    Y_pred_classification = np.argmax(predictions[0], axis=1)
    Y_pred_index = np.argmax(predictions[1], axis=1)

    accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
    precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    accuracy_index = accuracy_score(Y_test_index, Y_pred_index)

    print(f"Fold {fold} - Acc: {accuracy_classification:.4f}, Prec: {precision_classification:.4f}, Rec: {recall_classification:.4f}, F1: {f1_classification:.4f}, Index Acc: {accuracy_index:.4f}")

    accs.append(accuracy_classification)
    precs.append(precision_classification)
    recs.append(recall_classification)
    f1s.append(f1_classification)
    idx_accs.append(accuracy_index)
    fold += 1

print("\nCross Validation Results:")
print(f"Classification Accuracy: Mean={np.mean(accs):.4f}, Std={np.std(accs):.4f}")
print(f"Precision: Mean={np.mean(precs):.4f}, Std={np.std(precs):.4f}")
print(f"Recall: Mean={np.mean(recs):.4f}, Std={np.std(recs):.4f}")
print(f"F1-score: Mean={np.mean(f1s):.4f}, Std={np.std(f1s):.4f}")
print(f"Misbehavior Index Accuracy: Mean={np.mean(idx_accs):.4f}, Std={np.std(idx_accs):.4f}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Fold 1 - Acc: 0.7260, Prec: 0.5753, Rec: 0.5648, F1: 0.5583, Index Acc: 0.9518
Fold 2 - Acc: 0.6964, Prec: 0.5402, Rec: 0.5450, F1: 0.5316, Index Acc: 0.9258
Fold 3 - Acc: 0.7098, Prec: 0.5608, Rec: 0.5486, F1: 0.5345, Index Acc: 0.9446
Fold 4 - Acc: 0.7090, Prec: 0.5580, Rec: 0.5365, F1: 0.5296, Index Acc: 0.9394
Fold 5 - Acc: 0.7278, Prec: 0.5854, Rec: 0.5731, F1: 0.5615, Index Acc: 0.9402

Cross Validation Results:
Classification Accuracy: Mean=0.7138, Std=0.0117
Precision: Mean=0.5639, Std=0.0155
Recall: Mean=0.5536, Std=0.0134
F1-score: Mean=0.5431, Std=0.0138
Misbehavior Index Accuracy: Mean=0.9404, Std=0.0085

Training time: 297.771817 seconds


In [7]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_3x3_ea'
FAC_LABELS_CSV = 'multiple_class_3x3_ea.csv'
FAC_IMAGE_SIZE = (3, 3, 12)

def preprocess_image(file_path):
    """
    Load and preprocess an image for prediction.
    Expects .npy files containing image arrays.
    """
    image_array = np.load(file_path) / 255.0
    return np.expand_dims(image_array, axis=0)

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

random_index = np.random.choice(valid_labels.index)
selected_label = valid_labels.loc[random_index]
selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

start_time = time.time()
image_array = preprocess_image(selected_image_path)

predictions = model.predict(image_array)
predicted_classification = np.argmax(predictions[0])
predicted_index = np.argmax(predictions[1])
end_time = time.time()

print(f"Selected Image Path: {selected_image_path}")
print(f"True Classification: {selected_label['classification']}, True Misbehavior Index: {selected_label['misbehavior_index']}")
print(f"Predicted Classification: {predicted_classification}, Predicted Misbehavior Index: {predicted_index}")
print(f"\nElapsed time: {end_time - start_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Selected Image Path: veremi_multilevel_images_3x3_ea\image_17728.npy
True Classification: 17, True Misbehavior Index: 4
Predicted Classification: 17, Predicted Misbehavior Index: 4

Elapsed time: 0.078577 seconds


In [9]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_3x3_ea'
FAC_LABELS_CSV = 'multiple_class_3x3_ea.csv'
FAC_IMAGE_SIZE = (3, 3)
NUM_SAMPLES = 500
TIME_THRESHOLD = 0.1

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

def preprocess_image(file_path, target_size=(3, 3)):
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

total_classification_correct = 0
total_index_correct = 0
total_prediction_time = 0.0
predictions_above_threshold = 0
total_samples_processed = 0

true_classifications = []
predicted_classifications = []
true_indices = []
predicted_indices = []

for _ in range(NUM_SAMPLES):
    random_index = np.random.choice(valid_labels.index)
    selected_label = valid_labels.loc[random_index]
    selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

    if not os.path.exists(selected_image_path):
        continue

    try:
        start_time = time.time()
        image_array = preprocess_image(selected_image_path)
        image_array = np.expand_dims(image_array, axis=0)

        predictions = model.predict(image_array)
        predicted_classification = np.argmax(predictions[0])
        predicted_index = np.argmax(predictions[1])

        true_classifications.append(int(selected_label['classification']))
        predicted_classifications.append(predicted_classification)
        true_indices.append(int(selected_label['misbehavior_index']))
        predicted_indices.append(predicted_index)

        if predicted_classification == int(selected_label['classification']):
            total_classification_correct += 1
        if predicted_index == int(selected_label['misbehavior_index']):
            total_index_correct += 1

        prediction_time = time.time() - start_time
        total_prediction_time += prediction_time

        if prediction_time >= TIME_THRESHOLD:
            predictions_above_threshold += 1

        total_samples_processed += 1

    except Exception as e:
        print(f"Error during prediction: {e}")

classification_cm = confusion_matrix(true_classifications, predicted_classifications)
index_cm = confusion_matrix(true_indices, predicted_indices)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    total = np.sum(cm)

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    accuracy = np.sum(tp) / total if total > 0 else 0.0

    return accuracy, precision, recall, f1

classification_accuracy, classification_precision, classification_recall, classification_f1 = calculate_metrics(classification_cm)
index_accuracy, index_precision, index_recall, index_f1 = calculate_metrics(index_cm)

print("\nClassification Metrics:")
print(f"Accuracy: {classification_accuracy:.2f}")
print(f"Average Precision: {np.mean(classification_precision):.2f}")
print(f"Average Recall: {np.mean(classification_recall):.2f}")
print(f"Average F1-Score: {np.mean(classification_f1):.2f}")

print("\nIndex Prediction Metrics:")
print(f"Accuracy: {index_accuracy:.2f}")
print(f"Average Precision: {np.mean(index_precision):.2f}")
print(f"Average Recall: {np.mean(index_recall):.2f}")
print(f"Average F1-Score: {np.mean(index_f1):.2f}")

print(f"\nAverage Prediction Time: {total_prediction_time / total_samples_processed:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━

In [85]:
import numpy as np
import pandas as pd
import os

# Constants
FAC_IMAGE_WIDTH = 4
FAC_IMAGE_HEIGHT = 4
FAC_NUM_CHANNELS = 12
FAC_SPECIAL_VALUE = 255
NUM_CLASSES = 20

def normalize_and_scale(values, min_values, max_values):
    scaled = 255 * (values - min_values) / (max_values - min_values)
    scaled[np.isnan(values) | ~np.isfinite(values)] = FAC_SPECIAL_VALUE
    scaled[values == max_values] = 0
    return np.clip(scaled, 0, 255).astype(np.uint8)

def generate_image(group, misbehavior_index=None):
    min_values = np.array([group[col].min() for col in final_numeric_columns])
    max_values = np.array([group[col].max() for col in final_numeric_columns])

    data = group[final_numeric_columns].to_numpy()
    normalized_data = normalize_and_scale(data, min_values, max_values)

    reshaped_data = normalized_data.reshape(FAC_IMAGE_HEIGHT, FAC_IMAGE_WIDTH, FAC_NUM_CHANNELS)

    if misbehavior_index:
        row = (misbehavior_index - 1) // FAC_IMAGE_WIDTH
        col = (misbehavior_index - 1) % FAC_IMAGE_WIDTH
        reshaped_data[row, col, :3] = [255, 0, 0]
        
    return reshaped_data

final_numeric_columns = [
    'sendTime', 'senderPseudo', 'posx', 'posy', 'posx_n', 'posy_n',
    'spdx_n', 'spdy_n', 'aclx_n', 'acly_n', 'hedx_n', 'hedy_n'
]

output_dir = 'veremi_multilevel_images_4x4_ea'
os.makedirs(output_dir, exist_ok=True)

normal_messages = final_df[final_df['class'] == 0]

classification_results = []
image_counter = 1

for _ in range(2500):
    sampled_group = normal_messages.sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
    image_data = generate_image(sampled_group)
    npy_name = f'{output_dir}/image_{image_counter}.npy'
    np.save(npy_name, image_data)
    classification_results.append({'file_name': npy_name, 'classification': 0, 'misbehavior_index': 0})
    image_counter += 1

for misbehavior_class in range(1, 20):
    for _ in range(300):
        attack_group = final_df[final_df['class'] == misbehavior_class].sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
        misbehavior_index = np.random.randint(1, FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT + 1)
        image_data = generate_image(attack_group, misbehavior_index)
        npy_name = f'{output_dir}/image_{image_counter}.npy'
        np.save(npy_name, image_data)
        classification_results.append({'file_name': npy_name, 'classification': misbehavior_class, 'misbehavior_index': misbehavior_index})
        image_counter += 1

classification_df = pd.DataFrame(classification_results)
classification_df.to_csv('multiple_class_4x4_ea.csv', index=False)

print("Images generated and saved to 'multiple_class_4x4_ea.csv'.")

Images generated and saved to 'multiple_class_4x4_ea.csv'.


In [10]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (4, 4, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_4x4_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_4x4_ea.csv')

fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X = []
    Y_classification = []
    Y_index = []
    
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path)
            image_array = image_array / 255.0

            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)

    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

X_train, X_test, Y_train_classification, Y_test_classification, Y_train_index, Y_test_index = train_test_split(
    X, Y_classification, Y_index, test_size=0.2, random_state=42)
input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))

x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

classification_output = Dense(20, activation='softmax', name='classification_output')(x)

index_output = Dense(26, activation='softmax', name='index_output')(x)

model = Model(inputs=input_layer, outputs=[classification_output, index_output])

model.compile(optimizer='adam', 
              loss={'classification_output': 'sparse_categorical_crossentropy', 
                    'index_output': 'sparse_categorical_crossentropy'},
              metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, 
          {'classification_output': Y_train_classification, 'index_output': Y_train_index}, 
          epochs=FAC_EPOCHS, 
          batch_size=FAC_BATCH_SIZE, 
          validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
          callbacks=[early_stopping])

predictions = model.predict(X_test)
Y_pred_classification = np.argmax(predictions[0], axis=1)
Y_pred_index = np.argmax(predictions[1], axis=1)

accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)

accuracy_index = accuracy_score(Y_test_index, Y_pred_index)
f1_classification_manual = 2 * (precision_classification * recall_classification) / (precision_classification + recall_classification)

print("Classification Accuracy:", accuracy_classification)
print("Classification Precision:", precision_classification)
print("Classification Recall:", recall_classification)
print("Classification F1-score (sklearn):", f1_classification)
print("Classification F1-score (manual calculation):", f1_classification_manual)

print("Misbehavior Index Accuracy:", accuracy_index)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

save_dir = "models"
os.makedirs(save_dir, exist_ok=True)

model_save_path = os.path.join(save_dir, "veremi_images_m_4x4_ea_i.keras")
model.save(model_save_path)

print(f"\nModel saved at: {model_save_path}")

Epoch 1/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - classification_output_accuracy: 0.4093 - classification_output_loss: 2.0857 - index_output_accuracy: 0.4833 - index_output_loss: 1.9438 - loss: 4.0296 - val_classification_output_accuracy: 0.3128 - val_classification_output_loss: 3.6458 - val_index_output_accuracy: 0.3128 - val_index_output_loss: 4.0544 - val_loss: 7.7072
Epoch 2/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - classification_output_accuracy: 0.6452 - classification_output_loss: 1.1453 - index_output_accuracy: 0.8830 - index_output_loss: 0.3399 - loss: 1.4852 - val_classification_output_accuracy: 0.3341 - val_classification_output_loss: 3.0866 - val_index_output_accuracy: 0.3561 - val_index_output_loss: 2.5327 - val_loss: 5.6287
Epoch 3/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - classification_output_accuracy: 0.7213 - classification_output_loss: 0.8495 - index_output_accuracy: 0.9226 - index_output_loss: 0.2226 - loss: 1.0721 - val_classification_output_accuracy

In [10]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (4, 4, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_4x4_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_4x4_ea.csv')
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X, Y_classification, Y_index = [], [], []
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path) / 255.0
            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)
    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accs, precs, recs, f1s, idx_accs = [], [], [], [], []
start_time = time.time()
fold = 1

for train_index, test_index in skf.split(X, Y_classification):
    X_train, X_test = X[train_index], X[test_index]
    Y_train_classification, Y_test_classification = Y_classification[train_index], Y_classification[test_index]
    Y_train_index, Y_test_index = Y_index[train_index], Y_index[test_index]

    input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))
    x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)

    classification_output = Dense(20, activation='softmax', name='classification_output')(x)
    index_output = Dense(26, activation='softmax', name='index_output')(x)

    model = Model(inputs=input_layer, outputs=[classification_output, index_output])
    model.compile(optimizer='adam',
                  loss={'classification_output': 'sparse_categorical_crossentropy',
                        'index_output': 'sparse_categorical_crossentropy'},
                  metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(X_train,
              {'classification_output': Y_train_classification, 'index_output': Y_train_index},
              epochs=FAC_EPOCHS,
              batch_size=FAC_BATCH_SIZE,
              validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
              callbacks=[early_stopping],
              verbose=0)

    predictions = model.predict(X_test, verbose=0)
    Y_pred_classification = np.argmax(predictions[0], axis=1)
    Y_pred_index = np.argmax(predictions[1], axis=1)

    accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
    precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    accuracy_index = accuracy_score(Y_test_index, Y_pred_index)

    print(f"Fold {fold} - Acc: {accuracy_classification:.4f}, Prec: {precision_classification:.4f}, Rec: {recall_classification:.4f}, F1: {f1_classification:.4f}, Index Acc: {accuracy_index:.4f}")

    accs.append(accuracy_classification)
    precs.append(precision_classification)
    recs.append(recall_classification)
    f1s.append(f1_classification)
    idx_accs.append(accuracy_index)
    fold += 1

print("\nCross Validation Results:")
print(f"Classification Accuracy: Mean={np.mean(accs):.4f}, Std={np.std(accs):.4f}")
print(f"Precision: Mean={np.mean(precs):.4f}, Std={np.std(precs):.4f}")
print(f"Recall: Mean={np.mean(recs):.4f}, Std={np.std(recs):.4f}")
print(f"F1-score: Mean={np.mean(f1s):.4f}, Std={np.std(f1s):.4f}")
print(f"Misbehavior Index Accuracy: Mean={np.mean(idx_accs):.4f}, Std={np.std(idx_accs):.4f}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Fold 1 - Acc: 0.6329, Prec: 0.5471, Rec: 0.5042, F1: 0.4867, Index Acc: 0.8841
Fold 2 - Acc: 0.6390, Prec: 0.5554, Rec: 0.5089, F1: 0.4985, Index Acc: 0.8976
Fold 3 - Acc: 0.6055, Prec: 0.5330, Rec: 0.4748, F1: 0.4671, Index Acc: 0.8555
Fold 4 - Acc: 0.6360, Prec: 0.5618, Rec: 0.5091, F1: 0.5099, Index Acc: 0.8683
Fold 5 - Acc: 0.6732, Prec: 0.5752, Rec: 0.5665, F1: 0.5577, Index Acc: 0.9238

Cross Validation Results:
Classification Accuracy: Mean=0.6373, Std=0.0216
Precision: Mean=0.5545, Std=0.0141
Recall: Mean=0.5127, Std=0.0298
F1-score: Mean=0.5040, Std=0.0304
Misbehavior Index Accuracy: Mean=0.8859, Std=0.0237

Training time: 148.033273 seconds


In [11]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_4x4_ea'
FAC_LABELS_CSV = 'multiple_class_4x4_ea.csv'
FAC_IMAGE_SIZE = (4, 4, 12)

def preprocess_image(file_path):
    """
    Load and preprocess an image for prediction.
    Expects .npy files containing image arrays.
    """
    image_array = np.load(file_path) / 255.0
    return np.expand_dims(image_array, axis=0)

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

random_index = np.random.choice(valid_labels.index)
selected_label = valid_labels.loc[random_index]
selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

start_time = time.time()
image_array = preprocess_image(selected_image_path)

predictions = model.predict(image_array)
predicted_classification = np.argmax(predictions[0])
predicted_index = np.argmax(predictions[1])
end_time = time.time()

print(f"Selected Image Path: {selected_image_path}")
print(f"True Classification: {selected_label['classification']}, True Misbehavior Index: {selected_label['misbehavior_index']}")
print(f"Predicted Classification: {predicted_classification}, Predicted Misbehavior Index: {predicted_index}")
print(f"\nElapsed time: {end_time - start_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Selected Image Path: veremi_multilevel_images_4x4_ea\image_6473.npy
True Classification: 14, True Misbehavior Index: 11
Predicted Classification: 14, Predicted Misbehavior Index: 11

Elapsed time: 0.075318 seconds


In [12]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_4x4_ea'
FAC_LABELS_CSV = 'multiple_class_4x4_ea.csv'
FAC_IMAGE_SIZE = (4, 4)
NUM_SAMPLES = 500
TIME_THRESHOLD = 0.1

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

def preprocess_image(file_path, target_size=(4, 4)):
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

total_classification_correct = 0
total_index_correct = 0
total_prediction_time = 0.0
predictions_above_threshold = 0
total_samples_processed = 0

true_classifications = []
predicted_classifications = []
true_indices = []
predicted_indices = []

for _ in range(NUM_SAMPLES):
    random_index = np.random.choice(valid_labels.index)
    selected_label = valid_labels.loc[random_index]
    selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

    if not os.path.exists(selected_image_path):
        continue

    try:
        start_time = time.time()
        image_array = preprocess_image(selected_image_path)
        image_array = np.expand_dims(image_array, axis=0)

        predictions = model.predict(image_array)
        predicted_classification = np.argmax(predictions[0])
        predicted_index = np.argmax(predictions[1])

        true_classifications.append(int(selected_label['classification']))
        predicted_classifications.append(predicted_classification)
        true_indices.append(int(selected_label['misbehavior_index']))
        predicted_indices.append(predicted_index)

        if predicted_classification == int(selected_label['classification']):
            total_classification_correct += 1
        if predicted_index == int(selected_label['misbehavior_index']):
            total_index_correct += 1

        prediction_time = time.time() - start_time
        total_prediction_time += prediction_time

        if prediction_time >= TIME_THRESHOLD:
            predictions_above_threshold += 1

        total_samples_processed += 1

    except Exception as e:
        print(f"Error during prediction: {e}")

classification_cm = confusion_matrix(true_classifications, predicted_classifications)
index_cm = confusion_matrix(true_indices, predicted_indices)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    total = np.sum(cm)

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    accuracy = np.sum(tp) / total if total > 0 else 0.0

    return accuracy, precision, recall, f1

classification_accuracy, classification_precision, classification_recall, classification_f1 = calculate_metrics(classification_cm)
index_accuracy, index_precision, index_recall, index_f1 = calculate_metrics(index_cm)

print("\nClassification Metrics:")
print(f"Accuracy: {classification_accuracy:.2f}")
print(f"Average Precision: {np.mean(classification_precision):.2f}")
print(f"Average Recall: {np.mean(classification_recall):.2f}")
print(f"Average F1-Score: {np.mean(classification_f1):.2f}")

print("\nIndex Prediction Metrics:")
print(f"Accuracy: {index_accuracy:.2f}")
print(f"Average Precision: {np.mean(index_precision):.2f}")
print(f"Average Recall: {np.mean(index_recall):.2f}")
print(f"Average F1-Score: {np.mean(index_f1):.2f}")

print(f"\nAverage Prediction Time: {total_prediction_time / total_samples_processed:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━

In [28]:
import numpy as np
import pandas as pd
import os

# Constants
FAC_IMAGE_WIDTH = 5
FAC_IMAGE_HEIGHT = 5
FAC_NUM_CHANNELS = 12
FAC_SPECIAL_VALUE = 255
NUM_CLASSES = 20

def normalize_and_scale(values, min_values, max_values):
    scaled = 255 * (values - min_values) / (max_values - min_values)
    scaled[np.isnan(values) | ~np.isfinite(values)] = FAC_SPECIAL_VALUE
    scaled[values == max_values] = 0
    return np.clip(scaled, 0, 255).astype(np.uint8)

def generate_image(group, misbehavior_index=None):
    min_values = np.array([group[col].min() for col in final_numeric_columns])
    max_values = np.array([group[col].max() for col in final_numeric_columns])

    data = group[final_numeric_columns].to_numpy()
    normalized_data = normalize_and_scale(data, min_values, max_values)

    reshaped_data = normalized_data.reshape(FAC_IMAGE_HEIGHT, FAC_IMAGE_WIDTH, FAC_NUM_CHANNELS)

    if misbehavior_index:
        row = (misbehavior_index - 1) // FAC_IMAGE_WIDTH
        col = (misbehavior_index - 1) % FAC_IMAGE_WIDTH
        reshaped_data[row, col, :3] = [255, 0, 0]
        
    return reshaped_data

final_numeric_columns = [
    'sendTime', 'senderPseudo', 'posx', 'posy', 'posx_n', 'posy_n',
    'spdx_n', 'spdy_n', 'aclx_n', 'acly_n', 'hedx_n', 'hedy_n'
]

output_dir = 'veremi_multilevel_images_5x5_ea'
os.makedirs(output_dir, exist_ok=True)

normal_messages = final_df[final_df['class'] == 0]

classification_results = []
image_counter = 1

for _ in range(2500):
    sampled_group = normal_messages.sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
    image_data = generate_image(sampled_group)
    npy_name = f'{output_dir}/image_{image_counter}.npy'
    np.save(npy_name, image_data)
    classification_results.append({'file_name': npy_name, 'classification': 0, 'misbehavior_index': 0})
    image_counter += 1

for misbehavior_class in range(1, 20):
    for _ in range(300):
        attack_group = final_df[final_df['class'] == misbehavior_class].sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
        misbehavior_index = np.random.randint(1, FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT + 1)
        image_data = generate_image(attack_group, misbehavior_index)
        npy_name = f'{output_dir}/image_{image_counter}.npy'
        np.save(npy_name, image_data)
        classification_results.append({'file_name': npy_name, 'classification': misbehavior_class, 'misbehavior_index': misbehavior_index})
        image_counter += 1

classification_df = pd.DataFrame(classification_results)
classification_df.to_csv('multiple_class_5x5_ea.csv', index=False)

print("Images generated and saved to 'multiple_class_5x5_ea.csv'.")

Images generated and saved to 'multiple_class_5x5_ea.csv'.


In [11]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (5, 5, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_5x5_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_5x5_ea.csv')

fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X = []
    Y_classification = []
    Y_index = []
    
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path)
            image_array = image_array / 255.0

            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)

    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

X_train, X_test, Y_train_classification, Y_test_classification, Y_train_index, Y_test_index = train_test_split(
    X, Y_classification, Y_index, test_size=0.2, random_state=42)
input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))

x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

classification_output = Dense(20, activation='softmax', name='classification_output')(x)

index_output = Dense(26, activation='softmax', name='index_output')(x)

model = Model(inputs=input_layer, outputs=[classification_output, index_output])

model.compile(optimizer='adam', 
              loss={'classification_output': 'sparse_categorical_crossentropy', 
                    'index_output': 'sparse_categorical_crossentropy'},
              metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, 
          {'classification_output': Y_train_classification, 'index_output': Y_train_index}, 
          epochs=FAC_EPOCHS, 
          batch_size=FAC_BATCH_SIZE, 
          validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
          callbacks=[early_stopping])

predictions = model.predict(X_test)
Y_pred_classification = np.argmax(predictions[0], axis=1)
Y_pred_index = np.argmax(predictions[1], axis=1)

accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)

accuracy_index = accuracy_score(Y_test_index, Y_pred_index)
f1_classification_manual = 2 * (precision_classification * recall_classification) / (precision_classification + recall_classification)

print("Classification Accuracy:", accuracy_classification)
print("Classification Precision:", precision_classification)
print("Classification Recall:", recall_classification)
print("Classification F1-score (sklearn):", f1_classification)
print("Classification F1-score (manual calculation):", f1_classification_manual)

print("Misbehavior Index Accuracy:", accuracy_index)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

save_dir = "models"
os.makedirs(save_dir, exist_ok=True)

model_save_path = os.path.join(save_dir, "veremi_images_m_5x5_ea_i.keras")
model.save(model_save_path)

print(f"\nModel saved at: {model_save_path}")

Epoch 1/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - classification_output_accuracy: 0.4922 - classification_output_loss: 1.7132 - index_output_accuracy: 0.4071 - index_output_loss: 2.2761 - loss: 3.9893 - val_classification_output_accuracy: 0.3128 - val_classification_output_loss: 3.8189 - val_index_output_accuracy: 0.3128 - val_index_output_loss: 4.1872 - val_loss: 8.0107
Epoch 2/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - classification_output_accuracy: 0.7422 - classification_output_loss: 0.8149 - index_output_accuracy: 0.8633 - index_output_loss: 0.4313 - loss: 1.2462 - val_classification_output_accuracy: 0.4104 - val_classification_output_loss: 2.4414 - val_index_output_accuracy: 0.4146 - val_index_output_loss: 2.0448 - val_loss: 4.4981
Epoch 3/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - classification_output_accuracy: 0.8341 - classification_output_loss: 0.5439 - index_output_accuracy: 0.9279 - index_output_loss: 0.2192 - loss: 0.7631 - val_classification_output_accuracy

In [13]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (5, 5, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_5x5_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_5x5_ea.csv')
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X, Y_classification, Y_index = [], [], []
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path) / 255.0
            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)
    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accs, precs, recs, f1s, idx_accs = [], [], [], [], []
start_time = time.time()
fold = 1

for train_index, test_index in skf.split(X, Y_classification):
    X_train, X_test = X[train_index], X[test_index]
    Y_train_classification, Y_test_classification = Y_classification[train_index], Y_classification[test_index]
    Y_train_index, Y_test_index = Y_index[train_index], Y_index[test_index]

    input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))
    x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)

    classification_output = Dense(20, activation='softmax', name='classification_output')(x)
    index_output = Dense(26, activation='softmax', name='index_output')(x)

    model = Model(inputs=input_layer, outputs=[classification_output, index_output])
    model.compile(optimizer='adam',
                  loss={'classification_output': 'sparse_categorical_crossentropy',
                        'index_output': 'sparse_categorical_crossentropy'},
                  metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(X_train,
              {'classification_output': Y_train_classification, 'index_output': Y_train_index},
              epochs=FAC_EPOCHS,
              batch_size=FAC_BATCH_SIZE,
              validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
              callbacks=[early_stopping],
              verbose=0)

    predictions = model.predict(X_test, verbose=0)
    Y_pred_classification = np.argmax(predictions[0], axis=1)
    Y_pred_index = np.argmax(predictions[1], axis=1)

    accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
    precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    accuracy_index = accuracy_score(Y_test_index, Y_pred_index)

    print(f"Fold {fold} - Acc: {accuracy_classification:.4f}, Prec: {precision_classification:.4f}, Rec: {recall_classification:.4f}, F1: {f1_classification:.4f}, Index Acc: {accuracy_index:.4f}")

    accs.append(accuracy_classification)
    precs.append(precision_classification)
    recs.append(recall_classification)
    f1s.append(f1_classification)
    idx_accs.append(accuracy_index)
    fold += 1

print("\nCross Validation Results:")
print(f"Classification Accuracy: Mean={np.mean(accs):.4f}, Std={np.std(accs):.4f}")
print(f"Precision: Mean={np.mean(precs):.4f}, Std={np.std(precs):.4f}")
print(f"Recall: Mean={np.mean(recs):.4f}, Std={np.std(recs):.4f}")
print(f"F1-score: Mean={np.mean(f1s):.4f}, Std={np.std(f1s):.4f}")
print(f"Misbehavior Index Accuracy: Mean={np.mean(idx_accs):.4f}, Std={np.std(idx_accs):.4f}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Fold 1 - Acc: 0.7409, Prec: 0.6998, Rec: 0.6473, F1: 0.6468, Index Acc: 0.8713
Fold 2 - Acc: 0.7402, Prec: 0.7024, Rec: 0.6567, F1: 0.6409, Index Acc: 0.8616
Fold 3 - Acc: 0.7171, Prec: 0.6678, Rec: 0.6141, F1: 0.6139, Index Acc: 0.8122
Fold 4 - Acc: 0.7287, Prec: 0.6762, Rec: 0.6299, F1: 0.6259, Index Acc: 0.8537
Fold 5 - Acc: 0.7793, Prec: 0.7367, Rec: 0.7064, F1: 0.7021, Index Acc: 0.9146

Cross Validation Results:
Classification Accuracy: Mean=0.7412, Std=0.0209
Precision: Mean=0.6966, Std=0.0241
Recall: Mean=0.6509, Std=0.0314
F1-score: Mean=0.6459, Std=0.0304
Misbehavior Index Accuracy: Mean=0.8627, Std=0.0329

Training time: 169.910955 seconds


In [14]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_5x5_ea'
FAC_LABELS_CSV = 'multiple_class_5x5_ea.csv'
FAC_IMAGE_SIZE = (5, 5, 12)

def preprocess_image(file_path):
    """
    Load and preprocess an image for prediction.
    Expects .npy files containing image arrays.
    """
    image_array = np.load(file_path) / 255.0
    return np.expand_dims(image_array, axis=0)

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

random_index = np.random.choice(valid_labels.index)
selected_label = valid_labels.loc[random_index]
selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

start_time = time.time()
image_array = preprocess_image(selected_image_path)

predictions = model.predict(image_array)
predicted_classification = np.argmax(predictions[0])
predicted_index = np.argmax(predictions[1])
end_time = time.time()

print(f"Selected Image Path: {selected_image_path}")
print(f"True Classification: {selected_label['classification']}, True Misbehavior Index: {selected_label['misbehavior_index']}")
print(f"Predicted Classification: {predicted_classification}, Predicted Misbehavior Index: {predicted_index}")
print(f"\nElapsed time: {end_time - start_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Selected Image Path: veremi_multilevel_images_5x5_ea\image_4835.npy
True Classification: 8, True Misbehavior Index: 10
Predicted Classification: 8, Predicted Misbehavior Index: 10

Elapsed time: 0.076264 seconds


In [15]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_5x5_ea'
FAC_LABELS_CSV = 'multiple_class_5x5_ea.csv'
FAC_IMAGE_SIZE = (5, 5)
NUM_SAMPLES = 500
TIME_THRESHOLD = 0.1

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

def preprocess_image(file_path, target_size=(5, 5)):
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

total_classification_correct = 0
total_index_correct = 0
total_prediction_time = 0.0
predictions_above_threshold = 0
total_samples_processed = 0

true_classifications = []
predicted_classifications = []
true_indices = []
predicted_indices = []

for _ in range(NUM_SAMPLES):
    random_index = np.random.choice(valid_labels.index)
    selected_label = valid_labels.loc[random_index]
    selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

    if not os.path.exists(selected_image_path):
        continue

    try:
        start_time = time.time()
        image_array = preprocess_image(selected_image_path)
        image_array = np.expand_dims(image_array, axis=0)

        predictions = model.predict(image_array)
        predicted_classification = np.argmax(predictions[0])
        predicted_index = np.argmax(predictions[1])

        true_classifications.append(int(selected_label['classification']))
        predicted_classifications.append(predicted_classification)
        true_indices.append(int(selected_label['misbehavior_index']))
        predicted_indices.append(predicted_index)

        if predicted_classification == int(selected_label['classification']):
            total_classification_correct += 1
        if predicted_index == int(selected_label['misbehavior_index']):
            total_index_correct += 1

        prediction_time = time.time() - start_time
        total_prediction_time += prediction_time

        if prediction_time >= TIME_THRESHOLD:
            predictions_above_threshold += 1

        total_samples_processed += 1

    except Exception as e:
        print(f"Error during prediction: {e}")

classification_cm = confusion_matrix(true_classifications, predicted_classifications)
index_cm = confusion_matrix(true_indices, predicted_indices)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    total = np.sum(cm)

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    accuracy = np.sum(tp) / total if total > 0 else 0.0

    return accuracy, precision, recall, f1

classification_accuracy, classification_precision, classification_recall, classification_f1 = calculate_metrics(classification_cm)
index_accuracy, index_precision, index_recall, index_f1 = calculate_metrics(index_cm)

print("\nClassification Metrics:")
print(f"Accuracy: {classification_accuracy:.2f}")
print(f"Average Precision: {np.mean(classification_precision):.2f}")
print(f"Average Recall: {np.mean(classification_recall):.2f}")
print(f"Average F1-Score: {np.mean(classification_f1):.2f}")

print("\nIndex Prediction Metrics:")
print(f"Accuracy: {index_accuracy:.2f}")
print(f"Average Precision: {np.mean(index_precision):.2f}")
print(f"Average Recall: {np.mean(index_recall):.2f}")
print(f"Average F1-Score: {np.mean(index_f1):.2f}")

print(f"\nAverage Prediction Time: {total_prediction_time / total_samples_processed:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━

In [13]:
import numpy as np
import pandas as pd
import os

FAC_IMAGE_WIDTH = 2
FAC_IMAGE_HEIGHT = 1
FAC_NUM_CHANNELS = 12
FAC_SPECIAL_VALUE = 255

final_numeric_columns = [
    'sendTime', 'senderPseudo', 'posx', 'posy', 'posx_n', 'posy_n',
    'spdx_n', 'spdy_n', 'aclx_n', 'acly_n', 'hedx_n', 'hedy_n'
]

min_max_values = {
    col: (final_df[col].min(), final_df[col].max()) for col in final_numeric_columns if col != 'class'
}

def normalize_and_scale(value, min_value, max_value):
    if pd.isna(value) or not np.isfinite(value):
        return FAC_SPECIAL_VALUE
    if max_value == min_value:
        return 0 if value == min_value else FAC_SPECIAL_VALUE
    scaled_value = 255 * (value - min_value) / (max_value - min_value)
    return int(np.clip(scaled_value, 0, 255))

def records_to_image(group, misbehavior_index=None):
    image_data = np.zeros((FAC_IMAGE_HEIGHT, FAC_IMAGE_WIDTH, FAC_NUM_CHANNELS), dtype=np.uint8)

    group = group.sort_values('sendTime')
    
    for i, (_, row) in enumerate(group.iterrows()):
        row_pos = 0
        col_pos = i % FAC_IMAGE_WIDTH
        
        for j, col in enumerate(final_numeric_columns[:-1]):
            min_value, max_value = min_max_values[col]
            value = normalize_and_scale(row[col], min_value, max_value)
            image_data[row_pos, col_pos, j] = value

    if misbehavior_index is not None:
        misbehavior_row = 0
        misbehavior_col = (misbehavior_index - 1) % FAC_IMAGE_WIDTH
        image_data[misbehavior_row, misbehavior_col, 0] = 255
        image_data[misbehavior_row, misbehavior_col, 1] = 0

    return image_data

output_dir = 'veremi_multilevel_images_1x2_ea'
os.makedirs(output_dir, exist_ok=True)

classification_results = []
image_counter = 1

normal_messages = final_df[final_df['class'] == 0]
attack_messages = final_df[final_df['class'] != 0]

for _ in range(8000):
    for obu_id, group in normal_messages.groupby('senderPseudo'):
        chunk = group.sample(n=2, random_state=image_counter)

        image_data = records_to_image(chunk)
        misbehavior_index = 0

        npy_file_name = f'{output_dir}/image_{image_counter}.npy'
        np.save(npy_file_name, image_data)

        classification_results.append({'npy_file_name': npy_file_name, 'classification': 0, 'misbehavior_index': misbehavior_index})

        image_counter += 1
        break

for misbehavior_class in range(1, 20):
    for _ in range(600):
        for obu_id, group in normal_messages.groupby('senderPseudo'):
            normal_chunk = group.sample(n=1, random_state=image_counter)
            attack_chunk = attack_messages[attack_messages['class'] == misbehavior_class].sample(n=1, random_state=image_counter)

            combined_chunk = pd.concat([normal_chunk, attack_chunk])

            misbehavior_index = np.random.randint(1, 3)
            image_data = records_to_image(combined_chunk, misbehavior_index)

            npy_file_name = f'{output_dir}/image_{image_counter}.npy'
            np.save(npy_file_name, image_data)

            classification_results.append({
                'npy_file_name': npy_file_name, 
                'classification': misbehavior_class, 
                'misbehavior_index': misbehavior_index
            })

            image_counter += 1
            break

classification_df = pd.DataFrame(classification_results)
classification_df.to_csv('multiple_class_1x2_ea.csv', index=False)

print("Classification results saved to 'multiple_class_1x2_ea.csv'.")

Classification results saved to 'multiple_class_1x2_ea.csv'.


In [12]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (1, 2, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_1x2_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_1x2_ea.csv')

fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X = []
    Y_classification = []
    Y_index = []
    
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path)
            image_array = image_array / 255.0

            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)

    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

X_train, X_test, Y_train_classification, Y_test_classification, Y_train_index, Y_test_index = train_test_split(
    X, Y_classification, Y_index, test_size=0.2, random_state=42)

input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))

x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

classification_output = Dense(20, activation='softmax', name='classification_output')(x)

index_output = Dense(3, activation='softmax', name='index_output')(x)

model = Model(inputs=input_layer, outputs=[classification_output, index_output])

model.compile(optimizer='adam', 
              loss={'classification_output': 'sparse_categorical_crossentropy', 
                    'index_output': 'sparse_categorical_crossentropy'},
              metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, 
          {'classification_output': Y_train_classification, 'index_output': Y_train_index}, 
          epochs=FAC_EPOCHS, 
          batch_size=FAC_BATCH_SIZE, 
          validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
          callbacks=[early_stopping])

predictions = model.predict(X_test)
Y_pred_classification = np.argmax(predictions[0], axis=1)
Y_pred_index = np.argmax(predictions[1], axis=1)

accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro')
recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro')
f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro')

accuracy_index = accuracy_score(Y_test_index, Y_pred_index)
f1_classification_manual = 2 * (precision_classification * recall_classification) / (precision_classification + recall_classification)

print("Classification Accuracy:", accuracy_classification)
print("Classification Precision:", precision_classification)
print("Classification Recall:", recall_classification)
print("Classification F1-score (sklearn):", f1_classification)
print("Classification F1-score (manual calculation):", f1_classification_manual)

print("Misbehavior Index Accuracy:", accuracy_index)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

save_dir = "models"
os.makedirs(save_dir, exist_ok=True)

model_save_path = os.path.join(save_dir, "veremi_images_m_1x2_ea_i.keras")
model.save(model_save_path)

print(f"\nModel saved at: {model_save_path}")

Epoch 1/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - classification_output_accuracy: 0.4618 - classification_output_loss: 1.7997 - index_output_accuracy: 0.9921 - index_output_loss: 0.0278 - loss: 1.8275 - val_classification_output_accuracy: 0.4683 - val_classification_output_loss: 1.8129 - val_index_output_accuracy: 1.0000 - val_index_output_loss: 0.0326 - val_loss: 1.8493
Epoch 2/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - classification_output_accuracy: 0.5159 - classification_output_loss: 1.5525 - index_output_accuracy: 0.9999 - index_output_loss: 0.0010 - loss: 1.5535 - val_classification_output_accuracy: 0.4768 - val_classification_output_loss: 1.9648 - val_index_output_accuracy: 1.0000 - val_index_output_loss: 5.7373e-04 - val_loss: 1.9712
Epoch 3/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - classification_output_accuracy: 0.5195 - classification_output_loss: 1.5222 - index_output_accuracy: 0.9999 - index_output_loss: 7.8150e-04 - loss: 1.5230 - val_classification_output_

In [17]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (1, 2, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_1x2_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_1x2_ea.csv')
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X, Y_classification, Y_index = [], [], []
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path) / 255.0
            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)
    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accs, precs, recs, f1s, idx_accs = [], [], [], [], []
start_time = time.time()
fold = 1

for train_index, test_index in skf.split(X, Y_classification):
    X_train, X_test = X[train_index], X[test_index]
    Y_train_classification, Y_test_classification = Y_classification[train_index], Y_classification[test_index]
    Y_train_index, Y_test_index = Y_index[train_index], Y_index[test_index]

    input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))
    x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)

    classification_output = Dense(20, activation='softmax', name='classification_output')(x)
    index_output = Dense(3, activation='softmax', name='index_output')(x)

    model = Model(inputs=input_layer, outputs=[classification_output, index_output])
    model.compile(optimizer='adam',
                  loss={'classification_output': 'sparse_categorical_crossentropy',
                        'index_output': 'sparse_categorical_crossentropy'},
                  metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(X_train,
              {'classification_output': Y_train_classification, 'index_output': Y_train_index},
              epochs=FAC_EPOCHS,
              batch_size=FAC_BATCH_SIZE,
              validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
              callbacks=[early_stopping],
              verbose=0)

    predictions = model.predict(X_test, verbose=0)
    Y_pred_classification = np.argmax(predictions[0], axis=1)
    Y_pred_index = np.argmax(predictions[1], axis=1)

    accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
    precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    accuracy_index = accuracy_score(Y_test_index, Y_pred_index)

    print(f"Fold {fold} - Acc: {accuracy_classification:.4f}, Prec: {precision_classification:.4f}, Rec: {recall_classification:.4f}, F1: {f1_classification:.4f}, Index Acc: {accuracy_index:.4f}")

    accs.append(accuracy_classification)
    precs.append(precision_classification)
    recs.append(recall_classification)
    f1s.append(f1_classification)
    idx_accs.append(accuracy_index)
    fold += 1

print("\nCross Validation Results:")
print(f"Classification Accuracy: Mean={np.mean(accs):.4f}, Std={np.std(accs):.4f}")
print(f"Precision: Mean={np.mean(precs):.4f}, Std={np.std(precs):.4f}")
print(f"Recall: Mean={np.mean(recs):.4f}, Std={np.std(recs):.4f}")
print(f"F1-score: Mean={np.mean(f1s):.4f}, Std={np.std(f1s):.4f}")
print(f"Misbehavior Index Accuracy: Mean={np.mean(idx_accs):.4f}, Std={np.std(idx_accs):.4f}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Fold 1 - Acc: 0.6046, Prec: 0.3765, Rec: 0.3608, F1: 0.3510, Index Acc: 0.9995
Fold 2 - Acc: 0.6652, Prec: 0.4752, Rec: 0.4587, F1: 0.4481, Index Acc: 0.9995
Fold 3 - Acc: 0.6250, Prec: 0.4287, Rec: 0.3937, F1: 0.3941, Index Acc: 0.9997
Fold 4 - Acc: 0.6214, Prec: 0.4142, Rec: 0.3879, F1: 0.3760, Index Acc: 1.0000
Fold 5 - Acc: 0.5536, Prec: 0.2820, Rec: 0.2783, F1: 0.2545, Index Acc: 1.0000

Cross Validation Results:
Classification Accuracy: Mean=0.6140, Std=0.0361
Precision: Mean=0.3953, Std=0.0649
Recall: Mean=0.3759, Std=0.0584
F1-score: Mean=0.3647, Std=0.0637
Misbehavior Index Accuracy: Mean=0.9997, Std=0.0002

Training time: 326.989568 seconds


In [18]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_1x2_ea'
FAC_LABELS_CSV = 'multiple_class_1x2_ea.csv'
FAC_IMAGE_SIZE = (1, 2)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

random_index = np.random.choice(valid_labels.index)
selected_label = valid_labels.loc[random_index]
selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

def preprocess_image(file_path, target_size=(1, 2)):
    """Load and preprocess a .npy image file for prediction."""
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

image_array = preprocess_image(selected_image_path)
image_array = np.expand_dims(image_array, axis=0)

predictions = model.predict(image_array)

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

if not os.path.exists(selected_image_path):
    raise FileNotFoundError(f"File {selected_image_path} not found.")

start_time = time.time()
try:
    image_array = preprocess_image(selected_image_path)
    image_array = np.expand_dims(image_array, axis=0)
    predictions = model.predict(image_array)
    predicted_classification = np.argmax(predictions[0])
    predicted_index = np.argmax(predictions[1])
    
    print(f"True Classification: {selected_label['classification']}, True Misbehavior Index: {selected_label['misbehavior_index']}")
    print(f"Predicted Classification: {predicted_classification}, Predicted Misbehavior Index: {predicted_index}")
except Exception as e:
    print(f"Error: {e}")
end_time = time.time()

print(f"\nElapsed time: {end_time - start_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
True Classification: 13, True Misbehavior Index: 2
Predicted Classification: 13, Predicted Misbehavior Index: 2

Elapsed time: 0.068591 seconds


In [19]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_1x2_ea'
FAC_LABELS_CSV = 'multiple_class_1x2_ea.csv'
FAC_IMAGE_SIZE = (1, 2)
NUM_SAMPLES = 500
TIME_THRESHOLD = 0.1

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

def preprocess_image(file_path, target_size=(1, 2)):
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

total_classification_correct = 0
total_index_correct = 0
total_prediction_time = 0.0
predictions_above_threshold = 0
total_samples_processed = 0

true_classifications = []
predicted_classifications = []
true_indices = []
predicted_indices = []

for _ in range(NUM_SAMPLES):
    random_index = np.random.choice(valid_labels.index)
    selected_label = valid_labels.loc[random_index]
    selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

    if not os.path.exists(selected_image_path):
        continue

    try:
        start_time = time.time()
        image_array = preprocess_image(selected_image_path)
        image_array = np.expand_dims(image_array, axis=0)

        predictions = model.predict(image_array)
        predicted_classification = np.argmax(predictions[0])
        predicted_index = np.argmax(predictions[1])

        true_classifications.append(int(selected_label['classification']))
        predicted_classifications.append(predicted_classification)
        true_indices.append(int(selected_label['misbehavior_index']))
        predicted_indices.append(predicted_index)

        if predicted_classification == int(selected_label['classification']):
            total_classification_correct += 1
        if predicted_index == int(selected_label['misbehavior_index']):
            total_index_correct += 1

        prediction_time = time.time() - start_time
        total_prediction_time += prediction_time

        if prediction_time >= TIME_THRESHOLD:
            predictions_above_threshold += 1

        total_samples_processed += 1

    except Exception as e:
        print(f"Error during prediction: {e}")

classification_cm = confusion_matrix(true_classifications, predicted_classifications)
index_cm = confusion_matrix(true_indices, predicted_indices)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    total = np.sum(cm)

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    accuracy = np.sum(tp) / total if total > 0 else 0.0

    return accuracy, precision, recall, f1

classification_accuracy, classification_precision, classification_recall, classification_f1 = calculate_metrics(classification_cm)
index_accuracy, index_precision, index_recall, index_f1 = calculate_metrics(index_cm)

print("\nClassification Metrics:")
print(f"Accuracy: {classification_accuracy:.2f}")
print(f"Average Precision: {np.mean(classification_precision):.2f}")
print(f"Average Recall: {np.mean(classification_recall):.2f}")
print(f"Average F1-Score: {np.mean(classification_f1):.2f}")

print("\nIndex Prediction Metrics:")
print(f"Accuracy: {index_accuracy:.2f}")
print(f"Average Precision: {np.mean(index_precision):.2f}")
print(f"Average Recall: {np.mean(index_recall):.2f}")
print(f"Average F1-Score: {np.mean(index_f1):.2f}")

print(f"\nAverage Prediction Time: {total_prediction_time / total_samples_processed:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━

C:\Users\david\AppData\Local\Temp\ipykernel_15464\1336652421.py:91: RuntimeWarning: invalid value encountered in divide
  precision = np.nan_to_num(tp / (tp + fp))
C:\Users\david\AppData\Local\Temp\ipykernel_15464\1336652421.py:93: RuntimeWarning: invalid value encountered in divide
  f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))


In [219]:
import numpy as np
import pandas as pd
import os

FAC_IMAGE_WIDTH = 3
FAC_IMAGE_HEIGHT = 1
FAC_NUM_CHANNELS = 12
FAC_SPECIAL_VALUE = 255

final_numeric_columns = [
    'sendTime', 'senderPseudo', 'posx', 'posy', 'posx_n', 'posy_n',
    'spdx_n', 'spdy_n', 'aclx_n', 'acly_n', 'hedx_n', 'hedy_n'
]

min_max_values = {
    col: (final_df[col].min(), final_df[col].max()) for col in final_numeric_columns if col != 'class'
}

def normalize_and_scale(value, min_value, max_value):
    if pd.isna(value) or not np.isfinite(value):
        return FAC_SPECIAL_VALUE
    if max_value == min_value:
        return 0 if value == min_value else FAC_SPECIAL_VALUE
    scaled_value = 255 * (value - min_value) / (max_value - min_value)
    return int(np.clip(scaled_value, 0, 255))

def records_to_image(group, misbehavior_index=None):
    image_data = np.zeros((FAC_IMAGE_HEIGHT, FAC_IMAGE_WIDTH, FAC_NUM_CHANNELS), dtype=np.uint8)

    group = group.sort_values('sendTime')
    
    for i, (_, row) in enumerate(group.iterrows()):
        row_pos = 0
        col_pos = i % FAC_IMAGE_WIDTH
        
        for j, col in enumerate(final_numeric_columns[:-1]):
            min_value, max_value = min_max_values[col]
            value = normalize_and_scale(row[col], min_value, max_value)
            image_data[row_pos, col_pos, j] = value

    if misbehavior_index is not None:
        misbehavior_row = 0
        misbehavior_col = (misbehavior_index - 1) % FAC_IMAGE_WIDTH
        image_data[misbehavior_row, misbehavior_col, 0] = 255
        image_data[misbehavior_row, misbehavior_col, 1] = 0

    return image_data

output_dir = 'veremi_multilevel_images_1x3_ea'
os.makedirs(output_dir, exist_ok=True)

classification_results = []
image_counter = 1

normal_messages = final_df[final_df['class'] == 0]
attack_messages = final_df[final_df['class'] != 0]

for _ in range(8000):
    for obu_id, group in normal_messages.groupby('senderPseudo'):
        if len(group) < 3:
            group = group.sample(n=3, replace=True, random_state=image_counter)
        else:
            group = group.sample(n=3, random_state=image_counter)

        image_data = records_to_image(chunk)
        misbehavior_index = 0

        npy_file_name = f'{output_dir}/image_{image_counter}.npy'
        np.save(npy_file_name, image_data)

        classification_results.append({'npy_file_name': npy_file_name, 'classification': 0, 'misbehavior_index': misbehavior_index})

        image_counter += 1
        break

for misbehavior_class in range(1, 20):
    for _ in range(600):
        for obu_id, group in normal_messages.groupby('senderPseudo'):
            normal_chunk = group.sample(n=2, random_state=image_counter)
            attack_chunk = attack_messages[attack_messages['class'] == misbehavior_class].sample(n=1, random_state=image_counter)

            combined_chunk = pd.concat([normal_chunk, attack_chunk])

            misbehavior_index = np.random.randint(1, 4)
            image_data = records_to_image(combined_chunk, misbehavior_index)

            npy_file_name = f'{output_dir}/image_{image_counter}.npy'
            np.save(npy_file_name, image_data)

            classification_results.append({
                'npy_file_name': npy_file_name, 
                'classification': misbehavior_class, 
                'misbehavior_index': misbehavior_index
            })

            image_counter += 1
            break

classification_df = pd.DataFrame(classification_results)
classification_df.to_csv('multiple_class_1x3_ea.csv', index=False)

print("Classification results saved to 'multiple_class_1x3_ea.csv'.")

Classification results saved to 'multiple_class_1x3_ea.csv'.


In [13]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (1, 3, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_1x3_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_1x3_ea.csv')

fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X = []
    Y_classification = []
    Y_index = []
    
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path)
            image_array = image_array / 255.0

            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)

    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

X_train, X_test, Y_train_classification, Y_test_classification, Y_train_index, Y_test_index = train_test_split(
    X, Y_classification, Y_index, test_size=0.2, random_state=42)

input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))

x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

classification_output = Dense(20, activation='softmax', name='classification_output')(x)

index_output = Dense(4, activation='softmax', name='index_output')(x)

model = Model(inputs=input_layer, outputs=[classification_output, index_output])

model.compile(optimizer='adam', 
              loss={'classification_output': 'sparse_categorical_crossentropy', 
                    'index_output': 'sparse_categorical_crossentropy'},
              metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, 
          {'classification_output': Y_train_classification, 'index_output': Y_train_index}, 
          epochs=FAC_EPOCHS, 
          batch_size=FAC_BATCH_SIZE, 
          validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
          callbacks=[early_stopping])

predictions = model.predict(X_test)
Y_pred_classification = np.argmax(predictions[0], axis=1)
Y_pred_index = np.argmax(predictions[1], axis=1)

accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro')
recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro')
f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro')

accuracy_index = accuracy_score(Y_test_index, Y_pred_index)
f1_classification_manual = 2 * (precision_classification * recall_classification) / (precision_classification + recall_classification)

print("Classification Accuracy:", accuracy_classification)
print("Classification Precision:", precision_classification)
print("Classification Recall:", recall_classification)
print("Classification F1-score (sklearn):", f1_classification)
print("Classification F1-score (manual calculation):", f1_classification_manual)

print("Misbehavior Index Accuracy:", accuracy_index)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

save_dir = "models"
os.makedirs(save_dir, exist_ok=True)

model_save_path = os.path.join(save_dir, "veremi_images_m_1x3_ea_i.keras")
model.save(model_save_path)

print(f"\nModel saved at: {model_save_path}")

Epoch 1/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - classification_output_accuracy: 0.4587 - classification_output_loss: 1.8214 - index_output_accuracy: 0.9939 - index_output_loss: 0.0394 - loss: 1.8608 - val_classification_output_accuracy: 0.4567 - val_classification_output_loss: 1.8232 - val_index_output_accuracy: 0.9191 - val_index_output_loss: 0.2065 - val_loss: 2.0345
Epoch 2/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - classification_output_accuracy: 0.5064 - classification_output_loss: 1.5872 - index_output_accuracy: 0.9999 - index_output_loss: 0.0039 - loss: 1.5911 - val_classification_output_accuracy: 0.4869 - val_classification_output_loss: 1.6633 - val_index_output_accuracy: 1.0000 - val_index_output_loss: 0.0034 - val_loss: 1.6721
Epoch 3/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - classification_output_accuracy: 0.5225 - classification_output_loss: 1.5128 - index_output_accuracy: 0.9992 - index_output_loss: 0.0039 - loss: 1.5167 - val_classification_output_accurac

In [20]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (1, 3, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_1x3_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_1x3_ea.csv')
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X, Y_classification, Y_index = [], [], []
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path) / 255.0
            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)
    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accs, precs, recs, f1s, idx_accs = [], [], [], [], []
start_time = time.time()
fold = 1

for train_index, test_index in skf.split(X, Y_classification):
    X_train, X_test = X[train_index], X[test_index]
    Y_train_classification, Y_test_classification = Y_classification[train_index], Y_classification[test_index]
    Y_train_index, Y_test_index = Y_index[train_index], Y_index[test_index]

    input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))
    x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)

    classification_output = Dense(20, activation='softmax', name='classification_output')(x)
    index_output = Dense(4, activation='softmax', name='index_output')(x)

    model = Model(inputs=input_layer, outputs=[classification_output, index_output])
    model.compile(optimizer='adam',
                  loss={'classification_output': 'sparse_categorical_crossentropy',
                        'index_output': 'sparse_categorical_crossentropy'},
                  metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(X_train,
              {'classification_output': Y_train_classification, 'index_output': Y_train_index},
              epochs=FAC_EPOCHS,
              batch_size=FAC_BATCH_SIZE,
              validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
              callbacks=[early_stopping],
              verbose=0)

    predictions = model.predict(X_test, verbose=0)
    Y_pred_classification = np.argmax(predictions[0], axis=1)
    Y_pred_index = np.argmax(predictions[1], axis=1)

    accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
    precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    accuracy_index = accuracy_score(Y_test_index, Y_pred_index)

    print(f"Fold {fold} - Acc: {accuracy_classification:.4f}, Prec: {precision_classification:.4f}, Rec: {recall_classification:.4f}, F1: {f1_classification:.4f}, Index Acc: {accuracy_index:.4f}")

    accs.append(accuracy_classification)
    precs.append(precision_classification)
    recs.append(recall_classification)
    f1s.append(f1_classification)
    idx_accs.append(accuracy_index)
    fold += 1

print("\nCross Validation Results:")
print(f"Classification Accuracy: Mean={np.mean(accs):.4f}, Std={np.std(accs):.4f}")
print(f"Precision: Mean={np.mean(precs):.4f}, Std={np.std(precs):.4f}")
print(f"Recall: Mean={np.mean(recs):.4f}, Std={np.std(recs):.4f}")
print(f"F1-score: Mean={np.mean(f1s):.4f}, Std={np.std(f1s):.4f}")
print(f"Misbehavior Index Accuracy: Mean={np.mean(idx_accs):.4f}, Std={np.std(idx_accs):.4f}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Fold 1 - Acc: 0.5987, Prec: 0.3958, Rec: 0.3513, F1: 0.3242, Index Acc: 0.9995
Fold 2 - Acc: 0.5178, Prec: 0.2475, Rec: 0.2204, F1: 0.2041, Index Acc: 0.9997
Fold 3 - Acc: 0.5879, Prec: 0.3578, Rec: 0.3337, F1: 0.3297, Index Acc: 0.9995
Fold 4 - Acc: 0.5057, Prec: 0.2508, Rec: 0.2008, F1: 0.1834, Index Acc: 1.0000
Fold 5 - Acc: 0.6129, Prec: 0.4376, Rec: 0.3742, F1: 0.3574, Index Acc: 0.9997

Cross Validation Results:
Classification Accuracy: Mean=0.5646, Std=0.0440
Precision: Mean=0.3379, Std=0.0768
Recall: Mean=0.2961, Std=0.0712
F1-score: Mean=0.2798, Std=0.0714
Misbehavior Index Accuracy: Mean=0.9997, Std=0.0002

Training time: 302.912666 seconds


In [32]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_1x3_ea'
FAC_LABELS_CSV = 'multiple_class_1x3_ea.csv'
FAC_IMAGE_SIZE = (1, 3)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

random_index = np.random.choice(valid_labels.index)
selected_label = valid_labels.loc[random_index]
selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

def preprocess_image(file_path, target_size=(1, 3)):
    """Load and preprocess a .npy image file for prediction."""
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

image_array = preprocess_image(selected_image_path)
image_array = np.expand_dims(image_array, axis=0)

predictions = model.predict(image_array)

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

if not os.path.exists(selected_image_path):
    raise FileNotFoundError(f"File {selected_image_path} not found.")

start_time = time.time()
try:
    image_array = preprocess_image(selected_image_path)
    image_array = np.expand_dims(image_array, axis=0)
    predictions = model.predict(image_array)
    predicted_classification = np.argmax(predictions[0])
    predicted_index = np.argmax(predictions[1])
    
    print(f"True Classification: {selected_label['classification']}, True Misbehavior Index: {selected_label['misbehavior_index']}")
    print(f"Predicted Classification: {predicted_classification}, Predicted Misbehavior Index: {predicted_index}")
except Exception as e:
    print(f"Error: {e}")
end_time = time.time()

print(f"\nElapsed time: {end_time - start_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
True Classification: 18, True Misbehavior Index: 3
Predicted Classification: 18, Predicted Misbehavior Index: 3

Elapsed time: 0.059296 seconds


In [34]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_1x3_ea'
FAC_LABELS_CSV = 'multiple_class_1x3_ea.csv'
FAC_IMAGE_SIZE = (1, 3)
NUM_SAMPLES = 500
TIME_THRESHOLD = 0.1

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

def preprocess_image(file_path, target_size=(1, 3)):
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

total_classification_correct = 0
total_index_correct = 0
total_prediction_time = 0.0
predictions_above_threshold = 0
total_samples_processed = 0

true_classifications = []
predicted_classifications = []
true_indices = []
predicted_indices = []

for _ in range(NUM_SAMPLES):
    random_index = np.random.choice(valid_labels.index)
    selected_label = valid_labels.loc[random_index]
    selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

    if not os.path.exists(selected_image_path):
        continue

    try:
        start_time = time.time()
        image_array = preprocess_image(selected_image_path)
        image_array = np.expand_dims(image_array, axis=0)

        predictions = model.predict(image_array)
        predicted_classification = np.argmax(predictions[0])
        predicted_index = np.argmax(predictions[1])

        true_classifications.append(int(selected_label['classification']))
        predicted_classifications.append(predicted_classification)
        true_indices.append(int(selected_label['misbehavior_index']))
        predicted_indices.append(predicted_index)

        if predicted_classification == int(selected_label['classification']):
            total_classification_correct += 1
        if predicted_index == int(selected_label['misbehavior_index']):
            total_index_correct += 1

        prediction_time = time.time() - start_time
        total_prediction_time += prediction_time

        if prediction_time >= TIME_THRESHOLD:
            predictions_above_threshold += 1

        total_samples_processed += 1

    except Exception as e:
        print(f"Error during prediction: {e}")

classification_cm = confusion_matrix(true_classifications, predicted_classifications)
index_cm = confusion_matrix(true_indices, predicted_indices)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    total = np.sum(cm)

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    accuracy = np.sum(tp) / total if total > 0 else 0.0

    return accuracy, precision, recall, f1

classification_accuracy, classification_precision, classification_recall, classification_f1 = calculate_metrics(classification_cm)
index_accuracy, index_precision, index_recall, index_f1 = calculate_metrics(index_cm)

print("\nClassification Metrics:")
print(f"Accuracy: {classification_accuracy:.2f}")
print(f"Average Precision: {np.mean(classification_precision):.2f}")
print(f"Average Recall: {np.mean(classification_recall):.2f}")
print(f"Average F1-Score: {np.mean(classification_f1):.2f}")

print("\nIndex Prediction Metrics:")
print(f"Accuracy: {index_accuracy:.2f}")
print(f"Average Precision: {np.mean(index_precision):.2f}")
print(f"Average Recall: {np.mean(index_recall):.2f}")
print(f"Average F1-Score: {np.mean(index_f1):.2f}")

print(f"\nAverage Prediction Time: {total_prediction_time / total_samples_processed:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━

In [209]:
import numpy as np
import pandas as pd
import os

FAC_IMAGE_WIDTH = 4
FAC_IMAGE_HEIGHT = 1
FAC_NUM_CHANNELS = 12
FAC_SPECIAL_VALUE = 255

final_numeric_columns = [
    'sendTime', 'senderPseudo', 'posx', 'posy', 'posx_n', 'posy_n',
    'spdx_n', 'spdy_n', 'aclx_n', 'acly_n', 'hedx_n', 'hedy_n'
]

min_max_values = {
    col: (final_df[col].min(), final_df[col].max()) for col in final_numeric_columns if col != 'class'
}

def normalize_and_scale(value, min_value, max_value):
    if pd.isna(value) or not np.isfinite(value):
        return FAC_SPECIAL_VALUE
    if max_value == min_value:
        return 0 if value == min_value else FAC_SPECIAL_VALUE
    scaled_value = 255 * (value - min_value) / (max_value - min_value)
    return int(np.clip(scaled_value, 0, 255))

def records_to_image(group, misbehavior_index=None):
    image_data = np.zeros((FAC_IMAGE_HEIGHT, FAC_IMAGE_WIDTH, FAC_NUM_CHANNELS), dtype=np.uint8)

    group = group.sort_values('sendTime')
    
    for i, (_, row) in enumerate(group.iterrows()):
        row_pos = 0
        col_pos = i % FAC_IMAGE_WIDTH
        
        for j, col in enumerate(final_numeric_columns[:-1]):
            min_value, max_value = min_max_values[col]
            value = normalize_and_scale(row[col], min_value, max_value)
            image_data[row_pos, col_pos, j] = value

    if misbehavior_index is not None:
        misbehavior_row = 0
        misbehavior_col = (misbehavior_index - 1) % FAC_IMAGE_WIDTH
        image_data[misbehavior_row, misbehavior_col, 0] = 255
        image_data[misbehavior_row, misbehavior_col, 1] = 0

    return image_data

output_dir = 'veremi_multilevel_images_1x4_ea'
os.makedirs(output_dir, exist_ok=True)

classification_results = []
image_counter = 1

normal_messages = final_df[final_df['class'] == 0]
attack_messages = final_df[final_df['class'] != 0]

for _ in range(8000):
    for obu_id, group in normal_messages.groupby('senderPseudo'):
        if len(group) < 4:
            group = group.sample(n=4, replace=True, random_state=image_counter)
        else:
            group = group.sample(n=4, random_state=image_counter)

        image_data = records_to_image(group)
        misbehavior_index = 0

        npy_file_name = f'{output_dir}/image_{image_counter}.npy'
        np.save(npy_file_name, image_data)

        classification_results.append({
            'npy_file_name': npy_file_name, 
            'classification': 0, 
            'misbehavior_index': misbehavior_index
        })

        image_counter += 1
        break

for misbehavior_class in range(1, 20):
    for _ in range(600):
        for obu_id, group in normal_messages.groupby('senderPseudo'):
            if len(group) < 3:
                normal_chunk = group.sample(n=3, replace=True, random_state=image_counter)
            else:
                normal_chunk = group.sample(n=3, random_state=image_counter)
            attack_chunk = attack_messages[attack_messages['class'] == misbehavior_class].sample(n=1, random_state=image_counter)
            
            combined_chunk = pd.concat([normal_chunk, attack_chunk])

            misbehavior_index = np.random.randint(1, 5)
            image_data = records_to_image(combined_chunk, misbehavior_index)

            npy_file_name = f'{output_dir}/image_{image_counter}.npy'
            np.save(npy_file_name, image_data)
           
            classification_results.append({
                'npy_file_name': npy_file_name, 
                'classification': misbehavior_class,  # Misbehavior class value assigned
                'misbehavior_index': misbehavior_index
            })

            image_counter += 1
            break

classification_df = pd.DataFrame(classification_results)
classification_df.to_csv('multiple_class_1x4_ea.csv', index=False)

print("Classification results saved to 'multiple_class_1x4_ea.csv'.")

Classification results saved to 'multiple_class_1x4_ea.csv'.


In [14]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (1, 4, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_1x4_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_1x4_ea.csv')

fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X = []
    Y_classification = []
    Y_index = []
    
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path)
            image_array = image_array / 255.0

            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)

    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

X_train, X_test, Y_train_classification, Y_test_classification, Y_train_index, Y_test_index = train_test_split(
    X, Y_classification, Y_index, test_size=0.2, random_state=42)

input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))

x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

classification_output = Dense(20, activation='softmax', name='classification_output')(x)

index_output = Dense(5, activation='softmax', name='index_output')(x)

model = Model(inputs=input_layer, outputs=[classification_output, index_output])

model.compile(optimizer='adam', 
              loss={'classification_output': 'sparse_categorical_crossentropy', 
                    'index_output': 'sparse_categorical_crossentropy'},
              metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, 
          {'classification_output': Y_train_classification, 'index_output': Y_train_index}, 
          epochs=FAC_EPOCHS, 
          batch_size=FAC_BATCH_SIZE, 
          validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
          callbacks=[early_stopping])

predictions = model.predict(X_test)
Y_pred_classification = np.argmax(predictions[0], axis=1)
Y_pred_index = np.argmax(predictions[1], axis=1)

accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro')
recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro')
f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro')

accuracy_index = accuracy_score(Y_test_index, Y_pred_index)
f1_classification_manual = 2 * (precision_classification * recall_classification) / (precision_classification + recall_classification)

print("Classification Accuracy:", accuracy_classification)
print("Classification Precision:", precision_classification)
print("Classification Recall:", recall_classification)
print("Classification F1-score (sklearn):", f1_classification)
print("Classification F1-score (manual calculation):", f1_classification_manual)

print("Misbehavior Index Accuracy:", accuracy_index)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

save_dir = "models"
os.makedirs(save_dir, exist_ok=True)

model_save_path = os.path.join(save_dir, "veremi_images_m_1x4_ea_i.keras")
model.save(model_save_path)

print(f"\nModel saved at: {model_save_path}")

Epoch 1/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - classification_output_accuracy: 0.4582 - classification_output_loss: 1.8567 - index_output_accuracy: 0.9888 - index_output_loss: 0.0347 - loss: 1.8914 - val_classification_output_accuracy: 0.4716 - val_classification_output_loss: 1.7527 - val_index_output_accuracy: 0.9992 - val_index_output_loss: 0.0877 - val_loss: 1.8444
Epoch 2/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - classification_output_accuracy: 0.5130 - classification_output_loss: 1.5639 - index_output_accuracy: 1.0000 - index_output_loss: 8.1368e-04 - loss: 1.5648 - val_classification_output_accuracy: 0.5093 - val_classification_output_loss: 1.5544 - val_index_output_accuracy: 1.0000 - val_index_output_loss: 9.1694e-05 - val_loss: 1.5593
Epoch 3/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - classification_output_accuracy: 0.5283 - classification_output_loss: 1.4869 - index_output_accuracy: 0.9997 - index_output_loss: 0.0015 - loss: 1.4884 - val_classification_output

In [21]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (1, 4, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_1x4_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_1x4_ea.csv')
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X, Y_classification, Y_index = [], [], []
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path) / 255.0
            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)
    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accs, precs, recs, f1s, idx_accs = [], [], [], [], []
start_time = time.time()
fold = 1

for train_index, test_index in skf.split(X, Y_classification):
    X_train, X_test = X[train_index], X[test_index]
    Y_train_classification, Y_test_classification = Y_classification[train_index], Y_classification[test_index]
    Y_train_index, Y_test_index = Y_index[train_index], Y_index[test_index]

    input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))
    x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)

    classification_output = Dense(20, activation='softmax', name='classification_output')(x)
    index_output = Dense(5, activation='softmax', name='index_output')(x)

    model = Model(inputs=input_layer, outputs=[classification_output, index_output])
    model.compile(optimizer='adam',
                  loss={'classification_output': 'sparse_categorical_crossentropy',
                        'index_output': 'sparse_categorical_crossentropy'},
                  metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(X_train,
              {'classification_output': Y_train_classification, 'index_output': Y_train_index},
              epochs=FAC_EPOCHS,
              batch_size=FAC_BATCH_SIZE,
              validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
              callbacks=[early_stopping],
              verbose=0)

    predictions = model.predict(X_test, verbose=0)
    Y_pred_classification = np.argmax(predictions[0], axis=1)
    Y_pred_index = np.argmax(predictions[1], axis=1)

    accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
    precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    accuracy_index = accuracy_score(Y_test_index, Y_pred_index)

    print(f"Fold {fold} - Acc: {accuracy_classification:.4f}, Prec: {precision_classification:.4f}, Rec: {recall_classification:.4f}, F1: {f1_classification:.4f}, Index Acc: {accuracy_index:.4f}")

    accs.append(accuracy_classification)
    precs.append(precision_classification)
    recs.append(recall_classification)
    f1s.append(f1_classification)
    idx_accs.append(accuracy_index)
    fold += 1

print("\nCross Validation Results:")
print(f"Classification Accuracy: Mean={np.mean(accs):.4f}, Std={np.std(accs):.4f}")
print(f"Precision: Mean={np.mean(precs):.4f}, Std={np.std(precs):.4f}")
print(f"Recall: Mean={np.mean(recs):.4f}, Std={np.std(recs):.4f}")
print(f"F1-score: Mean={np.mean(f1s):.4f}, Std={np.std(f1s):.4f}")
print(f"Misbehavior Index Accuracy: Mean={np.mean(idx_accs):.4f}, Std={np.std(idx_accs):.4f}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Fold 1 - Acc: 0.6727, Prec: 0.4971, Rec: 0.4708, F1: 0.4592, Index Acc: 0.9997
Fold 2 - Acc: 0.5332, Prec: 0.3090, Rec: 0.2454, F1: 0.2312, Index Acc: 0.9997
Fold 3 - Acc: 0.6683, Prec: 0.4846, Rec: 0.4637, F1: 0.4596, Index Acc: 0.9990
Fold 4 - Acc: 0.6090, Prec: 0.4374, Rec: 0.3679, F1: 0.3632, Index Acc: 0.9987
Fold 5 - Acc: 0.5165, Prec: 0.2071, Rec: 0.2183, F1: 0.1839, Index Acc: 1.0000

Cross Validation Results:
Classification Accuracy: Mean=0.5999, Std=0.0655
Precision: Mean=0.3871, Std=0.1119
Recall: Mean=0.3533, Std=0.1059
F1-score: Mean=0.3394, Std=0.1142
Misbehavior Index Accuracy: Mean=0.9994, Std=0.0005

Training time: 330.366508 seconds


In [42]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_1x4_ea'
FAC_LABELS_CSV = 'multiple_class_1x4_ea.csv'
FAC_IMAGE_SIZE = (1, 4)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

random_index = np.random.choice(valid_labels.index)
selected_label = valid_labels.loc[random_index]
selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

def preprocess_image(file_path, target_size=(1, 4)):
    """Load and preprocess a .npy image file for prediction."""
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

image_array = preprocess_image(selected_image_path)
image_array = np.expand_dims(image_array, axis=0)

predictions = model.predict(image_array)

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

if not os.path.exists(selected_image_path):
    raise FileNotFoundError(f"File {selected_image_path} not found.")

start_time = time.time()
try:
    image_array = preprocess_image(selected_image_path)
    image_array = np.expand_dims(image_array, axis=0)
    predictions = model.predict(image_array)
    predicted_classification = np.argmax(predictions[0])
    predicted_index = np.argmax(predictions[1])
    
    print(f"True Classification: {selected_label['classification']}, True Misbehavior Index: {selected_label['misbehavior_index']}")
    print(f"Predicted Classification: {predicted_classification}, Predicted Misbehavior Index: {predicted_index}")
except Exception as e:
    print(f"Error: {e}")
end_time = time.time()

print(f"\nElapsed time: {end_time - start_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
True Classification: 6, True Misbehavior Index: 4
Predicted Classification: 6, Predicted Misbehavior Index: 4

Elapsed time: 0.061949 seconds


In [43]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_1x4_ea'
FAC_LABELS_CSV = 'multiple_class_1x4_ea.csv'
FAC_IMAGE_SIZE = (1, 4)
NUM_SAMPLES = 500
TIME_THRESHOLD = 0.1

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

def preprocess_image(file_path, target_size=(1, 4)):
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

total_classification_correct = 0
total_index_correct = 0
total_prediction_time = 0.0
predictions_above_threshold = 0
total_samples_processed = 0

true_classifications = []
predicted_classifications = []
true_indices = []
predicted_indices = []

for _ in range(NUM_SAMPLES):
    random_index = np.random.choice(valid_labels.index)
    selected_label = valid_labels.loc[random_index]
    selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

    if not os.path.exists(selected_image_path):
        continue

    try:
        start_time = time.time()
        image_array = preprocess_image(selected_image_path)
        image_array = np.expand_dims(image_array, axis=0)

        predictions = model.predict(image_array)
        predicted_classification = np.argmax(predictions[0])
        predicted_index = np.argmax(predictions[1])

        true_classifications.append(int(selected_label['classification']))
        predicted_classifications.append(predicted_classification)
        true_indices.append(int(selected_label['misbehavior_index']))
        predicted_indices.append(predicted_index)

        if predicted_classification == int(selected_label['classification']):
            total_classification_correct += 1
        if predicted_index == int(selected_label['misbehavior_index']):
            total_index_correct += 1

        prediction_time = time.time() - start_time
        total_prediction_time += prediction_time

        if prediction_time >= TIME_THRESHOLD:
            predictions_above_threshold += 1

        total_samples_processed += 1

    except Exception as e:
        print(f"Error during prediction: {e}")

classification_cm = confusion_matrix(true_classifications, predicted_classifications)
index_cm = confusion_matrix(true_indices, predicted_indices)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    total = np.sum(cm)

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    accuracy = np.sum(tp) / total if total > 0 else 0.0

    return accuracy, precision, recall, f1

classification_accuracy, classification_precision, classification_recall, classification_f1 = calculate_metrics(classification_cm)
index_accuracy, index_precision, index_recall, index_f1 = calculate_metrics(index_cm)

print("\nClassification Metrics:")
print(f"Accuracy: {classification_accuracy:.2f}")
print(f"Average Precision: {np.mean(classification_precision):.2f}")
print(f"Average Recall: {np.mean(classification_recall):.2f}")
print(f"Average F1-Score: {np.mean(classification_f1):.2f}")

print("\nIndex Prediction Metrics:")
print(f"Accuracy: {index_accuracy:.2f}")
print(f"Average Precision: {np.mean(index_precision):.2f}")
print(f"Average Recall: {np.mean(index_recall):.2f}")
print(f"Average F1-Score: {np.mean(index_f1):.2f}")

print(f"\nAverage Prediction Time: {total_prediction_time / total_samples_processed:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━

C:\Users\david\AppData\Local\Temp\ipykernel_12424\1744340218.py:93: RuntimeWarning: invalid value encountered in divide
  f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))


In [17]:
import numpy as np
import pandas as pd
import os

FAC_IMAGE_WIDTH = 5
FAC_IMAGE_HEIGHT = 1
FAC_NUM_CHANNELS = 12
FAC_SPECIAL_VALUE = 255

final_numeric_columns = [
    'sendTime', 'senderPseudo', 'posx', 'posy', 'posx_n', 'posy_n',
    'spdx_n', 'spdy_n', 'aclx_n', 'acly_n', 'hedx_n', 'hedy_n'
]

min_max_values = {
    col: (final_df[col].min(), final_df[col].max()) for col in final_numeric_columns if col != 'class'
}

def normalize_and_scale(value, min_value, max_value):
    if pd.isna(value) or not np.isfinite(value):
        return FAC_SPECIAL_VALUE
    if max_value == min_value:
        return 0 if value == min_value else FAC_SPECIAL_VALUE
    scaled_value = 255 * (value - min_value) / (max_value - min_value)
    return int(np.clip(scaled_value, 0, 255))

def records_to_image(group, misbehavior_index=None):
    image_data = np.zeros((FAC_IMAGE_HEIGHT, FAC_IMAGE_WIDTH, FAC_NUM_CHANNELS), dtype=np.uint8)

    group = group.sort_values('sendTime')
    
    for i, (_, row) in enumerate(group.iterrows()):
        row_pos = 0
        col_pos = i % FAC_IMAGE_WIDTH
        
        for j, col in enumerate(final_numeric_columns[:-1]):
            min_value, max_value = min_max_values[col]
            value = normalize_and_scale(row[col], min_value, max_value)
            image_data[row_pos, col_pos, j] = value

    if misbehavior_index is not None:
        misbehavior_row = 0
        misbehavior_col = (misbehavior_index - 1) % FAC_IMAGE_WIDTH
        image_data[misbehavior_row, misbehavior_col, 0] = 255
        image_data[misbehavior_row, misbehavior_col, 1] = 0

    return image_data

output_dir = 'veremi_multilevel_images_1x5_ea'
os.makedirs(output_dir, exist_ok=True)

classification_results = []
image_counter = 1

normal_messages = final_df[final_df['class'] == 0]
attack_messages = final_df[final_df['class'] != 0]

for _ in range(2500):
    for obu_id, group in normal_messages.groupby('senderPseudo'):
        if len(group) < 5:
            group = group.sample(n=5, replace=True, random_state=image_counter)
        else:
            group = group.sample(n=5, random_state=image_counter)

        # Pass 'group' instead of 'chunk'
        image_data = records_to_image(group)
        misbehavior_index = 0

        npy_file_name = f'{output_dir}/image_{image_counter}.npy'
        np.save(npy_file_name, image_data)

        classification_results.append({'npy_file_name': npy_file_name, 'classification': 0, 'misbehavior_index': misbehavior_index})

        image_counter += 1
        break

for misbehavior_class in range(1, 20):
    for _ in range(400):
        for obu_id, group in normal_messages.groupby('senderPseudo'):
            if len(group) < 4:
                normal_chunk = group.sample(n=4, replace=True, random_state=image_counter)
            else:
                normal_chunk = group.sample(n=4, random_state=image_counter)
            attack_chunk = attack_messages[attack_messages['class'] == misbehavior_class].sample(n=1, random_state=image_counter)

            combined_chunk = pd.concat([normal_chunk, attack_chunk])

            misbehavior_index = np.random.randint(1, 6)
            image_data = records_to_image(combined_chunk, misbehavior_index)

            npy_file_name = f'{output_dir}/image_{image_counter}.npy'
            np.save(npy_file_name, image_data)

            classification_results.append({
                'npy_file_name': npy_file_name, 
                'classification': misbehavior_class,
                'misbehavior_index': misbehavior_index
            })

            image_counter += 1
            break

classification_df = pd.DataFrame(classification_results)
classification_df.to_csv('multiple_class_1x5_ea.csv', index=False)

print("Classification results saved to 'multiple_class_1x5_ea.csv'.")

Classification results saved to 'multiple_class_1x5_ea.csv'.


In [15]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (1, 5, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_1x5_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 20

fac_labels_df = pd.read_csv('multiple_class_1x5_ea.csv')

fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X = []
    Y_classification = []
    Y_index = []
    
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path)
            image_array = image_array / 255.0

            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)

    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

X_train, X_test, Y_train_classification, Y_test_classification, Y_train_index, Y_test_index = train_test_split(
    X, Y_classification, Y_index, test_size=0.2, random_state=42)

input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))

x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)

classification_output = Dense(20, activation='softmax', name='classification_output')(x)

index_output = Dense(6, activation='softmax', name='index_output')(x)

model = Model(inputs=input_layer, outputs=[classification_output, index_output])

model.compile(optimizer='adam', 
              loss={'classification_output': 'sparse_categorical_crossentropy', 
                    'index_output': 'sparse_categorical_crossentropy'},
              metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

start_time = time.time()
model.fit(X_train, 
          {'classification_output': Y_train_classification, 'index_output': Y_train_index}, 
          epochs=FAC_EPOCHS, 
          batch_size=FAC_BATCH_SIZE, 
          validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
          callbacks=[early_stopping, lr_scheduler])

predictions = model.predict(X_test)
Y_pred_classification = np.argmax(predictions[0], axis=1)
Y_pred_index = np.argmax(predictions[1], axis=1)

accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro')
recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro')
f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro')

accuracy_index = accuracy_score(Y_test_index, Y_pred_index)
f1_classification_manual = 2 * (precision_classification * recall_classification) / (precision_classification + recall_classification)

print("Classification Accuracy:", accuracy_classification)
print("Classification Precision:", precision_classification)
print("Classification Recall:", recall_classification)
print("Classification F1-score (sklearn):", f1_classification)
print("Classification F1-score (manual calculation):", f1_classification_manual)

print("Misbehavior Index Accuracy:", accuracy_index)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

save_dir = "models"
os.makedirs(save_dir, exist_ok=True)

model_save_path = os.path.join(save_dir, "veremi_images_m_1x5_ea_i.keras")
model.save(model_save_path)

print(f"\nModel saved at: {model_save_path}")

Epoch 1/20
253/253 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - classification_output_accuracy: 0.3160 - classification_output_loss: 2.5119 - index_output_accuracy: 0.9774 - index_output_loss: 0.0863 - loss: 2.5982 - val_classification_output_accuracy: 0.2530 - val_classification_output_loss: 2.7491 - val_index_output_accuracy: 0.2723 - val_index_output_loss: 2.1023 - val_loss: 4.8222 - learning_rate: 0.0010
Epoch 2/20
253/253 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - classification_output_accuracy: 0.3817 - classification_output_loss: 2.0660 - index_output_accuracy: 0.9997 - index_output_loss: 0.0037 - loss: 2.0698 - val_classification_output_accuracy: 0.0797 - val_classification_output_loss: 2.9534 - val_index_output_accuracy: 0.9911 - val_index_output_loss: 0.2505 - val_loss: 3.1931 - learning_rate: 0.0010
Epoch 3/20
253/253 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - classification_output_accuracy: 0.3689 - classification_output_loss: 2.0417 - index_output_accuracy: 0.9992 - index_output_loss: 0.0042 - l

Epoch 19/20
253/253 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - classification_output_accuracy: 0.6836 - classification_output_loss: 0.9773 - index_output_accuracy: 0.9994 - index_output_loss: 0.0019 - loss: 0.9792 - val_classification_output_accuracy: 0.6129 - val_classification_output_loss: 1.2697 - val_index_output_accuracy: 0.9980 - val_index_output_loss: 0.0117 - val_loss: 1.2866 - learning_rate: 5.0000e-04
Epoch 20/20
253/253 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - classification_output_accuracy: 0.6833 - classification_output_loss: 0.9825 - index_output_accuracy: 0.9997 - index_output_loss: 0.0017 - loss: 0.9842 - val_classification_output_accuracy: 0.4896 - val_classification_output_loss: 1.8145 - val_index_output_accuracy: 1.0000 - val_index_output_loss: 0.0068 - val_loss: 1.8258 - learning_rate: 5.0000e-04
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Classification Accuracy: 0.6222772277227723
Classification Precision: 0.5334869686529243
Classification Recall: 0.5235450916197777
Classification

In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (1, 5, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_1x5_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 20

fac_labels_df = pd.read_csv('multiple_class_1x5_ea.csv')
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X, Y_classification, Y_index = [], [], []
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path) / 255.0
            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)
    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accs, precs, recs, f1s, idx_accs = [], [], [], [], []
start_time = time.time()
fold = 1

for train_index, test_index in skf.split(X, Y_classification):
    X_train, X_test = X[train_index], X[test_index]
    Y_train_classification, Y_test_classification = Y_classification[train_index], Y_classification[test_index]
    Y_train_index, Y_test_index = Y_index[train_index], Y_index[test_index]

    input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))
    x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.1)(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.1)(x)

    classification_output = Dense(20, activation='softmax', name='classification_output')(x)
    index_output = Dense(6, activation='softmax', name='index_output')(x)

    model = Model(inputs=input_layer, outputs=[classification_output, index_output])
    model.compile(optimizer='adam',
                  loss={'classification_output': 'sparse_categorical_crossentropy',
                        'index_output': 'sparse_categorical_crossentropy'},
                  metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=0)

    model.fit(X_train,
              {'classification_output': Y_train_classification, 'index_output': Y_train_index},
              epochs=FAC_EPOCHS,
              batch_size=FAC_BATCH_SIZE,
              validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
              callbacks=[early_stopping, lr_scheduler],
              verbose=0)

    predictions = model.predict(X_test, verbose=0)
    Y_pred_classification = np.argmax(predictions[0], axis=1)
    Y_pred_index = np.argmax(predictions[1], axis=1)

    accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
    precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    accuracy_index = accuracy_score(Y_test_index, Y_pred_index)

    print(f"Fold {fold} - Acc: {accuracy_classification:.4f}, Prec: {precision_classification:.4f}, Rec: {recall_classification:.4f}, F1: {f1_classification:.4f}, Index Acc: {accuracy_index:.4f}")

    accs.append(accuracy_classification)
    precs.append(precision_classification)
    recs.append(recall_classification)
    f1s.append(f1_classification)
    idx_accs.append(accuracy_index)
    fold += 1

print("\nCross Validation Results:")
print(f"Classification Accuracy: Mean={np.mean(accs):.4f}, Std={np.std(accs):.4f}")
print(f"Precision: Mean={np.mean(precs):.4f}, Std={np.std(precs):.4f}")
print(f"Recall: Mean={np.mean(recs):.4f}, Std={np.std(recs):.4f}")
print(f"F1-score: Mean={np.mean(f1s):.4f}, Std={np.std(f1s):.4f}")
print(f"Misbehavior Index Accuracy: Mean={np.mean(idx_accs):.4f}, Std={np.std(idx_accs):.4f}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Fold 1 - Acc: 0.6158, Prec: 0.5302, Rec: 0.5150, F1: 0.5133, Index Acc: 0.9995
Fold 2 - Acc: 0.6510, Prec: 0.5582, Rec: 0.5594, F1: 0.5517, Index Acc: 0.9980
Fold 3 - Acc: 0.6644, Prec: 0.5855, Rec: 0.5763, F1: 0.5694, Index Acc: 0.9995
Fold 4 - Acc: 0.6693, Prec: 0.5946, Rec: 0.5825, F1: 0.5771, Index Acc: 1.0000
Fold 5 - Acc: 0.6470, Prec: 0.5607, Rec: 0.5544, F1: 0.5504, Index Acc: 1.0000

Cross Validation Results:
Classification Accuracy: Mean=0.6495, Std=0.0187
Precision: Mean=0.5658, Std=0.0227
Recall: Mean=0.5575, Std=0.0236
F1-score: Mean=0.5524, Std=0.0221
Misbehavior Index Accuracy: Mean=0.9994, Std=0.0007

Training time: 420.064208 seconds


In [3]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_1x5_ea'
FAC_LABELS_CSV = 'multiple_class_1x5_ea.csv'
FAC_IMAGE_SIZE = (1, 5)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

random_index = np.random.choice(valid_labels.index)
selected_label = valid_labels.loc[random_index]
selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

def preprocess_image(file_path, target_size=(1, 5)):
    """Load and preprocess a .npy image file for prediction."""
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

image_array = preprocess_image(selected_image_path)
image_array = np.expand_dims(image_array, axis=0)

predictions = model.predict(image_array)

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

if not os.path.exists(selected_image_path):
    raise FileNotFoundError(f"File {selected_image_path} not found.")

start_time = time.time()
try:
    image_array = preprocess_image(selected_image_path)
    image_array = np.expand_dims(image_array, axis=0)
    predictions = model.predict(image_array)
    predicted_classification = np.argmax(predictions[0])
    predicted_index = np.argmax(predictions[1])
    
    print(f"True Classification: {selected_label['classification']}, True Misbehavior Index: {selected_label['misbehavior_index']}")
    print(f"Predicted Classification: {predicted_classification}, Predicted Misbehavior Index: {predicted_index}")
except Exception as e:
    print(f"Error: {e}")
end_time = time.time()

print(f"\nElapsed time: {end_time - start_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
True Classification: 16, True Misbehavior Index: 5
Predicted Classification: 9, Predicted Misbehavior Index: 5

Elapsed time: 0.065484 seconds


In [4]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_1x5_ea'
FAC_LABELS_CSV = 'multiple_class_1x5_ea.csv'
FAC_IMAGE_SIZE = (1, 5)
NUM_SAMPLES = 500
TIME_THRESHOLD = 0.1

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

def preprocess_image(file_path, target_size=(1, 5)):
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

total_classification_correct = 0
total_index_correct = 0
total_prediction_time = 0.0
predictions_above_threshold = 0
total_samples_processed = 0

true_classifications = []
predicted_classifications = []
true_indices = []
predicted_indices = []

for _ in range(NUM_SAMPLES):
    random_index = np.random.choice(valid_labels.index)
    selected_label = valid_labels.loc[random_index]
    selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

    if not os.path.exists(selected_image_path):
        continue

    try:
        start_time = time.time()
        image_array = preprocess_image(selected_image_path)
        image_array = np.expand_dims(image_array, axis=0)

        predictions = model.predict(image_array)
        predicted_classification = np.argmax(predictions[0])
        predicted_index = np.argmax(predictions[1])

        true_classifications.append(int(selected_label['classification']))
        predicted_classifications.append(predicted_classification)
        true_indices.append(int(selected_label['misbehavior_index']))
        predicted_indices.append(predicted_index)

        if predicted_classification == int(selected_label['classification']):
            total_classification_correct += 1
        if predicted_index == int(selected_label['misbehavior_index']):
            total_index_correct += 1

        prediction_time = time.time() - start_time
        total_prediction_time += prediction_time

        if prediction_time >= TIME_THRESHOLD:
            predictions_above_threshold += 1

        total_samples_processed += 1

    except Exception as e:
        print(f"Error during prediction: {e}")

classification_cm = confusion_matrix(true_classifications, predicted_classifications)
index_cm = confusion_matrix(true_indices, predicted_indices)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    total = np.sum(cm)

    precision = np.divide(tp, (tp + fp), out=np.zeros_like(tp, dtype=float), where=(tp + fp) != 0)
    recall = np.divide(tp, (tp + fn), out=np.zeros_like(tp, dtype=float), where=(tp + fn) != 0)
    f1 = np.divide(2 * (precision * recall), (precision + recall), 
                   out=np.zeros_like(precision, dtype=float), where=(precision + recall) != 0)

    accuracy = np.sum(tp) / total if total > 0 else 0.0

    return accuracy, precision, recall, f1

classification_accuracy, classification_precision, classification_recall, classification_f1 = calculate_metrics(classification_cm)
index_accuracy, index_precision, index_recall, index_f1 = calculate_metrics(index_cm)

print("\nClassification Metrics:")
print(f"Accuracy: {classification_accuracy:.2f}")
print(f"Average Precision: {np.mean(classification_precision):.2f}")
print(f"Average Recall: {np.mean(classification_recall):.2f}")
print(f"Average F1-Score: {np.mean(classification_f1):.2f}")

print("\nIndex Prediction Metrics:")
print(f"Accuracy: {index_accuracy:.2f}")
print(f"Average Precision: {np.mean(index_precision):.2f}")
print(f"Average Recall: {np.mean(index_recall):.2f}")
print(f"Average F1-Score: {np.mean(index_f1):.2f}")

print(f"\nAverage Prediction Time: {total_prediction_time / total_samples_processed:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━

In [234]:
import numpy as np
import pandas as pd
import os

# Constants
FAC_IMAGE_WIDTH = 2
FAC_IMAGE_HEIGHT = 5
FAC_NUM_CHANNELS = 12
FAC_SPECIAL_VALUE = 255
NUM_CLASSES = 20

def normalize_and_scale(values, min_values, max_values):
    scaled = 255 * (values - min_values) / (max_values - min_values)
    scaled[np.isnan(values) | ~np.isfinite(values)] = FAC_SPECIAL_VALUE
    scaled[values == max_values] = 0
    return np.clip(scaled, 0, 255).astype(np.uint8)

def generate_image(group, misbehavior_index=None):
    min_values = np.array([group[col].min() for col in final_numeric_columns])
    max_values = np.array([group[col].max() for col in final_numeric_columns])

    data = group[final_numeric_columns].to_numpy()
    normalized_data = normalize_and_scale(data, min_values, max_values)

    reshaped_data = normalized_data.reshape(FAC_IMAGE_HEIGHT, FAC_IMAGE_WIDTH, FAC_NUM_CHANNELS)

    if misbehavior_index:
        row = (misbehavior_index - 1) // FAC_IMAGE_WIDTH
        col = (misbehavior_index - 1) % FAC_IMAGE_WIDTH
        reshaped_data[row, col, :3] = [255, 0, 0]
        
    return reshaped_data

final_numeric_columns = [
    'sendTime', 'senderPseudo', 'posx', 'posy', 'posx_n', 'posy_n',
    'spdx_n', 'spdy_n', 'aclx_n', 'acly_n', 'hedx_n', 'hedy_n'
]

output_dir = 'veremi_multilevel_images_2x5_ea'
os.makedirs(output_dir, exist_ok=True)

normal_messages = final_df[final_df['class'] == 0]

classification_results = []
image_counter = 1

for _ in range(8000):
    sampled_group = normal_messages.sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
    image_data = generate_image(sampled_group)
    npy_name = f'{output_dir}/image_{image_counter}.npy'
    np.save(npy_name, image_data)
    classification_results.append({'file_name': npy_name, 'classification': 0, 'misbehavior_index': 0})
    image_counter += 1

for misbehavior_class in range(1, 20):
    for _ in range(600):
        attack_group = final_df[final_df['class'] == misbehavior_class].sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
        misbehavior_index = np.random.randint(1, FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT + 1)
        image_data = generate_image(attack_group, misbehavior_index)
        npy_name = f'{output_dir}/image_{image_counter}.npy'
        np.save(npy_name, image_data)
        classification_results.append({'file_name': npy_name, 'classification': misbehavior_class, 'misbehavior_index': misbehavior_index})
        image_counter += 1

classification_df = pd.DataFrame(classification_results)
classification_df.to_csv('multiple_class_2x5_ea.csv', index=False)

print("Images generated and saved to 'multiple_class_2x5_ea.csv'.")

Images generated and saved to 'multiple_class_2x5_ea.csv'.


In [16]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (5, 2, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_2x5_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_2x5_ea.csv')

fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X = []
    Y_classification = []
    Y_index = []
    
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path)
            image_array = image_array / 255.0

            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)

    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

X_train, X_test, Y_train_classification, Y_test_classification, Y_train_index, Y_test_index = train_test_split(
    X, Y_classification, Y_index, test_size=0.2, random_state=42)
input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))

x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

classification_output = Dense(20, activation='softmax', name='classification_output')(x)

index_output = Dense(26, activation='softmax', name='index_output')(x)

model = Model(inputs=input_layer, outputs=[classification_output, index_output])

model.compile(optimizer='adam', 
              loss={'classification_output': 'sparse_categorical_crossentropy', 
                    'index_output': 'sparse_categorical_crossentropy'},
              metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, 
          {'classification_output': Y_train_classification, 'index_output': Y_train_index}, 
          epochs=FAC_EPOCHS, 
          batch_size=FAC_BATCH_SIZE, 
          validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
          callbacks=[early_stopping])

predictions = model.predict(X_test)
Y_pred_classification = np.argmax(predictions[0], axis=1)
Y_pred_index = np.argmax(predictions[1], axis=1)

accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)

accuracy_index = accuracy_score(Y_test_index, Y_pred_index)
f1_classification_manual = 2 * (precision_classification * recall_classification) / (precision_classification + recall_classification)

print("Classification Accuracy:", accuracy_classification)
print("Classification Precision:", precision_classification)
print("Classification Recall:", recall_classification)
print("Classification F1-score (sklearn):", f1_classification)
print("Classification F1-score (manual calculation):", f1_classification_manual)

print("Misbehavior Index Accuracy:", accuracy_index)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

save_dir = "models"
os.makedirs(save_dir, exist_ok=True)

model_save_path = os.path.join(save_dir, "veremi_images_m_2x5_ea_i.keras")
model.save(model_save_path)

print(f"\nModel saved at: {model_save_path}")

Epoch 1/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - classification_output_accuracy: 0.4480 - classification_output_loss: 2.0360 - index_output_accuracy: 0.6964 - index_output_loss: 1.1109 - loss: 3.1468 - val_classification_output_accuracy: 0.5098 - val_classification_output_loss: 1.8771 - val_index_output_accuracy: 0.8147 - val_index_output_loss: 0.4997 - val_loss: 2.3840
Epoch 2/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - classification_output_accuracy: 0.6255 - classification_output_loss: 1.2156 - index_output_accuracy: 0.9324 - index_output_loss: 0.2035 - loss: 1.4192 - val_classification_output_accuracy: 0.6296 - val_classification_output_loss: 1.2154 - val_index_output_accuracy: 0.9219 - val_index_output_loss: 0.2574 - val_loss: 1.4735
Epoch 3/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - classification_output_accuracy: 0.6700 - classification_output_loss: 1.0302 - index_output_accuracy: 0.9438 - index_output_loss: 0.1536 - loss: 1.1838 - val_classification_output_accurac

In [5]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (5, 2, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_2x5_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_2x5_ea.csv')
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X, Y_classification, Y_index = [], [], []
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path) / 255.0
            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)
    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accs, precs, recs, f1s, idx_accs = [], [], [], [], []
start_time = time.time()
fold = 1

for train_index, test_index in skf.split(X, Y_classification):
    X_train, X_test = X[train_index], X[test_index]
    Y_train_classification, Y_test_classification = Y_classification[train_index], Y_classification[test_index]
    Y_train_index, Y_test_index = Y_index[train_index], Y_index[test_index]

    input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))
    x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)

    classification_output = Dense(20, activation='softmax', name='classification_output')(x)
    index_output = Dense(26, activation='softmax', name='index_output')(x)

    model = Model(inputs=input_layer, outputs=[classification_output, index_output])
    model.compile(optimizer='adam',
                  loss={'classification_output': 'sparse_categorical_crossentropy',
                        'index_output': 'sparse_categorical_crossentropy'},
                  metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(X_train,
              {'classification_output': Y_train_classification, 'index_output': Y_train_index},
              epochs=FAC_EPOCHS,
              batch_size=FAC_BATCH_SIZE,
              validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
              callbacks=[early_stopping],
              verbose=0)

    predictions = model.predict(X_test, verbose=0)
    Y_pred_classification = np.argmax(predictions[0], axis=1)
    Y_pred_index = np.argmax(predictions[1], axis=1)

    accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
    precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    accuracy_index = accuracy_score(Y_test_index, Y_pred_index)

    print(f"Fold {fold} - Acc: {accuracy_classification:.4f}, Prec: {precision_classification:.4f}, Rec: {recall_classification:.4f}, F1: {f1_classification:.4f}, Index Acc: {accuracy_index:.4f}")

    accs.append(accuracy_classification)
    precs.append(precision_classification)
    recs.append(recall_classification)
    f1s.append(f1_classification)
    idx_accs.append(accuracy_index)
    fold += 1

print("\nCross Validation Results:")
print(f"Classification Accuracy: Mean={np.mean(accs):.4f}, Std={np.std(accs):.4f}")
print(f"Precision: Mean={np.mean(precs):.4f}, Std={np.std(precs):.4f}")
print(f"Recall: Mean={np.mean(recs):.4f}, Std={np.std(recs):.4f}")
print(f"F1-score: Mean={np.mean(f1s):.4f}, Std={np.std(f1s):.4f}")
print(f"Misbehavior Index Accuracy: Mean={np.mean(idx_accs):.4f}, Std={np.std(idx_accs):.4f}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Fold 1 - Acc: 0.6879, Prec: 0.5158, Rec: 0.5008, F1: 0.4950, Index Acc: 0.9567
Fold 2 - Acc: 0.6559, Prec: 0.4924, Rec: 0.4499, F1: 0.4383, Index Acc: 0.9557
Fold 3 - Acc: 0.6418, Prec: 0.4322, Rec: 0.4297, F1: 0.3994, Index Acc: 0.9369
Fold 4 - Acc: 0.6791, Prec: 0.5044, Rec: 0.4859, F1: 0.4712, Index Acc: 0.9621
Fold 5 - Acc: 0.6881, Prec: 0.5083, Rec: 0.5016, F1: 0.4934, Index Acc: 0.9588

Cross Validation Results:
Classification Accuracy: Mean=0.6706, Std=0.0186
Precision: Mean=0.4906, Std=0.0302
Recall: Mean=0.4736, Std=0.0289
F1-score: Mean=0.4595, Std=0.0364
Misbehavior Index Accuracy: Mean=0.9540, Std=0.0089

Training time: 306.625935 seconds


In [240]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_2x5_ea'
FAC_LABELS_CSV = 'multiple_class_2x5_ea.csv'
FAC_IMAGE_SIZE = (5, 2, 12)

def preprocess_image(file_path):
    """
    Load and preprocess an image for prediction.
    Expects .npy files containing image arrays.
    """
    image_array = np.load(file_path) / 255.0
    return np.expand_dims(image_array, axis=0)

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

random_index = np.random.choice(valid_labels.index)
selected_label = valid_labels.loc[random_index]
selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

start_time = time.time()
image_array = preprocess_image(selected_image_path)

predictions = model.predict(image_array)
predicted_classification = np.argmax(predictions[0])
predicted_index = np.argmax(predictions[1])
end_time = time.time()

print(f"Selected Image Path: {selected_image_path}")
print(f"True Classification: {selected_label['classification']}, True Misbehavior Index: {selected_label['misbehavior_index']}")
print(f"Predicted Classification: {predicted_classification}, Predicted Misbehavior Index: {predicted_index}")
print(f"\nElapsed time: {end_time - start_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Selected Image Path: veremi_multilevel_images_2x5_ea\image_18749.npy
True Classification: 18, True Misbehavior Index: 2
Predicted Classification: 18, Predicted Misbehavior Index: 2

Elapsed time: 0.068709 seconds


In [242]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_2x5_ea'
FAC_LABELS_CSV = 'multiple_class_2x5_ea.csv'
FAC_IMAGE_SIZE = (5, 2)
NUM_SAMPLES = 500
TIME_THRESHOLD = 0.1

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

def preprocess_image(file_path, target_size=(5, 2)):
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

total_classification_correct = 0
total_index_correct = 0
total_prediction_time = 0.0
predictions_above_threshold = 0
total_samples_processed = 0

true_classifications = []
predicted_classifications = []
true_indices = []
predicted_indices = []

for _ in range(NUM_SAMPLES):
    random_index = np.random.choice(valid_labels.index)
    selected_label = valid_labels.loc[random_index]
    selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

    if not os.path.exists(selected_image_path):
        continue

    try:
        start_time = time.time()
        image_array = preprocess_image(selected_image_path)
        image_array = np.expand_dims(image_array, axis=0)

        predictions = model.predict(image_array)
        predicted_classification = np.argmax(predictions[0])
        predicted_index = np.argmax(predictions[1])

        true_classifications.append(int(selected_label['classification']))
        predicted_classifications.append(predicted_classification)
        true_indices.append(int(selected_label['misbehavior_index']))
        predicted_indices.append(predicted_index)

        if predicted_classification == int(selected_label['classification']):
            total_classification_correct += 1
        if predicted_index == int(selected_label['misbehavior_index']):
            total_index_correct += 1

        prediction_time = time.time() - start_time
        total_prediction_time += prediction_time

        if prediction_time >= TIME_THRESHOLD:
            predictions_above_threshold += 1

        total_samples_processed += 1

    except Exception as e:
        print(f"Error during prediction: {e}")

classification_cm = confusion_matrix(true_classifications, predicted_classifications)
index_cm = confusion_matrix(true_indices, predicted_indices)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    total = np.sum(cm)

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    accuracy = np.sum(tp) / total if total > 0 else 0.0

    return accuracy, precision, recall, f1

classification_accuracy, classification_precision, classification_recall, classification_f1 = calculate_metrics(classification_cm)
index_accuracy, index_precision, index_recall, index_f1 = calculate_metrics(index_cm)

print("\nClassification Metrics:")
print(f"Accuracy: {classification_accuracy:.2f}")
print(f"Average Precision: {np.mean(classification_precision):.2f}")
print(f"Average Recall: {np.mean(classification_recall):.2f}")
print(f"Average F1-Score: {np.mean(classification_f1):.2f}")

print("\nIndex Prediction Metrics:")
print(f"Accuracy: {index_accuracy:.2f}")
print(f"Average Precision: {np.mean(index_precision):.2f}")
print(f"Average Recall: {np.mean(index_recall):.2f}")
print(f"Average F1-Score: {np.mean(index_f1):.2f}")

print(f"\nAverage Prediction Time: {total_prediction_time / total_samples_processed:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━

In [243]:
import numpy as np
import pandas as pd
import os

# Constants
FAC_IMAGE_WIDTH = 3
FAC_IMAGE_HEIGHT = 5
FAC_NUM_CHANNELS = 12
FAC_SPECIAL_VALUE = 255
NUM_CLASSES = 20

def normalize_and_scale(values, min_values, max_values):
    scaled = 255 * (values - min_values) / (max_values - min_values)
    scaled[np.isnan(values) | ~np.isfinite(values)] = FAC_SPECIAL_VALUE
    scaled[values == max_values] = 0
    return np.clip(scaled, 0, 255).astype(np.uint8)

def generate_image(group, misbehavior_index=None):
    min_values = np.array([group[col].min() for col in final_numeric_columns])
    max_values = np.array([group[col].max() for col in final_numeric_columns])

    data = group[final_numeric_columns].to_numpy()
    normalized_data = normalize_and_scale(data, min_values, max_values)

    reshaped_data = normalized_data.reshape(FAC_IMAGE_HEIGHT, FAC_IMAGE_WIDTH, FAC_NUM_CHANNELS)

    if misbehavior_index:
        row = (misbehavior_index - 1) // FAC_IMAGE_WIDTH
        col = (misbehavior_index - 1) % FAC_IMAGE_WIDTH
        reshaped_data[row, col, :3] = [255, 0, 0]
        
    return reshaped_data

final_numeric_columns = [
    'sendTime', 'senderPseudo', 'posx', 'posy', 'posx_n', 'posy_n',
    'spdx_n', 'spdy_n', 'aclx_n', 'acly_n', 'hedx_n', 'hedy_n'
]

output_dir = 'veremi_multilevel_images_3x5_ea'
os.makedirs(output_dir, exist_ok=True)

normal_messages = final_df[final_df['class'] == 0]

classification_results = []
image_counter = 1

for _ in range(3000):
    sampled_group = normal_messages.sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
    image_data = generate_image(sampled_group)
    npy_name = f'{output_dir}/image_{image_counter}.npy'
    np.save(npy_name, image_data)
    classification_results.append({'file_name': npy_name, 'classification': 0, 'misbehavior_index': 0})
    image_counter += 1

for misbehavior_class in range(1, 20):
    for _ in range(600):
        attack_group = final_df[final_df['class'] == misbehavior_class].sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
        misbehavior_index = np.random.randint(1, FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT + 1)
        image_data = generate_image(attack_group, misbehavior_index)
        npy_name = f'{output_dir}/image_{image_counter}.npy'
        np.save(npy_name, image_data)
        classification_results.append({'file_name': npy_name, 'classification': misbehavior_class, 'misbehavior_index': misbehavior_index})
        image_counter += 1

classification_df = pd.DataFrame(classification_results)
classification_df.to_csv('multiple_class_3x5_ea.csv', index=False)

print("Images generated and saved to 'multiple_class_3x5_ea.csv'.")

Images generated and saved to 'multiple_class_3x5_ea.csv'.


In [17]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (5, 3, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_3x5_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_3x5_ea.csv')

fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X = []
    Y_classification = []
    Y_index = []
    
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path)
            image_array = image_array / 255.0

            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)

    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

X_train, X_test, Y_train_classification, Y_test_classification, Y_train_index, Y_test_index = train_test_split(
    X, Y_classification, Y_index, test_size=0.2, random_state=42)
input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))

x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

classification_output = Dense(20, activation='softmax', name='classification_output')(x)

index_output = Dense(26, activation='softmax', name='index_output')(x)

model = Model(inputs=input_layer, outputs=[classification_output, index_output])

model.compile(optimizer='adam', 
              loss={'classification_output': 'sparse_categorical_crossentropy', 
                    'index_output': 'sparse_categorical_crossentropy'},
              metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, 
          {'classification_output': Y_train_classification, 'index_output': Y_train_index}, 
          epochs=FAC_EPOCHS, 
          batch_size=FAC_BATCH_SIZE, 
          validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
          callbacks=[early_stopping])

predictions = model.predict(X_test)
Y_pred_classification = np.argmax(predictions[0], axis=1)
Y_pred_index = np.argmax(predictions[1], axis=1)

accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)

accuracy_index = accuracy_score(Y_test_index, Y_pred_index)
f1_classification_manual = 2 * (precision_classification * recall_classification) / (precision_classification + recall_classification)

print("Classification Accuracy:", accuracy_classification)
print("Classification Precision:", precision_classification)
print("Classification Recall:", recall_classification)
print("Classification F1-score (sklearn):", f1_classification)
print("Classification F1-score (manual calculation):", f1_classification_manual)

print("Misbehavior Index Accuracy:", accuracy_index)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

save_dir = "models"
os.makedirs(save_dir, exist_ok=True)

model_save_path = os.path.join(save_dir, "veremi_images_m_3x5_ea_i.keras")
model.save(model_save_path)

print(f"\nModel saved at: {model_save_path}")

Epoch 1/10
360/360 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - classification_output_accuracy: 0.3755 - classification_output_loss: 2.1325 - index_output_accuracy: 0.5633 - index_output_loss: 1.6117 - loss: 3.7443 - val_classification_output_accuracy: 0.2326 - val_classification_output_loss: 3.3390 - val_index_output_accuracy: 0.2361 - val_index_output_loss: 3.2512 - val_loss: 6.5902
Epoch 2/10
360/360 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - classification_output_accuracy: 0.6060 - classification_output_loss: 1.2320 - index_output_accuracy: 0.8903 - index_output_loss: 0.3123 - loss: 1.5443 - val_classification_output_accuracy: 0.5976 - val_classification_output_loss: 1.2853 - val_index_output_accuracy: 0.8861 - val_index_output_loss: 0.3437 - val_loss: 1.6290
Epoch 3/10
360/360 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - classification_output_accuracy: 0.6881 - classification_output_loss: 0.9575 - index_output_accuracy: 0.9323 - index_output_loss: 0.1888 - loss: 1.1463 - val_classification_output_accuracy

In [6]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (5, 3, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_3x5_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_3x5_ea.csv')
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X, Y_classification, Y_index = [], [], []
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path) / 255.0
            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)
    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accs, precs, recs, f1s, idx_accs = [], [], [], [], []
start_time = time.time()
fold = 1

for train_index, test_index in skf.split(X, Y_classification):
    X_train, X_test = X[train_index], X[test_index]
    Y_train_classification, Y_test_classification = Y_classification[train_index], Y_classification[test_index]
    Y_train_index, Y_test_index = Y_index[train_index], Y_index[test_index]

    input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))
    x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)

    classification_output = Dense(20, activation='softmax', name='classification_output')(x)
    index_output = Dense(26, activation='softmax', name='index_output')(x)

    model = Model(inputs=input_layer, outputs=[classification_output, index_output])
    model.compile(optimizer='adam',
                  loss={'classification_output': 'sparse_categorical_crossentropy',
                        'index_output': 'sparse_categorical_crossentropy'},
                  metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(X_train,
              {'classification_output': Y_train_classification, 'index_output': Y_train_index},
              epochs=FAC_EPOCHS,
              batch_size=FAC_BATCH_SIZE,
              validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
              callbacks=[early_stopping],
              verbose=0)

    predictions = model.predict(X_test, verbose=0)
    Y_pred_classification = np.argmax(predictions[0], axis=1)
    Y_pred_index = np.argmax(predictions[1], axis=1)

    accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
    precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    accuracy_index = accuracy_score(Y_test_index, Y_pred_index)

    print(f"Fold {fold} - Acc: {accuracy_classification:.4f}, Prec: {precision_classification:.4f}, Rec: {recall_classification:.4f}, F1: {f1_classification:.4f}, Index Acc: {accuracy_index:.4f}")

    accs.append(accuracy_classification)
    precs.append(precision_classification)
    recs.append(recall_classification)
    f1s.append(f1_classification)
    idx_accs.append(accuracy_index)
    fold += 1

print("\nCross Validation Results:")
print(f"Classification Accuracy: Mean={np.mean(accs):.4f}, Std={np.std(accs):.4f}")
print(f"Precision: Mean={np.mean(precs):.4f}, Std={np.std(precs):.4f}")
print(f"Recall: Mean={np.mean(recs):.4f}, Std={np.std(recs):.4f}")
print(f"F1-score: Mean={np.mean(f1s):.4f}, Std={np.std(f1s):.4f}")
print(f"Misbehavior Index Accuracy: Mean={np.mean(idx_accs):.4f}, Std={np.std(idx_accs):.4f}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Fold 1 - Acc: 0.6580, Prec: 0.6130, Rec: 0.6033, F1: 0.5885, Index Acc: 0.9177
Fold 2 - Acc: 0.6472, Prec: 0.6258, Rec: 0.5837, F1: 0.5725, Index Acc: 0.9344
Fold 3 - Acc: 0.6858, Prec: 0.6435, Rec: 0.6259, F1: 0.6184, Index Acc: 0.9306
Fold 4 - Acc: 0.6142, Prec: 0.5901, Rec: 0.5424, F1: 0.5154, Index Acc: 0.9208
Fold 5 - Acc: 0.6740, Prec: 0.6433, Rec: 0.6128, F1: 0.6176, Index Acc: 0.9451

Cross Validation Results:
Classification Accuracy: Mean=0.6558, Std=0.0246
Precision: Mean=0.6231, Std=0.0201
Recall: Mean=0.5936, Std=0.0291
F1-score: Mean=0.5825, Std=0.0378
Misbehavior Index Accuracy: Mean=0.9297, Std=0.0098

Training time: 250.156373 seconds


In [8]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_3x5_ea'
FAC_LABELS_CSV = 'multiple_class_3x5_ea.csv'
FAC_IMAGE_SIZE = (5, 3, 12)

def preprocess_image(file_path):
    """
    Load and preprocess an image for prediction.
    Expects .npy files containing image arrays.
    """
    image_array = np.load(file_path) / 255.0
    return np.expand_dims(image_array, axis=0)

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

random_index = np.random.choice(valid_labels.index)
selected_label = valid_labels.loc[random_index]
selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

start_time = time.time()
image_array = preprocess_image(selected_image_path)

predictions = model.predict(image_array)
predicted_classification = np.argmax(predictions[0])
predicted_index = np.argmax(predictions[1])
end_time = time.time()

print(f"Selected Image Path: {selected_image_path}")
print(f"True Classification: {selected_label['classification']}, True Misbehavior Index: {selected_label['misbehavior_index']}")
print(f"Predicted Classification: {predicted_classification}, Predicted Misbehavior Index: {predicted_index}")
print(f"\nElapsed time: {end_time - start_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Selected Image Path: veremi_multilevel_images_3x5_ea\image_4353.npy
True Classification: 3, True Misbehavior Index: 15
Predicted Classification: 10, Predicted Misbehavior Index: 15

Elapsed time: 0.063639 seconds


In [9]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_3x5_ea'
FAC_LABELS_CSV = 'multiple_class_3x5_ea.csv'
FAC_IMAGE_SIZE = (5, 3)
NUM_SAMPLES = 500
TIME_THRESHOLD = 0.1

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

def preprocess_image(file_path, target_size=(5, 3)):
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

total_classification_correct = 0
total_index_correct = 0
total_prediction_time = 0.0
predictions_above_threshold = 0
total_samples_processed = 0

true_classifications = []
predicted_classifications = []
true_indices = []
predicted_indices = []

for _ in range(NUM_SAMPLES):
    random_index = np.random.choice(valid_labels.index)
    selected_label = valid_labels.loc[random_index]
    selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

    if not os.path.exists(selected_image_path):
        continue

    try:
        start_time = time.time()
        image_array = preprocess_image(selected_image_path)
        image_array = np.expand_dims(image_array, axis=0)

        predictions = model.predict(image_array)
        predicted_classification = np.argmax(predictions[0])
        predicted_index = np.argmax(predictions[1])

        true_classifications.append(int(selected_label['classification']))
        predicted_classifications.append(predicted_classification)
        true_indices.append(int(selected_label['misbehavior_index']))
        predicted_indices.append(predicted_index)

        if predicted_classification == int(selected_label['classification']):
            total_classification_correct += 1
        if predicted_index == int(selected_label['misbehavior_index']):
            total_index_correct += 1

        prediction_time = time.time() - start_time
        total_prediction_time += prediction_time

        if prediction_time >= TIME_THRESHOLD:
            predictions_above_threshold += 1

        total_samples_processed += 1

    except Exception as e:
        print(f"Error during prediction: {e}")

classification_cm = confusion_matrix(true_classifications, predicted_classifications)
index_cm = confusion_matrix(true_indices, predicted_indices)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    total = np.sum(cm)

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    accuracy = np.sum(tp) / total if total > 0 else 0.0

    return accuracy, precision, recall, f1

classification_accuracy, classification_precision, classification_recall, classification_f1 = calculate_metrics(classification_cm)
index_accuracy, index_precision, index_recall, index_f1 = calculate_metrics(index_cm)

print("\nClassification Metrics:")
print(f"Accuracy: {classification_accuracy:.2f}")
print(f"Average Precision: {np.mean(classification_precision):.2f}")
print(f"Average Recall: {np.mean(classification_recall):.2f}")
print(f"Average F1-Score: {np.mean(classification_f1):.2f}")

print("\nIndex Prediction Metrics:")
print(f"Accuracy: {index_accuracy:.2f}")
print(f"Average Precision: {np.mean(index_precision):.2f}")
print(f"Average Recall: {np.mean(index_recall):.2f}")
print(f"Average F1-Score: {np.mean(index_f1):.2f}")

print(f"\nAverage Prediction Time: {total_prediction_time / total_samples_processed:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━

In [15]:
import numpy as np
import pandas as pd
import os

# Constants
FAC_IMAGE_WIDTH = 4
FAC_IMAGE_HEIGHT = 5
FAC_NUM_CHANNELS = 12
FAC_SPECIAL_VALUE = 255
NUM_CLASSES = 20

def normalize_and_scale(values, min_values, max_values):
    scaled = 255 * (values - min_values) / (max_values - min_values)
    scaled[np.isnan(values) | ~np.isfinite(values)] = FAC_SPECIAL_VALUE
    scaled[values == max_values] = 0
    return np.clip(scaled, 0, 255).astype(np.uint8)

def generate_image(group, misbehavior_index=None):
    min_values = np.array([group[col].min() for col in final_numeric_columns])
    max_values = np.array([group[col].max() for col in final_numeric_columns])

    data = group[final_numeric_columns].to_numpy()
    normalized_data = normalize_and_scale(data, min_values, max_values)

    reshaped_data = normalized_data.reshape(FAC_IMAGE_HEIGHT, FAC_IMAGE_WIDTH, FAC_NUM_CHANNELS)

    if misbehavior_index:
        row = (misbehavior_index - 1) // FAC_IMAGE_WIDTH
        col = (misbehavior_index - 1) % FAC_IMAGE_WIDTH
        reshaped_data[row, col, :3] = [255, 0, 0]
        
    return reshaped_data

final_numeric_columns = [
    'sendTime', 'senderPseudo', 'posx', 'posy', 'posx_n', 'posy_n',
    'spdx_n', 'spdy_n', 'aclx_n', 'acly_n', 'hedx_n', 'hedy_n'
]

output_dir = 'veremi_multilevel_images_4x5_ea'
os.makedirs(output_dir, exist_ok=True)

normal_messages = final_df[final_df['class'] == 0]

classification_results = []
image_counter = 1

for _ in range(3000):
    sampled_group = normal_messages.sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
    image_data = generate_image(sampled_group)
    npy_name = f'{output_dir}/image_{image_counter}.npy'
    np.save(npy_name, image_data)
    classification_results.append({'file_name': npy_name, 'classification': 0, 'misbehavior_index': 0})
    image_counter += 1

for misbehavior_class in range(1, 20):
    for _ in range(500):
        attack_group = final_df[final_df['class'] == misbehavior_class].sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
        misbehavior_index = np.random.randint(1, FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT + 1)
        image_data = generate_image(attack_group, misbehavior_index)
        npy_name = f'{output_dir}/image_{image_counter}.npy'
        np.save(npy_name, image_data)
        classification_results.append({'file_name': npy_name, 'classification': misbehavior_class, 'misbehavior_index': misbehavior_index})
        image_counter += 1

classification_df = pd.DataFrame(classification_results)
classification_df.to_csv('multiple_class_4x5_ea.csv', index=False)

print("Images generated and saved to 'multiple_class_4x5_ea.csv'.")

Images generated and saved to 'multiple_class_4x5_ea.csv'.


In [18]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (5, 4, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_4x5_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_4x5_ea.csv')

fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X = []
    Y_classification = []
    Y_index = []
    
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path)
            image_array = image_array / 255.0

            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)

    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

X_train, X_test, Y_train_classification, Y_test_classification, Y_train_index, Y_test_index = train_test_split(
    X, Y_classification, Y_index, test_size=0.2, random_state=42)
input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))

x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

classification_output = Dense(20, activation='softmax', name='classification_output')(x)

index_output = Dense(26, activation='softmax', name='index_output')(x)

model = Model(inputs=input_layer, outputs=[classification_output, index_output])

model.compile(optimizer='adam', 
              loss={'classification_output': 'sparse_categorical_crossentropy', 
                    'index_output': 'sparse_categorical_crossentropy'},
              metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, 
          {'classification_output': Y_train_classification, 'index_output': Y_train_index}, 
          epochs=FAC_EPOCHS, 
          batch_size=FAC_BATCH_SIZE, 
          validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
          callbacks=[early_stopping])

predictions = model.predict(X_test)
Y_pred_classification = np.argmax(predictions[0], axis=1)
Y_pred_index = np.argmax(predictions[1], axis=1)

accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)

accuracy_index = accuracy_score(Y_test_index, Y_pred_index)
f1_classification_manual = 2 * (precision_classification * recall_classification) / (precision_classification + recall_classification)

print("Classification Accuracy:", accuracy_classification)
print("Classification Precision:", precision_classification)
print("Classification Recall:", recall_classification)
print("Classification F1-score (sklearn):", f1_classification)
print("Classification F1-score (manual calculation):", f1_classification_manual)

print("Misbehavior Index Accuracy:", accuracy_index)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

save_dir = "models"
os.makedirs(save_dir, exist_ok=True)

model_save_path = os.path.join(save_dir, "veremi_images_m_4x5_ea_i.keras")
model.save(model_save_path)

print(f"\nModel saved at: {model_save_path}")

Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - classification_output_accuracy: 0.4548 - classification_output_loss: 1.8609 - index_output_accuracy: 0.5012 - index_output_loss: 1.8457 - loss: 3.7066 - val_classification_output_accuracy: 0.3488 - val_classification_output_loss: 2.2955 - val_index_output_accuracy: 0.3664 - val_index_output_loss: 2.1252 - val_loss: 4.3911
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - classification_output_accuracy: 0.6847 - classification_output_loss: 0.9973 - index_output_accuracy: 0.8832 - index_output_loss: 0.3349 - loss: 1.3322 - val_classification_output_accuracy: 0.6280 - val_classification_output_loss: 1.2865 - val_index_output_accuracy: 0.8564 - val_index_output_loss: 0.4211 - val_loss: 1.7039
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - classification_output_accuracy: 0.7662 - classification_output_loss: 0.7101 - index_output_accuracy: 0.9456 - index_output_loss: 0.1674 - loss: 0.8775 - val_classification_output_accuracy

In [7]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (5, 4, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_4x5_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_4x5_ea.csv')
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X, Y_classification, Y_index = [], [], []
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path) / 255.0
            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)
    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accs, precs, recs, f1s, idx_accs = [], [], [], [], []
start_time = time.time()
fold = 1

for train_index, test_index in skf.split(X, Y_classification):
    X_train, X_test = X[train_index], X[test_index]
    Y_train_classification, Y_test_classification = Y_classification[train_index], Y_classification[test_index]
    Y_train_index, Y_test_index = Y_index[train_index], Y_index[test_index]

    input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))
    x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)

    classification_output = Dense(20, activation='softmax', name='classification_output')(x)
    index_output = Dense(26, activation='softmax', name='index_output')(x)

    model = Model(inputs=input_layer, outputs=[classification_output, index_output])
    model.compile(optimizer='adam',
                  loss={'classification_output': 'sparse_categorical_crossentropy',
                        'index_output': 'sparse_categorical_crossentropy'},
                  metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(X_train,
              {'classification_output': Y_train_classification, 'index_output': Y_train_index},
              epochs=FAC_EPOCHS,
              batch_size=FAC_BATCH_SIZE,
              validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
              callbacks=[early_stopping],
              verbose=0)

    predictions = model.predict(X_test, verbose=0)
    Y_pred_classification = np.argmax(predictions[0], axis=1)
    Y_pred_index = np.argmax(predictions[1], axis=1)

    accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
    precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    accuracy_index = accuracy_score(Y_test_index, Y_pred_index)

    print(f"Fold {fold} - Acc: {accuracy_classification:.4f}, Prec: {precision_classification:.4f}, Rec: {recall_classification:.4f}, F1: {f1_classification:.4f}, Index Acc: {accuracy_index:.4f}")

    accs.append(accuracy_classification)
    precs.append(precision_classification)
    recs.append(recall_classification)
    f1s.append(f1_classification)
    idx_accs.append(accuracy_index)
    fold += 1

print("\nCross Validation Results:")
print(f"Classification Accuracy: Mean={np.mean(accs):.4f}, Std={np.std(accs):.4f}")
print(f"Precision: Mean={np.mean(precs):.4f}, Std={np.std(precs):.4f}")
print(f"Recall: Mean={np.mean(recs):.4f}, Std={np.std(recs):.4f}")
print(f"F1-score: Mean={np.mean(f1s):.4f}, Std={np.std(f1s):.4f}")
print(f"Misbehavior Index Accuracy: Mean={np.mean(idx_accs):.4f}, Std={np.std(idx_accs):.4f}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Fold 1 - Acc: 0.7152, Prec: 0.6912, Rec: 0.6519, F1: 0.6391, Index Acc: 0.9196
Fold 2 - Acc: 0.7580, Prec: 0.7152, Rec: 0.7013, F1: 0.6953, Index Acc: 0.9256
Fold 3 - Acc: 0.6648, Prec: 0.6744, Rec: 0.5956, F1: 0.5877, Index Acc: 0.9048
Fold 4 - Acc: 0.7348, Prec: 0.7107, Rec: 0.6852, F1: 0.6754, Index Acc: 0.9168
Fold 5 - Acc: 0.7040, Prec: 0.7141, Rec: 0.6500, F1: 0.6495, Index Acc: 0.8832

Cross Validation Results:
Classification Accuracy: Mean=0.7154, Std=0.0312
Precision: Mean=0.7011, Std=0.0160
Recall: Mean=0.6568, Std=0.0363
F1-score: Mean=0.6494, Std=0.0366
Misbehavior Index Accuracy: Mean=0.9100, Std=0.0150

Training time: 243.850720 seconds


In [9]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_4x5_ea'
FAC_LABELS_CSV = 'multiple_class_4x5_ea.csv'
FAC_IMAGE_SIZE = (5, 4, 12)

def preprocess_image(file_path):
    """
    Load and preprocess an image for prediction.
    Expects .npy files containing image arrays.
    """
    image_array = np.load(file_path) / 255.0
    return np.expand_dims(image_array, axis=0)

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

random_index = np.random.choice(valid_labels.index)
selected_label = valid_labels.loc[random_index]
selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

start_time = time.time()
image_array = preprocess_image(selected_image_path)

predictions = model.predict(image_array)
predicted_classification = np.argmax(predictions[0])
predicted_index = np.argmax(predictions[1])
end_time = time.time()

print(f"Selected Image Path: {selected_image_path}")
print(f"True Classification: {selected_label['classification']}, True Misbehavior Index: {selected_label['misbehavior_index']}")
print(f"Predicted Classification: {predicted_classification}, Predicted Misbehavior Index: {predicted_index}")
print(f"\nElapsed time: {end_time - start_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Selected Image Path: veremi_multilevel_images_4x5_ea\image_4314.npy
True Classification: 3, True Misbehavior Index: 8
Predicted Classification: 12, Predicted Misbehavior Index: 8

Elapsed time: 0.071186 seconds


In [10]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_4x5_ea'
FAC_LABELS_CSV = 'multiple_class_4x5_ea.csv'
FAC_IMAGE_SIZE = (5, 4)
NUM_SAMPLES = 500
TIME_THRESHOLD = 0.1

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

def preprocess_image(file_path, target_size=(5, 4)):
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

total_classification_correct = 0
total_index_correct = 0
total_prediction_time = 0.0
predictions_above_threshold = 0
total_samples_processed = 0

true_classifications = []
predicted_classifications = []
true_indices = []
predicted_indices = []

for _ in range(NUM_SAMPLES):
    random_index = np.random.choice(valid_labels.index)
    selected_label = valid_labels.loc[random_index]
    selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

    if not os.path.exists(selected_image_path):
        continue

    try:
        start_time = time.time()
        image_array = preprocess_image(selected_image_path)
        image_array = np.expand_dims(image_array, axis=0)

        predictions = model.predict(image_array)
        predicted_classification = np.argmax(predictions[0])
        predicted_index = np.argmax(predictions[1])

        true_classifications.append(int(selected_label['classification']))
        predicted_classifications.append(predicted_classification)
        true_indices.append(int(selected_label['misbehavior_index']))
        predicted_indices.append(predicted_index)

        if predicted_classification == int(selected_label['classification']):
            total_classification_correct += 1
        if predicted_index == int(selected_label['misbehavior_index']):
            total_index_correct += 1

        prediction_time = time.time() - start_time
        total_prediction_time += prediction_time

        if prediction_time >= TIME_THRESHOLD:
            predictions_above_threshold += 1

        total_samples_processed += 1

    except Exception as e:
        print(f"Error during prediction: {e}")

classification_cm = confusion_matrix(true_classifications, predicted_classifications)
index_cm = confusion_matrix(true_indices, predicted_indices)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    total = np.sum(cm)

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    accuracy = np.sum(tp) / total if total > 0 else 0.0

    return accuracy, precision, recall, f1

classification_accuracy, classification_precision, classification_recall, classification_f1 = calculate_metrics(classification_cm)
index_accuracy, index_precision, index_recall, index_f1 = calculate_metrics(index_cm)

print("\nClassification Metrics:")
print(f"Accuracy: {classification_accuracy:.2f}")
print(f"Average Precision: {np.mean(classification_precision):.2f}")
print(f"Average Recall: {np.mean(classification_recall):.2f}")
print(f"Average F1-Score: {np.mean(classification_f1):.2f}")

print("\nIndex Prediction Metrics:")
print(f"Accuracy: {index_accuracy:.2f}")
print(f"Average Precision: {np.mean(index_precision):.2f}")
print(f"Average Recall: {np.mean(index_recall):.2f}")
print(f"Average F1-Score: {np.mean(index_f1):.2f}")

print(f"\nAverage Prediction Time: {total_prediction_time / total_samples_processed:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━

In [7]:
import numpy as np
import pandas as pd
import os

# Constants
FAC_IMAGE_WIDTH = 4
FAC_IMAGE_HEIGHT = 3
FAC_NUM_CHANNELS = 12
FAC_SPECIAL_VALUE = 255
NUM_CLASSES = 20

def normalize_and_scale(values, min_values, max_values):
    scaled = 255 * (values - min_values) / (max_values - min_values)
    scaled[np.isnan(values) | ~np.isfinite(values)] = FAC_SPECIAL_VALUE
    scaled[values == max_values] = 0
    return np.clip(scaled, 0, 255).astype(np.uint8)

def generate_image(group, misbehavior_index=None):
    min_values = np.array([group[col].min() for col in final_numeric_columns])
    max_values = np.array([group[col].max() for col in final_numeric_columns])

    data = group[final_numeric_columns].to_numpy()
    normalized_data = normalize_and_scale(data, min_values, max_values)

    reshaped_data = normalized_data.reshape(FAC_IMAGE_HEIGHT, FAC_IMAGE_WIDTH, FAC_NUM_CHANNELS)

    if misbehavior_index:
        row = (misbehavior_index - 1) // FAC_IMAGE_WIDTH
        col = (misbehavior_index - 1) % FAC_IMAGE_WIDTH
        reshaped_data[row, col, :3] = [255, 0, 0]
        
    return reshaped_data

final_numeric_columns = [
    'sendTime', 'senderPseudo', 'posx', 'posy', 'posx_n', 'posy_n',
    'spdx_n', 'spdy_n', 'aclx_n', 'acly_n', 'hedx_n', 'hedy_n'
]

output_dir = 'veremi_multilevel_images_3x4_ea'
os.makedirs(output_dir, exist_ok=True)

normal_messages = final_df[final_df['class'] == 0]

classification_results = []
image_counter = 1

for _ in range(3000):
    sampled_group = normal_messages.sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
    image_data = generate_image(sampled_group)
    npy_name = f'{output_dir}/image_{image_counter}.npy'
    np.save(npy_name, image_data)
    classification_results.append({'file_name': npy_name, 'classification': 0, 'misbehavior_index': 0})
    image_counter += 1

for misbehavior_class in range(1, 20):
    for _ in range(500):
        attack_group = final_df[final_df['class'] == misbehavior_class].sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
        misbehavior_index = np.random.randint(1, FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT + 1)
        image_data = generate_image(attack_group, misbehavior_index)
        npy_name = f'{output_dir}/image_{image_counter}.npy'
        np.save(npy_name, image_data)
        classification_results.append({'file_name': npy_name, 'classification': misbehavior_class, 'misbehavior_index': misbehavior_index})
        image_counter += 1

classification_df = pd.DataFrame(classification_results)
classification_df.to_csv('multiple_class_3x4_ea.csv', index=False)

print("Images generated and saved to 'multiple_class_3x4_ea.csv'.")

Images generated and saved to 'multiple_class_3x4_ea.csv'.


In [19]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (3, 4, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_3x4_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_3x4_ea.csv')

fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X = []
    Y_classification = []
    Y_index = []
    
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path)
            image_array = image_array / 255.0

            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)

    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

X_train, X_test, Y_train_classification, Y_test_classification, Y_train_index, Y_test_index = train_test_split(
    X, Y_classification, Y_index, test_size=0.2, random_state=42)
input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))

x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

classification_output = Dense(20, activation='softmax', name='classification_output')(x)

index_output = Dense(26, activation='softmax', name='index_output')(x)

model = Model(inputs=input_layer, outputs=[classification_output, index_output])

model.compile(optimizer='adam', 
              loss={'classification_output': 'sparse_categorical_crossentropy', 
                    'index_output': 'sparse_categorical_crossentropy'},
              metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, 
          {'classification_output': Y_train_classification, 'index_output': Y_train_index}, 
          epochs=FAC_EPOCHS, 
          batch_size=FAC_BATCH_SIZE, 
          validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
          callbacks=[early_stopping])

predictions = model.predict(X_test)
Y_pred_classification = np.argmax(predictions[0], axis=1)
Y_pred_index = np.argmax(predictions[1], axis=1)

accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)

accuracy_index = accuracy_score(Y_test_index, Y_pred_index)
f1_classification_manual = 2 * (precision_classification * recall_classification) / (precision_classification + recall_classification)

print("Classification Accuracy:", accuracy_classification)
print("Classification Precision:", precision_classification)
print("Classification Recall:", recall_classification)
print("Classification F1-score (sklearn):", f1_classification)
print("Classification F1-score (manual calculation):", f1_classification_manual)

print("Misbehavior Index Accuracy:", accuracy_index)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

save_dir = "models"
os.makedirs(save_dir, exist_ok=True)

model_save_path = os.path.join(save_dir, "veremi_images_m_3x4_ea_i.keras")
model.save(model_save_path)

print(f"\nModel saved at: {model_save_path}")

Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - classification_output_accuracy: 0.3315 - classification_output_loss: 2.3448 - index_output_accuracy: 0.5821 - index_output_loss: 1.5308 - loss: 3.8756 - val_classification_output_accuracy: 0.2584 - val_classification_output_loss: 3.2741 - val_index_output_accuracy: 0.2488 - val_index_output_loss: 3.2212 - val_loss: 6.4444
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - classification_output_accuracy: 0.5420 - classification_output_loss: 1.4497 - index_output_accuracy: 0.8918 - index_output_loss: 0.2996 - loss: 1.7493 - val_classification_output_accuracy: 0.5356 - val_classification_output_loss: 1.4850 - val_index_output_accuracy: 0.8544 - val_index_output_loss: 0.4290 - val_loss: 1.9182
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - classification_output_accuracy: 0.6302 - classification_output_loss: 1.1568 - index_output_accuracy: 0.9261 - index_output_loss: 0.2101 - loss: 1.3669 - val_classification_output_accuracy

In [11]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (3, 4, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_3x4_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_3x4_ea.csv')
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X, Y_classification, Y_index = [], [], []
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path) / 255.0
            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)
    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accs, precs, recs, f1s, idx_accs = [], [], [], [], []
start_time = time.time()
fold = 1

for train_index, test_index in skf.split(X, Y_classification):
    X_train, X_test = X[train_index], X[test_index]
    Y_train_classification, Y_test_classification = Y_classification[train_index], Y_classification[test_index]
    Y_train_index, Y_test_index = Y_index[train_index], Y_index[test_index]

    input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))
    x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)

    classification_output = Dense(20, activation='softmax', name='classification_output')(x)
    index_output = Dense(26, activation='softmax', name='index_output')(x)

    model = Model(inputs=input_layer, outputs=[classification_output, index_output])
    model.compile(optimizer='adam',
                  loss={'classification_output': 'sparse_categorical_crossentropy',
                        'index_output': 'sparse_categorical_crossentropy'},
                  metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(X_train,
              {'classification_output': Y_train_classification, 'index_output': Y_train_index},
              epochs=FAC_EPOCHS,
              batch_size=FAC_BATCH_SIZE,
              validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
              callbacks=[early_stopping],
              verbose=0)

    predictions = model.predict(X_test, verbose=0)
    Y_pred_classification = np.argmax(predictions[0], axis=1)
    Y_pred_index = np.argmax(predictions[1], axis=1)

    accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
    precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    accuracy_index = accuracy_score(Y_test_index, Y_pred_index)

    print(f"Fold {fold} - Acc: {accuracy_classification:.4f}, Prec: {precision_classification:.4f}, Rec: {recall_classification:.4f}, F1: {f1_classification:.4f}, Index Acc: {accuracy_index:.4f}")

    accs.append(accuracy_classification)
    precs.append(precision_classification)
    recs.append(recall_classification)
    f1s.append(f1_classification)
    idx_accs.append(accuracy_index)
    fold += 1

print("\nCross Validation Results:")
print(f"Classification Accuracy: Mean={np.mean(accs):.4f}, Std={np.std(accs):.4f}")
print(f"Precision: Mean={np.mean(precs):.4f}, Std={np.std(precs):.4f}")
print(f"Recall: Mean={np.mean(recs):.4f}, Std={np.std(recs):.4f}")
print(f"F1-score: Mean={np.mean(f1s):.4f}, Std={np.std(f1s):.4f}")
print(f"Misbehavior Index Accuracy: Mean={np.mean(idx_accs):.4f}, Std={np.std(idx_accs):.4f}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Fold 1 - Acc: 0.5808, Prec: 0.5021, Rec: 0.4797, F1: 0.4732, Index Acc: 0.9132
Fold 2 - Acc: 0.5800, Prec: 0.5049, Rec: 0.4804, F1: 0.4760, Index Acc: 0.9112
Fold 3 - Acc: 0.5572, Prec: 0.5098, Rec: 0.4657, F1: 0.4515, Index Acc: 0.9204
Fold 4 - Acc: 0.5792, Prec: 0.5084, Rec: 0.4782, F1: 0.4650, Index Acc: 0.9080
Fold 5 - Acc: 0.5856, Prec: 0.5070, Rec: 0.5024, F1: 0.4955, Index Acc: 0.8884

Cross Validation Results:
Classification Accuracy: Mean=0.5766, Std=0.0099
Precision: Mean=0.5064, Std=0.0027
Recall: Mean=0.4813, Std=0.0119
F1-score: Mean=0.4722, Std=0.0144
Misbehavior Index Accuracy: Mean=0.9082, Std=0.0107

Training time: 176.392501 seconds


In [37]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_3x4_ea'
FAC_LABELS_CSV = 'multiple_class_3x4_ea.csv'
FAC_IMAGE_SIZE = (4, 3, 12)

def preprocess_image(file_path):
    """
    Load and preprocess an image for prediction.
    Expects .npy files containing image arrays.
    """
    image_array = np.load(file_path) / 255.0
    return np.expand_dims(image_array, axis=0)

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

random_index = np.random.choice(valid_labels.index)
selected_label = valid_labels.loc[random_index]
selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

start_time = time.time()
image_array = preprocess_image(selected_image_path)

predictions = model.predict(image_array)
predicted_classification = np.argmax(predictions[0])
predicted_index = np.argmax(predictions[1])
end_time = time.time()

print(f"Selected Image Path: {selected_image_path}")
print(f"True Classification: {selected_label['classification']}, True Misbehavior Index: {selected_label['misbehavior_index']}")
print(f"Predicted Classification: {predicted_classification}, Predicted Misbehavior Index: {predicted_index}")
print(f"\nElapsed time: {end_time - start_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Selected Image Path: veremi_multilevel_images_3x4_ea\image_6927.npy
True Classification: 8, True Misbehavior Index: 7
Predicted Classification: 8, Predicted Misbehavior Index: 7

Elapsed time: 0.062025 seconds


In [40]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_3x4_ea'
FAC_LABELS_CSV = 'multiple_class_3x4_ea.csv'
FAC_IMAGE_SIZE = (3, 4)
NUM_SAMPLES = 500
TIME_THRESHOLD = 0.1

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

def preprocess_image(file_path, target_size=(3, 4)):
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

total_classification_correct = 0
total_index_correct = 0
total_prediction_time = 0.0
predictions_above_threshold = 0
total_samples_processed = 0

true_classifications = []
predicted_classifications = []
true_indices = []
predicted_indices = []

for _ in range(NUM_SAMPLES):
    random_index = np.random.choice(valid_labels.index)
    selected_label = valid_labels.loc[random_index]
    selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

    if not os.path.exists(selected_image_path):
        continue

    try:
        start_time = time.time()
        image_array = preprocess_image(selected_image_path)
        image_array = np.expand_dims(image_array, axis=0)

        predictions = model.predict(image_array)
        predicted_classification = np.argmax(predictions[0])
        predicted_index = np.argmax(predictions[1])

        true_classifications.append(int(selected_label['classification']))
        predicted_classifications.append(predicted_classification)
        true_indices.append(int(selected_label['misbehavior_index']))
        predicted_indices.append(predicted_index)

        if predicted_classification == int(selected_label['classification']):
            total_classification_correct += 1
        if predicted_index == int(selected_label['misbehavior_index']):
            total_index_correct += 1

        prediction_time = time.time() - start_time
        total_prediction_time += prediction_time

        if prediction_time >= TIME_THRESHOLD:
            predictions_above_threshold += 1

        total_samples_processed += 1

    except Exception as e:
        print(f"Error during prediction: {e}")

classification_cm = confusion_matrix(true_classifications, predicted_classifications)
index_cm = confusion_matrix(true_indices, predicted_indices)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    total = np.sum(cm)

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    accuracy = np.sum(tp) / total if total > 0 else 0.0

    return accuracy, precision, recall, f1

classification_accuracy, classification_precision, classification_recall, classification_f1 = calculate_metrics(classification_cm)
index_accuracy, index_precision, index_recall, index_f1 = calculate_metrics(index_cm)

print("\nClassification Metrics:")
print(f"Accuracy: {classification_accuracy:.2f}")
print(f"Average Precision: {np.mean(classification_precision):.2f}")
print(f"Average Recall: {np.mean(classification_recall):.2f}")
print(f"Average F1-Score: {np.mean(classification_f1):.2f}")

print("\nIndex Prediction Metrics:")
print(f"Accuracy: {index_accuracy:.2f}")
print(f"Average Precision: {np.mean(index_precision):.2f}")
print(f"Average Recall: {np.mean(index_recall):.2f}")
print(f"Average F1-Score: {np.mean(index_f1):.2f}")

print(f"\nAverage Prediction Time: {total_prediction_time / total_samples_processed:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━

In [41]:
import numpy as np
import pandas as pd
import os

# Constants
FAC_IMAGE_WIDTH = 6
FAC_IMAGE_HEIGHT = 6
FAC_NUM_CHANNELS = 12
FAC_SPECIAL_VALUE = 255
NUM_CLASSES = 20

def normalize_and_scale(values, min_values, max_values):
    scaled = 255 * (values - min_values) / (max_values - min_values)
    scaled[np.isnan(values) | ~np.isfinite(values)] = FAC_SPECIAL_VALUE
    scaled[values == max_values] = 0
    return np.clip(scaled, 0, 255).astype(np.uint8)

def generate_image(group, misbehavior_index=None):
    min_values = np.array([group[col].min() for col in final_numeric_columns])
    max_values = np.array([group[col].max() for col in final_numeric_columns])

    data = group[final_numeric_columns].to_numpy()
    normalized_data = normalize_and_scale(data, min_values, max_values)

    reshaped_data = normalized_data.reshape(FAC_IMAGE_HEIGHT, FAC_IMAGE_WIDTH, FAC_NUM_CHANNELS)

    if misbehavior_index:
        row = (misbehavior_index - 1) // FAC_IMAGE_WIDTH
        col = (misbehavior_index - 1) % FAC_IMAGE_WIDTH
        reshaped_data[row, col, :3] = [255, 0, 0]
        
    return reshaped_data

final_numeric_columns = [
    'sendTime', 'senderPseudo', 'posx', 'posy', 'posx_n', 'posy_n',
    'spdx_n', 'spdy_n', 'aclx_n', 'acly_n', 'hedx_n', 'hedy_n'
]

output_dir = 'veremi_multilevel_images_6x6_ea'
os.makedirs(output_dir, exist_ok=True)

normal_messages = final_df[final_df['class'] == 0]

classification_results = []
image_counter = 1

for _ in range(3000):
    sampled_group = normal_messages.sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
    image_data = generate_image(sampled_group)
    npy_name = f'{output_dir}/image_{image_counter}.npy'
    np.save(npy_name, image_data)
    classification_results.append({'file_name': npy_name, 'classification': 0, 'misbehavior_index': 0})
    image_counter += 1

for misbehavior_class in range(1, 20):
    for _ in range(300):
        attack_group = final_df[final_df['class'] == misbehavior_class].sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
        misbehavior_index = np.random.randint(1, FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT + 1)
        image_data = generate_image(attack_group, misbehavior_index)
        npy_name = f'{output_dir}/image_{image_counter}.npy'
        np.save(npy_name, image_data)
        classification_results.append({'file_name': npy_name, 'classification': misbehavior_class, 'misbehavior_index': misbehavior_index})
        image_counter += 1

classification_df = pd.DataFrame(classification_results)
classification_df.to_csv('multiple_class_6x6_ea.csv', index=False)

print("Images generated and saved to 'multiple_class_6x6_ea.csv'.")

Images generated and saved to 'multiple_class_6x6_ea.csv'.


In [20]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (6, 6, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_6x6_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_6x6_ea.csv')

fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X = []
    Y_classification = []
    Y_index = []
    
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path)
            image_array = image_array / 255.0

            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)

    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

X_train, X_test, Y_train_classification, Y_test_classification, Y_train_index, Y_test_index = train_test_split(
    X, Y_classification, Y_index, test_size=0.2, random_state=42)
input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))

x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

classification_output = Dense(20, activation='softmax', name='classification_output')(x)

index_output = Dense(37, activation='softmax', name='index_output')(x)

model = Model(inputs=input_layer, outputs=[classification_output, index_output])

model.compile(optimizer='adam', 
              loss={'classification_output': 'sparse_categorical_crossentropy', 
                    'index_output': 'sparse_categorical_crossentropy'},
              metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, 
          {'classification_output': Y_train_classification, 'index_output': Y_train_index}, 
          epochs=FAC_EPOCHS, 
          batch_size=FAC_BATCH_SIZE, 
          validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
          callbacks=[early_stopping])

predictions = model.predict(X_test)
Y_pred_classification = np.argmax(predictions[0], axis=1)
Y_pred_index = np.argmax(predictions[1], axis=1)

accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)

accuracy_index = accuracy_score(Y_test_index, Y_pred_index)
f1_classification_manual = 2 * (precision_classification * recall_classification) / (precision_classification + recall_classification)

print("Classification Accuracy:", accuracy_classification)
print("Classification Precision:", precision_classification)
print("Classification Recall:", recall_classification)
print("Classification F1-score (sklearn):", f1_classification)
print("Classification F1-score (manual calculation):", f1_classification_manual)

print("Misbehavior Index Accuracy:", accuracy_index)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

save_dir = "models"
os.makedirs(save_dir, exist_ok=True)

model_save_path = os.path.join(save_dir, "veremi_images_m_6x6_ea_i.keras")
model.save(model_save_path)

print(f"\nModel saved at: {model_save_path}")

Epoch 1/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step - classification_output_accuracy: 0.5837 - classification_output_loss: 1.4115 - index_output_accuracy: 0.3797 - index_output_loss: 2.5882 - loss: 3.9997 - val_classification_output_accuracy: 0.3506 - val_classification_output_loss: 6.1876 - val_index_output_accuracy: 0.3506 - val_index_output_loss: 6.4157 - val_loss: 12.6247
Epoch 2/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - classification_output_accuracy: 0.8210 - classification_output_loss: 0.5659 - index_output_accuracy: 0.8447 - index_output_loss: 0.5110 - loss: 1.0768 - val_classification_output_accuracy: 0.5489 - val_classification_output_loss: 1.4586 - val_index_output_accuracy: 0.5546 - val_index_output_loss: 1.6334 - val_loss: 3.0989
Epoch 3/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - classification_output_accuracy: 0.8930 - classification_output_loss: 0.3292 - index_output_accuracy: 0.9415 - index_output_loss: 0.2059 - loss: 0.5351 - val_classification_output_accurac

In [12]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (6, 6, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_6x6_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_6x6_ea.csv')
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X, Y_classification, Y_index = [], [], []
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path) / 255.0
            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)
    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accs, precs, recs, f1s, idx_accs = [], [], [], [], []
start_time = time.time()
fold = 1

for train_index, test_index in skf.split(X, Y_classification):
    X_train, X_test = X[train_index], X[test_index]
    Y_train_classification, Y_test_classification = Y_classification[train_index], Y_classification[test_index]
    Y_train_index, Y_test_index = Y_index[train_index], Y_index[test_index]

    input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))
    x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)

    classification_output = Dense(20, activation='softmax', name='classification_output')(x)
    index_output = Dense(37, activation='softmax', name='index_output')(x)

    model = Model(inputs=input_layer, outputs=[classification_output, index_output])
    model.compile(optimizer='adam',
                  loss={'classification_output': 'sparse_categorical_crossentropy',
                        'index_output': 'sparse_categorical_crossentropy'},
                  metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(X_train,
              {'classification_output': Y_train_classification, 'index_output': Y_train_index},
              epochs=FAC_EPOCHS,
              batch_size=FAC_BATCH_SIZE,
              validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
              callbacks=[early_stopping],
              verbose=0)

    predictions = model.predict(X_test, verbose=0)
    Y_pred_classification = np.argmax(predictions[0], axis=1)
    Y_pred_index = np.argmax(predictions[1], axis=1)

    accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
    precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    accuracy_index = accuracy_score(Y_test_index, Y_pred_index)

    print(f"Fold {fold} - Acc: {accuracy_classification:.4f}, Prec: {precision_classification:.4f}, Rec: {recall_classification:.4f}, F1: {f1_classification:.4f}, Index Acc: {accuracy_index:.4f}")

    accs.append(accuracy_classification)
    precs.append(precision_classification)
    recs.append(recall_classification)
    f1s.append(f1_classification)
    idx_accs.append(accuracy_index)
    fold += 1

print("\nCross Validation Results:")
print(f"Classification Accuracy: Mean={np.mean(accs):.4f}, Std={np.std(accs):.4f}")
print(f"Precision: Mean={np.mean(precs):.4f}, Std={np.std(precs):.4f}")
print(f"Recall: Mean={np.mean(recs):.4f}, Std={np.std(recs):.4f}")
print(f"F1-score: Mean={np.mean(f1s):.4f}, Std={np.std(f1s):.4f}")
print(f"Misbehavior Index Accuracy: Mean={np.mean(idx_accs):.4f}, Std={np.std(idx_accs):.4f}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Fold 1 - Acc: 0.8132, Prec: 0.7843, Rec: 0.7314, F1: 0.7260, Index Acc: 0.8701
Fold 2 - Acc: 0.8454, Prec: 0.7817, Rec: 0.7773, F1: 0.7632, Index Acc: 0.9144
Fold 3 - Acc: 0.8626, Prec: 0.8352, Rec: 0.8031, F1: 0.8051, Index Acc: 0.9132
Fold 4 - Acc: 0.8213, Prec: 0.7779, Rec: 0.7408, F1: 0.7356, Index Acc: 0.8776
Fold 5 - Acc: 0.8420, Prec: 0.8012, Rec: 0.7731, F1: 0.7638, Index Acc: 0.9132

Cross Validation Results:
Classification Accuracy: Mean=0.8369, Std=0.0177
Precision: Mean=0.7961, Std=0.0211
Recall: Mean=0.7651, Std=0.0260
F1-score: Mean=0.7587, Std=0.0276
Misbehavior Index Accuracy: Mean=0.8977, Std=0.0196

Training time: 253.667027 seconds


In [50]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_6x6_ea'
FAC_LABELS_CSV = 'multiple_class_6x6_ea.csv'
FAC_IMAGE_SIZE = (6, 6, 12)

def preprocess_image(file_path):
    """
    Load and preprocess an image for prediction.
    Expects .npy files containing image arrays.
    """
    image_array = np.load(file_path) / 255.0
    return np.expand_dims(image_array, axis=0)

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

random_index = np.random.choice(valid_labels.index)
selected_label = valid_labels.loc[random_index]
selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

start_time = time.time()
image_array = preprocess_image(selected_image_path)

predictions = model.predict(image_array)
predicted_classification = np.argmax(predictions[0])
predicted_index = np.argmax(predictions[1])
end_time = time.time()

print(f"Selected Image Path: {selected_image_path}")
print(f"True Classification: {selected_label['classification']}, True Misbehavior Index: {selected_label['misbehavior_index']}")
print(f"Predicted Classification: {predicted_classification}, Predicted Misbehavior Index: {predicted_index}")
print(f"\nElapsed time: {end_time - start_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Selected Image Path: veremi_multilevel_images_6x6_ea\image_3562.npy
True Classification: 2, True Misbehavior Index: 34
Predicted Classification: 2, Predicted Misbehavior Index: 34

Elapsed time: 0.057880 seconds


In [51]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_6x6_ea'
FAC_LABELS_CSV = 'multiple_class_6x6_ea.csv'
FAC_IMAGE_SIZE = (6, 6)
NUM_SAMPLES = 500
TIME_THRESHOLD = 0.1

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

def preprocess_image(file_path, target_size=(6, 6)):
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

total_classification_correct = 0
total_index_correct = 0
total_prediction_time = 0.0
predictions_above_threshold = 0
total_samples_processed = 0

true_classifications = []
predicted_classifications = []
true_indices = []
predicted_indices = []

for _ in range(NUM_SAMPLES):
    random_index = np.random.choice(valid_labels.index)
    selected_label = valid_labels.loc[random_index]
    selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

    if not os.path.exists(selected_image_path):
        continue

    try:
        start_time = time.time()
        image_array = preprocess_image(selected_image_path)
        image_array = np.expand_dims(image_array, axis=0)

        predictions = model.predict(image_array)
        predicted_classification = np.argmax(predictions[0])
        predicted_index = np.argmax(predictions[1])

        true_classifications.append(int(selected_label['classification']))
        predicted_classifications.append(predicted_classification)
        true_indices.append(int(selected_label['misbehavior_index']))
        predicted_indices.append(predicted_index)

        if predicted_classification == int(selected_label['classification']):
            total_classification_correct += 1
        if predicted_index == int(selected_label['misbehavior_index']):
            total_index_correct += 1

        prediction_time = time.time() - start_time
        total_prediction_time += prediction_time

        if prediction_time >= TIME_THRESHOLD:
            predictions_above_threshold += 1

        total_samples_processed += 1

    except Exception as e:
        print(f"Error during prediction: {e}")

classification_cm = confusion_matrix(true_classifications, predicted_classifications)
index_cm = confusion_matrix(true_indices, predicted_indices)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    total = np.sum(cm)

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    accuracy = np.sum(tp) / total if total > 0 else 0.0

    return accuracy, precision, recall, f1

classification_accuracy, classification_precision, classification_recall, classification_f1 = calculate_metrics(classification_cm)
index_accuracy, index_precision, index_recall, index_f1 = calculate_metrics(index_cm)

print("\nClassification Metrics:")
print(f"Accuracy: {classification_accuracy:.2f}")
print(f"Average Precision: {np.mean(classification_precision):.2f}")
print(f"Average Recall: {np.mean(classification_recall):.2f}")
print(f"Average F1-Score: {np.mean(classification_f1):.2f}")

print("\nIndex Prediction Metrics:")
print(f"Accuracy: {index_accuracy:.2f}")
print(f"Average Precision: {np.mean(index_precision):.2f}")
print(f"Average Recall: {np.mean(index_recall):.2f}")
print(f"Average F1-Score: {np.mean(index_f1):.2f}")

print(f"\nAverage Prediction Time: {total_prediction_time / total_samples_processed:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━

In [52]:
import numpy as np
import pandas as pd
import os

# Constants
FAC_IMAGE_WIDTH = 5
FAC_IMAGE_HEIGHT = 6
FAC_NUM_CHANNELS = 12
FAC_SPECIAL_VALUE = 255
NUM_CLASSES = 20

def normalize_and_scale(values, min_values, max_values):
    scaled = 255 * (values - min_values) / (max_values - min_values)
    scaled[np.isnan(values) | ~np.isfinite(values)] = FAC_SPECIAL_VALUE
    scaled[values == max_values] = 0
    return np.clip(scaled, 0, 255).astype(np.uint8)

def generate_image(group, misbehavior_index=None):
    min_values = np.array([group[col].min() for col in final_numeric_columns])
    max_values = np.array([group[col].max() for col in final_numeric_columns])

    data = group[final_numeric_columns].to_numpy()
    normalized_data = normalize_and_scale(data, min_values, max_values)

    reshaped_data = normalized_data.reshape(FAC_IMAGE_HEIGHT, FAC_IMAGE_WIDTH, FAC_NUM_CHANNELS)

    if misbehavior_index:
        row = (misbehavior_index - 1) // FAC_IMAGE_WIDTH
        col = (misbehavior_index - 1) % FAC_IMAGE_WIDTH
        reshaped_data[row, col, :3] = [255, 0, 0]
        
    return reshaped_data

final_numeric_columns = [
    'sendTime', 'senderPseudo', 'posx', 'posy', 'posx_n', 'posy_n',
    'spdx_n', 'spdy_n', 'aclx_n', 'acly_n', 'hedx_n', 'hedy_n'
]

output_dir = 'veremi_multilevel_images_5x6_ea'
os.makedirs(output_dir, exist_ok=True)

normal_messages = final_df[final_df['class'] == 0]

classification_results = []
image_counter = 1

for _ in range(3000):
    sampled_group = normal_messages.sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
    image_data = generate_image(sampled_group)
    npy_name = f'{output_dir}/image_{image_counter}.npy'
    np.save(npy_name, image_data)
    classification_results.append({'file_name': npy_name, 'classification': 0, 'misbehavior_index': 0})
    image_counter += 1

for misbehavior_class in range(1, 20):
    for _ in range(300):
        attack_group = final_df[final_df['class'] == misbehavior_class].sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
        misbehavior_index = np.random.randint(1, FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT + 1)
        image_data = generate_image(attack_group, misbehavior_index)
        npy_name = f'{output_dir}/image_{image_counter}.npy'
        np.save(npy_name, image_data)
        classification_results.append({'file_name': npy_name, 'classification': misbehavior_class, 'misbehavior_index': misbehavior_index})
        image_counter += 1

classification_df = pd.DataFrame(classification_results)
classification_df.to_csv('multiple_class_5x6_ea.csv', index=False)

print("Images generated and saved to 'multiple_class_5x6_ea.csv'.")

Images generated and saved to 'multiple_class_5x6_ea.csv'.


In [21]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (6, 5, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_5x6_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_5x6_ea.csv')

fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X = []
    Y_classification = []
    Y_index = []
    
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path)
            image_array = image_array / 255.0

            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)

    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

X_train, X_test, Y_train_classification, Y_test_classification, Y_train_index, Y_test_index = train_test_split(
    X, Y_classification, Y_index, test_size=0.2, random_state=42)
input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))

x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

classification_output = Dense(20, activation='softmax', name='classification_output')(x)

index_output = Dense(31, activation='softmax', name='index_output')(x)

model = Model(inputs=input_layer, outputs=[classification_output, index_output])

model.compile(optimizer='adam', 
              loss={'classification_output': 'sparse_categorical_crossentropy', 
                    'index_output': 'sparse_categorical_crossentropy'},
              metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, 
          {'classification_output': Y_train_classification, 'index_output': Y_train_index}, 
          epochs=FAC_EPOCHS, 
          batch_size=FAC_BATCH_SIZE, 
          validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
          callbacks=[early_stopping])

predictions = model.predict(X_test)
Y_pred_classification = np.argmax(predictions[0], axis=1)
Y_pred_index = np.argmax(predictions[1], axis=1)

accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)

accuracy_index = accuracy_score(Y_test_index, Y_pred_index)
f1_classification_manual = 2 * (precision_classification * recall_classification) / (precision_classification + recall_classification)

print("Classification Accuracy:", accuracy_classification)
print("Classification Precision:", precision_classification)
print("Classification Recall:", recall_classification)
print("Classification F1-score (sklearn):", f1_classification)
print("Classification F1-score (manual calculation):", f1_classification_manual)

print("Misbehavior Index Accuracy:", accuracy_index)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

save_dir = "models"
os.makedirs(save_dir, exist_ok=True)

model_save_path = os.path.join(save_dir, "veremi_images_m_5x6_ea_i.keras")
model.save(model_save_path)

print(f"\nModel saved at: {model_save_path}")

Epoch 1/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 9s 25ms/step - classification_output_accuracy: 0.5404 - classification_output_loss: 1.5483 - index_output_accuracy: 0.4098 - index_output_loss: 2.3726 - loss: 3.9209 - val_classification_output_accuracy: 0.4287 - val_classification_output_loss: 2.8435 - val_index_output_accuracy: 0.3529 - val_index_output_loss: 3.0066 - val_loss: 5.8577
Epoch 2/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - classification_output_accuracy: 0.7822 - classification_output_loss: 0.6754 - index_output_accuracy: 0.8612 - index_output_loss: 0.4418 - loss: 1.1171 - val_classification_output_accuracy: 0.5822 - val_classification_output_loss: 1.5939 - val_index_output_accuracy: 0.7529 - val_index_output_loss: 0.8304 - val_loss: 2.4250
Epoch 3/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - classification_output_accuracy: 0.8663 - classification_output_loss: 0.4123 - index_output_accuracy: 0.9349 - index_output_loss: 0.2143 - loss: 0.6266 - val_classification_output_accuracy

In [13]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (6, 5, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_5x6_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_5x6_ea.csv')
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X, Y_classification, Y_index = [], [], []
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path) / 255.0
            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)
    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accs, precs, recs, f1s, idx_accs = [], [], [], [], []
start_time = time.time()
fold = 1

for train_index, test_index in skf.split(X, Y_classification):
    X_train, X_test = X[train_index], X[test_index]
    Y_train_classification, Y_test_classification = Y_classification[train_index], Y_classification[test_index]
    Y_train_index, Y_test_index = Y_index[train_index], Y_index[test_index]

    input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))
    x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)

    classification_output = Dense(20, activation='softmax', name='classification_output')(x)
    index_output = Dense(31, activation='softmax', name='index_output')(x)

    model = Model(inputs=input_layer, outputs=[classification_output, index_output])
    model.compile(optimizer='adam',
                  loss={'classification_output': 'sparse_categorical_crossentropy',
                        'index_output': 'sparse_categorical_crossentropy'},
                  metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(X_train,
              {'classification_output': Y_train_classification, 'index_output': Y_train_index},
              epochs=FAC_EPOCHS,
              batch_size=FAC_BATCH_SIZE,
              validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
              callbacks=[early_stopping],
              verbose=0)

    predictions = model.predict(X_test, verbose=0)
    Y_pred_classification = np.argmax(predictions[0], axis=1)
    Y_pred_index = np.argmax(predictions[1], axis=1)

    accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
    precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    accuracy_index = accuracy_score(Y_test_index, Y_pred_index)

    print(f"Fold {fold} - Acc: {accuracy_classification:.4f}, Prec: {precision_classification:.4f}, Rec: {recall_classification:.4f}, F1: {f1_classification:.4f}, Index Acc: {accuracy_index:.4f}")

    accs.append(accuracy_classification)
    precs.append(precision_classification)
    recs.append(recall_classification)
    f1s.append(f1_classification)
    idx_accs.append(accuracy_index)
    fold += 1

print("\nCross Validation Results:")
print(f"Classification Accuracy: Mean={np.mean(accs):.4f}, Std={np.std(accs):.4f}")
print(f"Precision: Mean={np.mean(precs):.4f}, Std={np.std(precs):.4f}")
print(f"Recall: Mean={np.mean(recs):.4f}, Std={np.std(recs):.4f}")
print(f"F1-score: Mean={np.mean(f1s):.4f}, Std={np.std(f1s):.4f}")
print(f"Misbehavior Index Accuracy: Mean={np.mean(idx_accs):.4f}, Std={np.std(idx_accs):.4f}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Fold 1 - Acc: 0.8103, Prec: 0.7508, Rec: 0.7408, F1: 0.7270, Index Acc: 0.9057
Fold 2 - Acc: 0.8287, Prec: 0.7651, Rec: 0.7524, F1: 0.7480, Index Acc: 0.8891
Fold 3 - Acc: 0.7621, Prec: 0.7195, Rec: 0.6648, F1: 0.6633, Index Acc: 0.8529
Fold 4 - Acc: 0.8126, Prec: 0.7840, Rec: 0.7306, F1: 0.7345, Index Acc: 0.8787
Fold 5 - Acc: 0.7983, Prec: 0.7513, Rec: 0.7120, F1: 0.7119, Index Acc: 0.8667

Cross Validation Results:
Classification Accuracy: Mean=0.8024, Std=0.0224
Precision: Mean=0.7542, Std=0.0211
Recall: Mean=0.7201, Std=0.0307
F1-score: Mean=0.7170, Std=0.0292
Misbehavior Index Accuracy: Mean=0.8786, Std=0.0182

Training time: 228.984427 seconds


In [64]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_5x6_ea'
FAC_LABELS_CSV = 'multiple_class_5x6_ea.csv'
FAC_IMAGE_SIZE = (6, 5, 12)

def preprocess_image(file_path):
    """
    Load and preprocess an image for prediction.
    Expects .npy files containing image arrays.
    """
    image_array = np.load(file_path) / 255.0
    return np.expand_dims(image_array, axis=0)

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

random_index = np.random.choice(valid_labels.index)
selected_label = valid_labels.loc[random_index]
selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

start_time = time.time()
image_array = preprocess_image(selected_image_path)

predictions = model.predict(image_array)
predicted_classification = np.argmax(predictions[0])
predicted_index = np.argmax(predictions[1])
end_time = time.time()

print(f"Selected Image Path: {selected_image_path}")
print(f"True Classification: {selected_label['classification']}, True Misbehavior Index: {selected_label['misbehavior_index']}")
print(f"Predicted Classification: {predicted_classification}, Predicted Misbehavior Index: {predicted_index}")
print(f"\nElapsed time: {end_time - start_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Selected Image Path: veremi_multilevel_images_5x6_ea\image_328.npy
True Classification: 0, True Misbehavior Index: 0
Predicted Classification: 0, Predicted Misbehavior Index: 0

Elapsed time: 0.069336 seconds


In [66]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_5x6_ea'
FAC_LABELS_CSV = 'multiple_class_5x6_ea.csv'
FAC_IMAGE_SIZE = (6, 5)
NUM_SAMPLES = 500
TIME_THRESHOLD = 0.1

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

def preprocess_image(file_path, target_size=(6, 5)):
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

total_classification_correct = 0
total_index_correct = 0
total_prediction_time = 0.0
predictions_above_threshold = 0
total_samples_processed = 0

true_classifications = []
predicted_classifications = []
true_indices = []
predicted_indices = []

for _ in range(NUM_SAMPLES):
    random_index = np.random.choice(valid_labels.index)
    selected_label = valid_labels.loc[random_index]
    selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

    if not os.path.exists(selected_image_path):
        continue

    try:
        start_time = time.time()
        image_array = preprocess_image(selected_image_path)
        image_array = np.expand_dims(image_array, axis=0)

        predictions = model.predict(image_array)
        predicted_classification = np.argmax(predictions[0])
        predicted_index = np.argmax(predictions[1])

        true_classifications.append(int(selected_label['classification']))
        predicted_classifications.append(predicted_classification)
        true_indices.append(int(selected_label['misbehavior_index']))
        predicted_indices.append(predicted_index)

        if predicted_classification == int(selected_label['classification']):
            total_classification_correct += 1
        if predicted_index == int(selected_label['misbehavior_index']):
            total_index_correct += 1

        prediction_time = time.time() - start_time
        total_prediction_time += prediction_time

        if prediction_time >= TIME_THRESHOLD:
            predictions_above_threshold += 1

        total_samples_processed += 1

    except Exception as e:
        print(f"Error during prediction: {e}")

classification_cm = confusion_matrix(true_classifications, predicted_classifications)
index_cm = confusion_matrix(true_indices, predicted_indices)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    total = np.sum(cm)

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    accuracy = np.sum(tp) / total if total > 0 else 0.0

    return accuracy, precision, recall, f1

classification_accuracy, classification_precision, classification_recall, classification_f1 = calculate_metrics(classification_cm)
index_accuracy, index_precision, index_recall, index_f1 = calculate_metrics(index_cm)

print("\nClassification Metrics:")
print(f"Accuracy: {classification_accuracy:.2f}")
print(f"Average Precision: {np.mean(classification_precision):.2f}")
print(f"Average Recall: {np.mean(classification_recall):.2f}")
print(f"Average F1-Score: {np.mean(classification_f1):.2f}")

print("\nIndex Prediction Metrics:")
print(f"Accuracy: {index_accuracy:.2f}")
print(f"Average Precision: {np.mean(index_precision):.2f}")
print(f"Average Recall: {np.mean(index_recall):.2f}")
print(f"Average F1-Score: {np.mean(index_f1):.2f}")

print(f"\nAverage Prediction Time: {total_prediction_time / total_samples_processed:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━

In [67]:
import numpy as np
import pandas as pd
import os

# Constants
FAC_IMAGE_WIDTH = 7
FAC_IMAGE_HEIGHT = 7
FAC_NUM_CHANNELS = 12
FAC_SPECIAL_VALUE = 255
NUM_CLASSES = 20

def normalize_and_scale(values, min_values, max_values):
    scaled = 255 * (values - min_values) / (max_values - min_values)
    scaled[np.isnan(values) | ~np.isfinite(values)] = FAC_SPECIAL_VALUE
    scaled[values == max_values] = 0
    return np.clip(scaled, 0, 255).astype(np.uint8)

def generate_image(group, misbehavior_index=None):
    min_values = np.array([group[col].min() for col in final_numeric_columns])
    max_values = np.array([group[col].max() for col in final_numeric_columns])

    data = group[final_numeric_columns].to_numpy()
    normalized_data = normalize_and_scale(data, min_values, max_values)

    reshaped_data = normalized_data.reshape(FAC_IMAGE_HEIGHT, FAC_IMAGE_WIDTH, FAC_NUM_CHANNELS)

    if misbehavior_index:
        row = (misbehavior_index - 1) // FAC_IMAGE_WIDTH
        col = (misbehavior_index - 1) % FAC_IMAGE_WIDTH
        reshaped_data[row, col, :3] = [255, 0, 0]
        
    return reshaped_data

final_numeric_columns = [
    'sendTime', 'senderPseudo', 'posx', 'posy', 'posx_n', 'posy_n',
    'spdx_n', 'spdy_n', 'aclx_n', 'acly_n', 'hedx_n', 'hedy_n'
]

output_dir = 'veremi_multilevel_images_7x7_ea'
os.makedirs(output_dir, exist_ok=True)

normal_messages = final_df[final_df['class'] == 0]

classification_results = []
image_counter = 1

for _ in range(3000):
    sampled_group = normal_messages.sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
    image_data = generate_image(sampled_group)
    npy_name = f'{output_dir}/image_{image_counter}.npy'
    np.save(npy_name, image_data)
    classification_results.append({'file_name': npy_name, 'classification': 0, 'misbehavior_index': 0})
    image_counter += 1

for misbehavior_class in range(1, 20):
    for _ in range(300):
        attack_group = final_df[final_df['class'] == misbehavior_class].sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
        misbehavior_index = np.random.randint(1, FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT + 1)
        image_data = generate_image(attack_group, misbehavior_index)
        npy_name = f'{output_dir}/image_{image_counter}.npy'
        np.save(npy_name, image_data)
        classification_results.append({'file_name': npy_name, 'classification': misbehavior_class, 'misbehavior_index': misbehavior_index})
        image_counter += 1

classification_df = pd.DataFrame(classification_results)
classification_df.to_csv('multiple_class_7x7_ea.csv', index=False)

print("Images generated and saved to 'multiple_class_7x7_ea.csv'.")

Images generated and saved to 'multiple_class_7x7_ea.csv'.


In [22]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (7, 7, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_7x7_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_7x7_ea.csv')

fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X = []
    Y_classification = []
    Y_index = []
    
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path)
            image_array = image_array / 255.0

            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)

    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

X_train, X_test, Y_train_classification, Y_test_classification, Y_train_index, Y_test_index = train_test_split(
    X, Y_classification, Y_index, test_size=0.2, random_state=42)
input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))

x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

classification_output = Dense(20, activation='softmax', name='classification_output')(x)

index_output = Dense(50, activation='softmax', name='index_output')(x)

model = Model(inputs=input_layer, outputs=[classification_output, index_output])

model.compile(optimizer='adam', 
              loss={'classification_output': 'sparse_categorical_crossentropy', 
                    'index_output': 'sparse_categorical_crossentropy'},
              metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, 
          {'classification_output': Y_train_classification, 'index_output': Y_train_index}, 
          epochs=FAC_EPOCHS, 
          batch_size=FAC_BATCH_SIZE, 
          validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
          callbacks=[early_stopping])

predictions = model.predict(X_test)
Y_pred_classification = np.argmax(predictions[0], axis=1)
Y_pred_index = np.argmax(predictions[1], axis=1)

accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)

accuracy_index = accuracy_score(Y_test_index, Y_pred_index)
f1_classification_manual = 2 * (precision_classification * recall_classification) / (precision_classification + recall_classification)

print("Classification Accuracy:", accuracy_classification)
print("Classification Precision:", precision_classification)
print("Classification Recall:", recall_classification)
print("Classification F1-score (sklearn):", f1_classification)
print("Classification F1-score (manual calculation):", f1_classification_manual)

print("Misbehavior Index Accuracy:", accuracy_index)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

save_dir = "models"
os.makedirs(save_dir, exist_ok=True)

model_save_path = os.path.join(save_dir, "veremi_images_m_7x7_ea_i.keras")
model.save(model_save_path)

print(f"\nModel saved at: {model_save_path}")

Epoch 1/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms/step - classification_output_accuracy: 0.6303 - classification_output_loss: 1.2381 - index_output_accuracy: 0.3505 - index_output_loss: 2.8653 - loss: 4.1035 - val_classification_output_accuracy: 0.3506 - val_classification_output_loss: 4.1781 - val_index_output_accuracy: 0.3506 - val_index_output_loss: 5.8877 - val_loss: 10.0753
Epoch 2/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - classification_output_accuracy: 0.8357 - classification_output_loss: 0.5008 - index_output_accuracy: 0.8129 - index_output_loss: 0.6562 - loss: 1.1570 - val_classification_output_accuracy: 0.5500 - val_classification_output_loss: 1.7124 - val_index_output_accuracy: 0.5816 - val_index_output_loss: 1.8283 - val_loss: 3.5495
Epoch 3/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 7s 33ms/step - classification_output_accuracy: 0.9215 - classification_output_loss: 0.2434 - index_output_accuracy: 0.9446 - index_output_loss: 0.2110 - loss: 0.4544 - val_classification_output_accura

In [6]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (7, 7, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_7x7_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_7x7_ea.csv')
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X, Y_classification, Y_index = [], [], []
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path) / 255.0
            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)
    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accs, precs, recs, f1s, idx_accs = [], [], [], [], []
start_time = time.time()
fold = 1

for train_index, test_index in skf.split(X, Y_classification):
    X_train, X_test = X[train_index], X[test_index]
    Y_train_classification, Y_test_classification = Y_classification[train_index], Y_classification[test_index]
    Y_train_index, Y_test_index = Y_index[train_index], Y_index[test_index]

    input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))
    x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)

    classification_output = Dense(20, activation='softmax', name='classification_output')(x)
    index_output = Dense(50, activation='softmax', name='index_output')(x)

    model = Model(inputs=input_layer, outputs=[classification_output, index_output])
    model.compile(optimizer='adam',
                  loss={'classification_output': 'sparse_categorical_crossentropy',
                        'index_output': 'sparse_categorical_crossentropy'},
                  metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(X_train,
              {'classification_output': Y_train_classification, 'index_output': Y_train_index},
              epochs=FAC_EPOCHS,
              batch_size=FAC_BATCH_SIZE,
              validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
              callbacks=[early_stopping],
              verbose=0)

    predictions = model.predict(X_test, verbose=0)
    Y_pred_classification = np.argmax(predictions[0], axis=1)
    Y_pred_index = np.argmax(predictions[1], axis=1)

    accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
    precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
    accuracy_index = accuracy_score(Y_test_index, Y_pred_index)

    print(f"Fold {fold} - Acc: {accuracy_classification:.4f}, Prec: {precision_classification:.4f}, Rec: {recall_classification:.4f}, F1: {f1_classification:.4f}, Index Acc: {accuracy_index:.4f}")

    accs.append(accuracy_classification)
    precs.append(precision_classification)
    recs.append(recall_classification)
    f1s.append(f1_classification)
    idx_accs.append(accuracy_index)
    fold += 1

print("\nCross Validation Results:")
print(f"Classification Accuracy: Mean={np.mean(accs):.4f}, Std={np.std(accs):.4f}")
print(f"Precision: Mean={np.mean(precs):.4f}, Std={np.std(precs):.4f}")
print(f"Recall: Mean={np.mean(recs):.4f}, Std={np.std(recs):.4f}")
print(f"F1-score: Mean={np.mean(f1s):.4f}, Std={np.std(f1s):.4f}")
print(f"Misbehavior Index Accuracy: Mean={np.mean(idx_accs):.4f}, Std={np.std(idx_accs):.4f}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Fold 1 - Acc: 0.9040, Prec: 0.8798, Rec: 0.8608, F1: 0.8620, Index Acc: 0.8707
Fold 2 - Acc: 0.8920, Prec: 0.8834, Rec: 0.8433, F1: 0.8452, Index Acc: 0.8828
Fold 3 - Acc: 0.9293, Prec: 0.9070, Rec: 0.8975, F1: 0.8970, Index Acc: 0.8931
Fold 4 - Acc: 0.8776, Prec: 0.8621, Rec: 0.8233, F1: 0.8172, Index Acc: 0.8454
Fold 5 - Acc: 0.8603, Prec: 0.8483, Rec: 0.8035, F1: 0.8016, Index Acc: 0.8557

Cross Validation Results:
Classification Accuracy: Mean=0.8926, Std=0.0234
Precision: Mean=0.8761, Std=0.0200
Recall: Mean=0.8457, Std=0.0323
F1-score: Mean=0.8446, Std=0.0336
Misbehavior Index Accuracy: Mean=0.8695, Std=0.0173

Training time: 345.141367 seconds


In [4]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_7x7_ea'
FAC_LABELS_CSV = 'multiple_class_7x7_ea.csv'
FAC_IMAGE_SIZE = (7, 7, 12)

def preprocess_image(file_path):
    """
    Load and preprocess an image for prediction.
    Expects .npy files containing image arrays.
    """
    image_array = np.load(file_path) / 255.0
    return np.expand_dims(image_array, axis=0)

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

random_index = np.random.choice(valid_labels.index)
selected_label = valid_labels.loc[random_index]
selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

start_time = time.time()
image_array = preprocess_image(selected_image_path)

predictions = model.predict(image_array)
predicted_classification = np.argmax(predictions[0])
predicted_index = np.argmax(predictions[1])
end_time = time.time()

print(f"Selected Image Path: {selected_image_path}")
print(f"True Classification: {selected_label['classification']}, True Misbehavior Index: {selected_label['misbehavior_index']}")
print(f"Predicted Classification: {predicted_classification}, Predicted Misbehavior Index: {predicted_index}")
print(f"\nElapsed time: {end_time - start_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
Selected Image Path: veremi_multilevel_images_7x7_ea\image_7019.npy
True Classification: 14, True Misbehavior Index: 28
Predicted Classification: 10, Predicted Misbehavior Index: 19

Elapsed time: 0.164445 seconds


In [80]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_7x7_ea'
FAC_LABELS_CSV = 'multiple_class_7x7_ea.csv'
FAC_IMAGE_SIZE = (7, 7)
NUM_SAMPLES = 500
TIME_THRESHOLD = 0.1

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

def preprocess_image(file_path, target_size=(7, 7)):
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

total_classification_correct = 0
total_index_correct = 0
total_prediction_time = 0.0
predictions_above_threshold = 0
total_samples_processed = 0

true_classifications = []
predicted_classifications = []
true_indices = []
predicted_indices = []

for _ in range(NUM_SAMPLES):
    random_index = np.random.choice(valid_labels.index)
    selected_label = valid_labels.loc[random_index]
    selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

    if not os.path.exists(selected_image_path):
        continue

    try:
        start_time = time.time()
        image_array = preprocess_image(selected_image_path)
        image_array = np.expand_dims(image_array, axis=0)

        predictions = model.predict(image_array)
        predicted_classification = np.argmax(predictions[0])
        predicted_index = np.argmax(predictions[1])

        true_classifications.append(int(selected_label['classification']))
        predicted_classifications.append(predicted_classification)
        true_indices.append(int(selected_label['misbehavior_index']))
        predicted_indices.append(predicted_index)

        if predicted_classification == int(selected_label['classification']):
            total_classification_correct += 1
        if predicted_index == int(selected_label['misbehavior_index']):
            total_index_correct += 1

        prediction_time = time.time() - start_time
        total_prediction_time += prediction_time

        if prediction_time >= TIME_THRESHOLD:
            predictions_above_threshold += 1

        total_samples_processed += 1

    except Exception as e:
        print(f"Error during prediction: {e}")

classification_cm = confusion_matrix(true_classifications, predicted_classifications)
index_cm = confusion_matrix(true_indices, predicted_indices)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    total = np.sum(cm)

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    accuracy = np.sum(tp) / total if total > 0 else 0.0

    return accuracy, precision, recall, f1

classification_accuracy, classification_precision, classification_recall, classification_f1 = calculate_metrics(classification_cm)
index_accuracy, index_precision, index_recall, index_f1 = calculate_metrics(index_cm)

print("\nClassification Metrics:")
print(f"Accuracy: {classification_accuracy:.2f}")
print(f"Average Precision: {np.mean(classification_precision):.2f}")
print(f"Average Recall: {np.mean(classification_recall):.2f}")
print(f"Average F1-Score: {np.mean(classification_f1):.2f}")

print("\nIndex Prediction Metrics:")
print(f"Accuracy: {index_accuracy:.2f}")
print(f"Average Precision: {np.mean(index_precision):.2f}")
print(f"Average Recall: {np.mean(index_recall):.2f}")
print(f"Average F1-Score: {np.mean(index_f1):.2f}")

print(f"\nAverage Prediction Time: {total_prediction_time / total_samples_processed:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━